# $\color{red}{\text{Kapittel 1 - Grunnleggende regning}}$

In [1]:
import sys
print(sys.version)

3.14.0 (tags/v3.14.0:ebf955d, Oct  7 2025, 10:15:03) [MSC v.1944 64 bit (AMD64)]


In [ ]:
# 1 Grunnleggende regning: 1.1 Hoderegning
# Addisjon, subtraksjon, multiplikasjon og divisjon

# Addisjon
addition_1 = 70 + 50
addition_2 = 700 + 500

# Subtraksjon
subtraction_1 = 120 - 50
subtraction_2 = 1200 - 500

# Multiplikasjon
multiplication_1 = 42.35 * 10
multiplication_2 = 348 * 1000

# Divisjon
division_1 = 225 / 100
division_2 = 7.75 / 1000

# Utskrift av resultatene med avrunding :.0f betyr feks ingen desimaler, :.1f betyr 1 desimal osv
print(f"70 kr + 50 kr = {addition_1:.0f} kr")
print(f"700 kr + 500 kr = {addition_2:.0f} kr")
print(f"120 kr - 50 kr = {subtraction_1:.0f} kr")
print(f"1200 kr - 500 kr = {subtraction_2:.0f} kr")
print(f"42.35 * 10 = {multiplication_1:.1f}")
print(f"348 * 1000 = {multiplication_2:.0f}")
print(f"225 / 100 = {division_1:.2f}")
print(f"7.75 / 1000 = {division_2:.5f}")

In [ ]:
# 1 Grunnleggende regning: 1.3 Overslagsregning
import math

def round_up(number, level):
    return math.ceil(number / level) * level

def round_down(number, level):
    return math.floor(number / level) * level

def round_nearest(number, level):
    return round(number / level) * level

def estimate(a, b, operation):
    if operation == '+':
        a_rounded = round_down(a, 10)
        b_rounded = round_up(b, 10)
        result = a_rounded + b_rounded
    elif operation == '-':
        a_rounded = round_up(a, 10)
        b_rounded = round_up(b, 10)
        result = a_rounded - b_rounded
    elif operation == '*':
        a_rounded = round_up(a, 10)
        b_rounded = round_down(b, 10)
        result = a_rounded * b_rounded
    elif operation == '/':
        a_rounded = round_down(a, 10)
        b_rounded = round_down(b, 1)
        result = a_rounded / b_rounded
    else:
        raise ValueError("Ugyldig operasjon")
    
    return a_rounded, b_rounded, result

def run_examples():
    examples = [
        (184.75, 257.20, '+'),
        (657.50, 379.45, '-'),
        (18.5, 26.3, '*'),
        (122, 3.12, '/'),
        (4.8, 14.18, '*'),
        (0.23, 18, '*'),
        (180, 47, '/')
    ]

    for a, b, op in examples:
        a_r, b_r, res = estimate(a, b, op)
        print(f"{a} {op} {b} ≈ {a_r} {op} {b_r} = {round(res)}")

def user_input():
    try:
        a = float(input("Skriv inn det første tallet: "))
        b = float(input("Skriv inn det andre tallet: "))
        op = input("Velg regneoperasjon (+, -, *, /): ")
        a_r, b_r, res = estimate(a, b, op)
        print(f"\nOverslag: {a} {op} {b} ≈ {a_r} {op} {b_r} = {round(res)}")
    except Exception as e:
        print(f"Feil: {e}")

def main():
    while True:
        print("\n--- OVERSLAGSREGNING ---")
        print("1. Kjør eksempler")
        print("2. Gjør egne beregninger")
        print("3. Avslutt")
        valg = input("Velg et alternativ (1-3): ")

        if valg == '1':
            run_examples()
        elif valg == '2':
            user_input()
        elif valg == '3':
            print("Avslutter programmet.")
            break
        else:
            print("Ugyldig valg. Prøv igjen.")

if __name__ == "__main__":
    main()

In [ ]:
# 1 Grunnleggende regning: 1.5 Regne med brøk
import ipywidgets as widgets
from IPython.display import display, clear_output
from fractions import Fraction
import math
import re
import ast
import operator

# GUI-komponenter
num_terms = widgets.IntSlider(value=2, min=2, max=6, description="Antall ledd:")
confirm_button = widgets.Button(description="OK")
input_container = widgets.VBox()
calculate_button = widgets.Button(description="Beregn")
output = widgets.Output()

term_inputs = []
operator_inputs = []

# Evaluer komplekse uttrykk i brøker
def eval_expr(expr):
    expr = expr.replace('^', '**')
    expr = re.sub(r'√(\d+)', r'math.sqrt(\1)', expr)
    expr = expr.replace('sqrt', 'math.sqrt')

    ops = {
        ast.Add: operator.add,
        ast.Sub: operator.sub,
        ast.Mult: operator.mul,
        ast.Div: operator.truediv,
        ast.Pow: operator.pow,
        ast.USub: operator.neg
    }

    def _eval(node):
        if isinstance(node, ast.Expression):
            return _eval(node.body)
        elif isinstance(node, ast.BinOp):
            left = _eval(node.left)
            right = _eval(node.right)
            res = ops[type(node.op)](left, right)
            return Fraction(str(res)).limit_denominator(10**6)
        elif isinstance(node, ast.UnaryOp):
            operand = _eval(node.operand)
            return ops[type(node.op)](operand)
        elif isinstance(node, ast.Call):
            if isinstance(node.func, ast.Attribute) and node.func.attr == 'sqrt':
                val = math.sqrt(float(_eval(node.args[0])))
                return Fraction(str(val)).limit_denominator(10**6)
            else:
                raise TypeError("Ugyldig funksjon")
        elif isinstance(node, ast.Constant):
            return Fraction(str(node.value))
        elif isinstance(node, ast.Num):
            return Fraction(str(node.n))
        else:
            raise TypeError(f"Ugyldig uttrykk: {node}")

    parsed = ast.parse(expr, mode='eval')
    return _eval(parsed.body)

# Opprett felt for ledd
def create_inputs(_):
    global term_inputs, operator_inputs
    term_inputs = []
    operator_inputs = []
    children = []

    for idx in range(num_terms.value):
        term_type = widgets.Dropdown(options=["Heltall", "Desimaltall", "Brøk", "Kompleks brøk"], description=f"Ledd {idx+1}:")
        sign = widgets.Dropdown(options=['+', '-'], description="Fortegn:")

        int_input = widgets.IntText(description="Heltall")
        float_input = widgets.FloatText(description="Desimal")
        br_num = widgets.IntText(description="Teller")
        br_den = widgets.IntText(description="Nevner", value=1)
        k_num_expr = widgets.Text(description="Telleruttrykk")
        k_den_expr = widgets.Text(description="Nevneruttrykk")

        container = widgets.VBox()

        # Lager separat funksjon for å unngå referanseproblem
        def make_updater(container_ref, type_widget, sign_widget, i_widget, f_widget, bn_widget, bd_widget, kn_widget, kd_widget):
            def update_term_fields(change=None):
                if type_widget.value == "Heltall":
                    container_ref.children = [sign_widget, type_widget, i_widget]
                elif type_widget.value == "Desimaltall":
                    container_ref.children = [sign_widget, type_widget, f_widget]
                elif type_widget.value == "Brøk":
                    container_ref.children = [sign_widget, type_widget, bn_widget, bd_widget]
                elif type_widget.value == "Kompleks brøk":
                    container_ref.children = [sign_widget, type_widget, kn_widget, kd_widget]
            return update_term_fields

        updater = make_updater(container, term_type, sign, int_input, float_input, br_num, br_den, k_num_expr, k_den_expr)
        term_type.observe(updater, names="value")
        updater()

        term_inputs.append((sign, term_type, int_input, float_input, br_num, br_den, k_num_expr, k_den_expr))
        children.append(container)

        if idx < num_terms.value - 1:
            op = widgets.Dropdown(options=['+', '-', '*', '/'], description=f"Operator {idx+1}:")
            operator_inputs.append(op)
            children.append(op)

    children.append(calculate_button)
    children.append(output)
    input_container.children = children

# Konverter et ledd til Fraction
def to_fraction(sign, typ, i, f, bn, bd, kn_expr, kd_expr):
    if typ == "Heltall":
        val = Fraction(i)
    elif typ == "Desimaltall":
        val = Fraction(str(f)).limit_denominator()
    elif typ == "Brøk":
        if bd == 0:
            raise ZeroDivisionError("Nevner kan ikke være 0")
        val = Fraction(bn, bd)
    elif typ == "Kompleks brøk":
        teller = eval_expr(kn_expr)
        nevner = eval_expr(kd_expr)
        if nevner == 0:
            raise ZeroDivisionError("Nevner kan ikke være 0")
        val = teller / nevner
    else:
        raise ValueError(f"Ukjent type: {typ}")
    return val if sign == '+' else -val

# Kalkuler hele uttrykket
def calculate(_):
    with output:
        output.clear_output()
        try:
            sign, typ, i, f, bn, bd, kn_expr, kd_expr = term_inputs[0]
            result = to_fraction(sign.value, typ.value, i.value, f.value, bn.value, bd.value, kn_expr.value, kd_expr.value)
            expr_str = f"({result})"

            for idx, op_widget in enumerate(operator_inputs):
                op = op_widget.value
                sign, typ, i, f, bn, bd, kn_expr, kd_expr = term_inputs[idx + 1]
                next_val = to_fraction(sign.value, typ.value, i.value, f.value, bn.value, bd.value, kn_expr.value, kd_expr.value)
                expr_str += f" {op} ({next_val})"

                if op == '+':
                    result += next_val
                elif op == '-':
                    result -= next_val
                elif op == '*':
                    result *= next_val
                elif op == '/':
                    if next_val == 0:
                        raise ZeroDivisionError("Kan ikke dele på null")
                    result /= next_val

            desimal = round(float(result), 2)

            if abs(result.numerator) > result.denominator:
                heltall = result.numerator // result.denominator
                rest = abs(result.numerator) % result.denominator
                blandet = f"{heltall} {rest}/{result.denominator}" if rest else str(heltall)
            else:
                blandet = str(result)

            print(f"Uttrykk: {expr_str}")
            print(f"Forenklet brøk: {result}")
            print(f"Blandet tall: {blandet}")
            print(f"Desimaltall: {desimal}")

        except Exception as e:
            print(f"Feil: {e}")

# Koble knapper
confirm_button.on_click(create_inputs)
calculate_button.on_click(calculate)

# Startvisning
display(widgets.HBox([num_terms, confirm_button]))
display(input_container)

In [ ]:
# Omgjøringskalkulator mellom desimaltall, brøk og prosent
from decimal import Decimal, getcontext
from math import gcd

getcontext().prec = 10  # Sett presisjon for desimaler

def hovedmeny():
    print("Velkommen til omgjøringskalkulator mellom desimaltall, brøk og prosent ")
    print("Trykk 'q' når som helst for å avslutte programmet og trykk enter.\n")
    print("Hva vil du beregne?")
    print("1. Prosent til desimaltall og brøk")
    print("2. Brøk til desimaltall og prosent")
    print("3. Desimaltall til brøk og prosent")

def beregn_prosentandel(delen: float, hele: float) -> float:
    if hele == 0:
        raise ValueError("Hele kan ikke være null.")
    return (delen / hele) * 100

def decimal_to_fraction_and_percent(digits: str):
    try:
        n = Decimal(digits)
    except InvalidOperation:
        print("Ugyldig desimaltall.")
        return

    exponent = len(digits.split('.')[1]) if '.' in digits else 0
    numerator = int(n * 10**exponent)
    denominator = 10**exponent
    percent = float(n * 100)
    factor = gcd(numerator, denominator)
    num = numerator // factor
    den = denominator // factor

    print(f"Desimaltallet er {round(n, 3)}")
    print(f"Brøken er {num} / {den}")
    print(f"Prosenten er {round(percent, 3)}%\n")

def percent_to_decimal_and_fraction(percent: float):
    decimal = percent / 100
    digits = str(decimal)
    decimal_to_fraction_and_percent(digits)

def fraction_to_decimal_and_percent(numerator: int, denominator: int):
    if denominator == 0:
        print("Nevneren kan ikke være null.")
        return
    decimal = Decimal(numerator) / Decimal(denominator)
    percent = float(decimal * 100)

    print(f"Desimaltallet er {round(decimal, 3)}")
    print(f"Prosenten er {round(percent, 3)}%\n")

def main():
    while True:
        hovedmeny()
        choice = input("Velg et alternativ (1/2/3): ").strip().lower()
        if choice == 'q':
            print("Programmet avsluttes.")
            break
        elif choice == '1':
            percent = float(input("Skriv inn prosentverdien: "))
            percent_to_decimal_and_fraction(percent)
        elif choice == '2':
            numerator = int(input("Skriv inn telleren til brøken, altså det øverste tallet: "))
            denominator = int(input("Skriv inn nevneren til brøken, altså det nederste tallet: "))
            fraction_to_decimal_and_percent(numerator, denominator)
        elif choice == '3':
            digits = input("Skriv inn ett desimaltall, husk punktum, for å konvertere til brøk og prosent: ")
            decimal_to_fraction_and_percent(digits)
        else:
            print("Ugyldig valg. Vennligst prøv igjen.")
        
        restart = input("Vil du starte på nytt? (ja/nei): ").strip().lower()
        if restart == 'q':
            print("Programmet avsluttes.")
            break
        elif restart != 'ja':
            break

if __name__ == "__main__":
    main()

In [ ]:
# 1 Grunnleggende regning: 1.6 Prosent
# 1.6 Prosent Finn % av ett tall. Formel: p % av ett tall = p/100 * tallet
while True:
    user_input = input("Skriv inn prosent (eller 'q' for å avslutte): ")
    if user_input.lower() == 'q':
        print("Avslutter programmet.")
        break
    try:
        prosent = float(user_input)
        hele_tallet = float(input("Skriv inn hele tallet: "))
        prosentdelen = prosent / 100 * hele_tallet
        print(f"{prosent}% av {hele_tallet} er {round(prosentdelen, 2)}")
    except ValueError:
        print("Ugyldig input. Vennligst skriv inn tallverdier.")

In [ ]:
# 1 Grunnleggende regning: 1.6 Prosent
# Hvor mange prosent delen av ett tall er av det hele tallet er, feks 10 er ...% av 30
from dataclasses import dataclass

@dataclass
class Prosentandel:
    delen: float
    hele: float

    def beregn(self) -> float:
        if self.hele == 0:
            raise ValueError("Hele kan ikke være null.")
        return (self.delen / self.hele) * 100

def main():
    print("Velkommen til programmet som regner ut hvor mange prosent ett tall er av ett annet tall")
    print("Trykk 'q' når som helst for å avslutte programmet.\n")

    while True:
        delen_input = input("Skriv inn delen av tallet (eller 'q' for å avslutte): ").strip().lower()
        if delen_input == 'q':
            print("Programmet avsluttes.")
            break

        hele_input = input("Skriv inn det hele tallet (eller 'q' for å avslutte): ").strip().lower()
        if hele_input == 'q':
            print("Programmet avsluttes.")
            break

        try:
            delen = float(delen_input)
            hele = float(hele_input)
            prosentandel = Prosentandel(delen=delen, hele=hele)
            prosent = prosentandel.beregn()
            print(f"{prosentandel.delen} er {prosent:.2f}% av {prosentandel.hele}.")
        except ValueError as e:
            print(e)
        except Exception as e:
            print(f"Ugyldig input: {e}")

if __name__ == "__main__":
    main()

# Prosentregning

In [1]:
# Regel 1. Del av tallet = (Hele tallet ∙ Prosenten) / 100
def beregn_hele_tallet(prosent, kroner):
    return kroner * 100 / prosent

def beregn_prosent(hele_tallet, kroner):
    return (kroner / hele_tallet) * 100

def beregn_del_av_tallet(hele_tallet, prosent):
    return hele_tallet * prosent / 100

def hovedprogram():
    print("Velkommen til kalkulatoren for: Del av tallet = (Hele tallet ∙ Prosenten) / 100")
    print("Trykk 'q' når som helst for å avslutte programmet og trykk enter.\n")
    print("Hva ønsker du å beregne?")
    
    print("1: Hele tallet")
    print("2: Prosent")
    print("3: Del av tallet")
    
    while True:
        valg = input("Skriv inn nummeret på det du ønsker å beregne (1, 2, eller 3): ")
        
        if valg.lower() == 'q':
            print("Avslutter programmet. Ha en fin dag!")
            break
        
        if valg == "1":
            try:
                prosent = float(input("Skriv inn prosentverdien: "))
                kroner = float(input("Skriv inn del av hele tallet: "))
                hele_tallet = beregn_hele_tallet(prosent, kroner)
                print(f"Hele tallet er: {hele_tallet:.2f}\n")
            except ValueError:
                print("Ugyldig input. Vennligst skriv inn gyldige tall.")
        
        elif valg == "2":
            try:
                hele_tallet = float(input("Skriv inn hele tallet: "))
                kroner = float(input("Skriv inn del av tallet: "))
                prosent = beregn_prosent(hele_tallet, kroner)
                print(f"Prosentverdien er: {prosent:.2f} %\n")
            except ValueError:
                print("Ugyldig input. Vennligst skriv inn gyldige tall.")
        
        elif valg == "3":
            try:
                hele_tallet = float(input("Skriv inn hele tallet: "))
                prosent = float(input("Skriv inn prosentverdien: "))
                del_av_tallet = beregn_del_av_tallet(hele_tallet, prosent)
                print(f"Del av tallet er: {del_av_tallet:.2f}\n")
            except ValueError:
                print("Ugyldig input. Vennligst skriv inn gyldige tall.")
        
        else:
            print("Ugyldig valg. Vennligst velg 1, 2, eller 3.")

# Kjør hovedprogrammet
hovedprogram()

Velkommen til kalkulatoren for: Del av tallet = (Hele tallet ∙ Prosenten) / 100
Trykk 'q' når som helst for å avslutte programmet og trykk enter.

Hva ønsker du å beregne?
1: Hele tallet
2: Prosent
3: Del av tallet


Skriv inn nummeret på det du ønsker å beregne (1, 2, eller 3):  1
Skriv inn prosentverdien:  20
Skriv inn del av hele tallet:  300


Hele tallet er: 1500.00



Skriv inn nummeret på det du ønsker å beregne (1, 2, eller 3):  q


Avslutter programmet. Ha en fin dag!


In [ ]:
# Finn endringen i prosentpoeng mellom to tall
while True:
    user_input = input("Skriv inn startprosent (eller 'q' for å avslutte): ")
    if user_input.lower() == 'q':
        print("Avslutter programmet.")
        break
    try:
        startprosent = float(user_input)
        sluttprosent = float(input("Skriv inn sluttprosent: "))
        prosentpoeng = sluttprosent - startprosent
        prosent = prosentpoeng / startprosent * 100
        print(f"Endringen i prosentpoeng er: {round(prosentpoeng, 2)}")
        print(f"Endringen i prosent er: {round(prosent, 2)}%")
    except ValueError:
        print("Ugyldig input. Vennligst skriv inn tallverdier.")

In [ ]:
# Regel 2. Endringen i prosent = (Ny verdi – Opprinnelig verdi)/(Opprinnelig verdi) ∙ 100 %
def beregn_prosentendring(ny_verdi=None, opprinnelig_verdi=None, endring_i_prosent=None):
    try:
        if endring_i_prosent is not None and opprinnelig_verdi is not None:
            ny_verdi = opprinnelig_verdi * (1 + endring_i_prosent / 100)
            return ny_verdi
        elif ny_verdi is not None and opprinnelig_verdi is not None:
            endring_i_prosent = ((ny_verdi - opprinnelig_verdi) / opprinnelig_verdi) * 100
            return endring_i_prosent
        elif ny_verdi is not None and endring_i_prosent is not None:
            opprinnelig_verdi = ny_verdi / (1 + endring_i_prosent / 100)
            return opprinnelig_verdi
        else:
            return "Ugyldig input. Vennligst oppgi to av de tre verdiene."
    except ZeroDivisionError:
        return "Opprinnelig verdi kan ikke være null."
    except Exception as e:
        return f"En feil oppstod: {e}"

def hovedmeny():
    print("Velkommen til kalkulatoren for: Endringen i prosent = (Ny verdi – Opprinnelig verdi)/(Opprinnelig verdi) ∙ 100 %")
    print("Trykk 'q' når som helst for å avslutte programmet og trykk enter.\n")
    print("Hva vil du beregne?")
    
    print("1: Endringen i prosent")
    print("2: Ny verdi")
    print("3: Opprinnelig verdi")
    
    valg = input("Velg et alternativ (1, 2, 3): ")
    return valg
def hovedprogram():
    while True:
        valg = hovedmeny()
        
        if valg.lower() == 'q':
            print("Avslutter programmet. Ha en fin dag!")
            break
        
        if valg == "1":
            try:
                ny_verdi = float(input("Oppgi ny verdi: "))
                if ny_verdi == 'q':
                    print("Avslutter programmet. Ha en fin dag!")
                    break
                opprinnelig_verdi = float(input("Oppgi opprinnelig verdi: "))
                prosentendring = beregn_prosentendring(ny_verdi=ny_verdi, opprinnelig_verdi=opprinnelig_verdi)
                print(f"Endringen i prosent: {prosentendring:.2f} %\n")
            except ValueError:
                print("Ugyldig input. Vennligst skriv inn gyldige tall.")
        
        elif valg == "2":
            try:
                opprinnelig_verdi = float(input("Oppgi opprinnelig verdi: "))
                if opprinnelig_verdi == 'q':
                    print("Avslutter programmet. Ha en fin dag!")
                    break
                endring_i_prosent = float(input("Oppgi endringen i prosent: "))
                ny_verdi = beregn_prosentendring(opprinnelig_verdi=opprinnelig_verdi, endring_i_prosent=endring_i_prosent)
                print(f"Ny verdi: {ny_verdi:.2f}\n")
            except ValueError:
                print("Ugyldig input. Vennligst skriv inn gyldige tall.")
        
        elif valg == "3":
            try:
                ny_verdi = float(input("Oppgi ny verdi: "))
                if ny_verdi == 'q':
                    print("Avslutter programmet. Ha en fin dag!")
                    break
                endring_i_prosent = float(input("Oppgi endringen i prosent: "))
                opprinnelig_verdi = beregn_prosentendring(ny_verdi=ny_verdi, endring_i_prosent=endring_i_prosent)
                print(f"Opprinnelig verdi: {opprinnelig_verdi:.2f}\n")
            except ValueError:
                print("Ugyldig input. Vennligst skriv inn gyldige tall.")
        
        else:
            print("Ugyldig valg. Vennligst velg enten 1, 2 eller 3.")

if __name__ == "__main__":
    hovedprogram()

In [ ]:
# 1 Grunnleggende regning: 1.7 Prosentvis endring
# Regel 3.1 Vekstfaktor - Prosentvis økning. VF = 1 + Prosenten som desimaltall
def beregn_vekstfaktor(prosent):
    """
    Beregner vekstfaktoren basert på en prosentvis økning.
    
    Args:
    prosent (float): Prosentvis økning
    
    Returns:
    float: Vekstfaktoren
    """
    return 1 + prosent / 100

def main():
    print("Dette programmet regner ut vekstfaktoren ved en økning på en bestemt prosent.")
    try:
        prosent = float(input("Skriv inn denne prosenten: "))
        vekstfaktor = beregn_vekstfaktor(prosent)
        print(f"Vekstfaktoren ved en økning på {prosent:.2f} % er {vekstfaktor:.2f}.")
    except ValueError:
        print("Vennligst skriv inn et gyldig tall for prosenten.")

if __name__ == "__main__":
    main()

In [ ]:
# 1 Grunnleggende regning: 1.7 Prosentvis endring
# Regel 3.2 Vekstfaktor - Prosentvis nedgang. VF = 1 - Prosenten som desimaltall
def beregn_nedgangsfaktor(prosent):
    """
    Beregner vekstfaktoren basert på en prosentvis nedgang.
    
    Args:
    prosent (float): Prosentvis nedgang
    
    Returns:
    float: Nedgangsfaktoren, avrundet til tre desimaler
    """
    nedgangsfaktor = 1 - prosent / 100
    return round(nedgangsfaktor, 3)

def main():
    print("Dette programmet regner ut vekstfaktoren ved en nedgang på en bestemt prosent.")
    try:
        prosent = float(input("Skriv inn denne prosenten: "))
        nedgangsfaktor = beregn_nedgangsfaktor(prosent)
        print(f"Vekstfaktoren ved en nedgang på {prosent:.2f} prosent er {nedgangsfaktor:.2f}.")
    except ValueError:
        print("Vennligst skriv inn et gyldig tall for prosenten.")

if __name__ == "__main__":
    main()

In [ ]:
#  Arne 4 (Forbedret). Ny verdi = Opprinnelig verdi * Vekstfaktor^n hvor n er tiden + løsning av en ukjent i formelen = ett tall 
import math

# Ny funksjon for å håndtere input med komma og matematiske uttrykk
def eval_input(prompt):
    svar = input(prompt)
    if svar.lower() == 'q':
        print("Du valgte å avslutte programmet.")
        return None
    
    # Erstatt komma med punktum for desimaltall
    svar_renset = svar.replace(',', '.')
    
    try:
        # Prøv å evaluere strengen som et matematisk uttrykk
        # Hvis det er f.eks. "2+3", blir det 5. Hvis det er "5", blir det 5.0
        # Hvis det er "x", vil eval feile og vi returnerer "x"
        return eval(svar_renset)
    except (SyntaxError, NameError):
        # Hvis eval feiler (f.eks. ved 'x' eller annen tekst), returnerer vi den rensede strengen
        return svar_renset

# Oppdatert funksjon for ren tekstinput (f.eks. 'a', 'm', 'q', 'x')
def spør_tekst(prompt):
    svar = input(prompt)
    if svar.lower() == 'q':
        print("Du valgte å avslutte programmet.")
        return None
    return svar

def main():
    print("Dette programmet regner ut den nye verdien på et tall som skal øke eller minke med en viss prosent over tid.")
    print("Du kan skrive 'q' når som helst for å avslutte programmet.")
    print("Du kan også skrive matematiske uttrykk som '2*5+3' eller '10/2' eller '2**3' der tall forventes.\n")

    svar = spør_tekst("Dersom tallet skal øke, skriv 'a'. Dersom tallet skal minke, skriv 'm': ")
    if svar is None:
        return
    svar = svar.lower()
    while svar not in ['a', 'm']:
        print("Du skrev inn verken 'a' eller 'm'.")
        svar = spør_tekst("Dersom tallet skal øke, skriv 'a'. Dersom tallet skal minke, skriv 'm': ")
        if svar is None:
            return
        svar = svar.lower()

    valg = spør_tekst("Vil du beregne ny verdi (n), gammel verdi (g), vekstfaktor (v), tid (t) eller løse for en ukjent verdi (x)? ")
    if valg is None:
        return
    valg = valg.lower()

    if valg == "n":
        beregn_ny_verdi(svar)
    elif valg == "g":
        beregn_gammel_verdi(svar)
    elif valg == "v":
        beregn_vekstfaktor(svar)
    elif valg == "t":
        beregn_tid(svar)
    elif valg == "x":
        løs_ukjent(svar)
    else:
        print("Ugyldig valg")

def beregn_ny_verdi(svar_okning_minking):
    tall_input = eval_input("Skriv inn den opprinnelige verdien: ")
    if tall_input is None: return
    prosent_input = eval_input("Skriv inn prosenten tallet skal endres med (f.eks. '5' for 5%, '2.5' for 2,5%): ")
    if prosent_input is None: return
    tid_input = eval_input("Skriv inn tiden i antall år: ")
    if tid_input is None: return

    try:
        tall = float(tall_input)
        prosent = float(prosent_input)
        tid = float(tid_input)
        
        vekstfaktor = 1 + prosent / 100 if svar_okning_minking == "a" else 1 - prosent / 100
        ny_verdi = tall * (vekstfaktor ** tid)
        print(f"Den nye verdien etter {tid:.2f} år er {ny_verdi:.2f}")
    except (ValueError, TypeError):
        print("Ugyldig input. Sørg for å skrive inn gyldige tall eller matematiske uttrykk.")

def beregn_gammel_verdi(svar_okning_minking):
    ny_verdi_input = eval_input("Skriv inn den nye verdien: ")
    if ny_verdi_input is None: return
    prosent_input = eval_input("Skriv inn prosenten tallet skal endres med: ")
    if prosent_input is None: return
    tid_input = eval_input("Skriv inn tiden i antall år: ")
    if tid_input is None: return

    try:
        ny_verdi = float(ny_verdi_input)
        prosent = float(prosent_input)
        tid = float(tid_input)
        
        vekstfaktor = 1 + prosent / 100 if svar_okning_minking == "a" else 1 - prosent / 100
        gammel_verdi = ny_verdi / (vekstfaktor ** tid)
        print(f"Den opprinnelige verdien var {gammel_verdi:.2f}")
    except (ValueError, TypeError):
        print("Ugyldig input. Sørg for å skrive inn gyldige tall eller matematiske uttrykk.")

def beregn_vekstfaktor(svar_okning_minking):
    gammel_verdi_input = eval_input("Skriv inn den opprinnelige verdien: ")
    if gammel_verdi_input is None: return
    ny_verdi_input = eval_input("Skriv inn den nye verdien: ")
    if ny_verdi_input is None: return
    tid_input = eval_input("Skriv inn tiden i antall år: ")
    if tid_input is None: return

    try:
        gammel_verdi = float(gammel_verdi_input)
        ny_verdi = float(ny_verdi_input)
        tid = float(tid_input)
        
        vekstfaktor = (ny_verdi / gammel_verdi) ** (1 / tid)
        prosent = (vekstfaktor - 1) * 100 if svar_okning_minking == "a" else (1 - vekstfaktor) * 100
        print(f"Vekstfaktoren er {vekstfaktor:.4f}, som tilsvarer en prosentvis endring på {prosent:.2f}%")
    except (ValueError, TypeError, ZeroDivisionError):
        print("Ugyldig input. Sørg for å skrive inn gyldige tall eller matematiske uttrykk. Sjekk også for null-divisjon.")

def beregn_tid(svar_okning_minking):
    gammel_verdi_input = eval_input("Skriv inn den opprinnelige verdien: ")
    if gammel_verdi_input is None: return
    ny_verdi_input = eval_input("Skriv inn den nye verdien: ")
    if ny_verdi_input is None: return
    prosent_input = eval_input("Skriv inn prosenten tallet skal endres med: ")
    if prosent_input is None: return

    try:
        gammel_verdi = float(gammel_verdi_input)
        ny_verdi = float(ny_verdi_input)
        prosent = float(prosent_input)
        
        vekstfaktor = 1 + prosent / 100 if svar_okning_minking == "a" else 1 - prosent / 100
        
        if vekstfaktor <= 0: # Kan ikke ta logaritmen av et negativt tall eller 0
            print("Ugyldig vekstfaktor for tidsberegning. Vekstfaktoren må være større enn 0.")
            return
        if (ny_verdi / gammel_verdi) <= 0: # Kan ikke ta logaritmen av et negativt tall eller 0
            print("Ugyldig forhold mellom ny og gammel verdi for tidsberegning. Forholdet må være større enn 0.")
            return

        tid = math.log(ny_verdi / gammel_verdi) / math.log(vekstfaktor)
        print(f"Tiden det tar for verdien å endres fra {gammel_verdi:.2f} til {ny_verdi:.2f} er {tid:.2f} år")
    except (ValueError, TypeError, ZeroDivisionError):
        print("Ugyldig input. Sørg for å skrive inn gyldige tall eller matematiske uttrykk. Sjekk også for null-divisjon.")
    except Exception as e:
        print(f"Det oppstod en uventet feil: {e}")

def løs_ukjent(svar_okning_minking):
    print("\nSkriv inn verdiene for tre av variablene. Skriv 'x' for den ukjente.")
    print("Du kan også skrive matematiske uttrykk som '2*5+3' eller '10/2' eller '2**3' der tall forventes.")

    prosent_eller_vekst = spør_tekst("Vil du bruke prosent (p) eller vekstfaktor (v)? ")
    if prosent_eller_vekst is None:
        return
    prosent_eller_vekst = prosent_eller_vekst.lower()
    while prosent_eller_vekst not in ['p', 'v']:
        prosent_eller_vekst = spør_tekst("Ugyldig valg. Skriv 'p' for prosent eller 'v' for vekstfaktor: ")
        if prosent_eller_vekst is None:
            return
        prosent_eller_vekst = prosent_eller_vekst.lower()

    n_input = eval_input("Ny verdi (skriv 'x' for ukjent): ")
    if n_input is None: return
    g_input = eval_input("Opprinnelig verdi (skriv 'x' for ukjent): ")
    if g_input is None: return
    faktor_input = eval_input("Prosent/vekstfaktor (skriv 'x' for ukjent): ")
    if faktor_input is None: return
    tid_input = eval_input("Tid (år) (skriv 'x' for ukjent): ")
    if tid_input is None: return

    # Teller hvor mange ukjente variabler som ble gitt
    ukjente = [var for var in [n_input, g_input, faktor_input, tid_input] if isinstance(var, str) and var.lower() == 'x']

    if len(ukjente) != 1:
        print("Du må skrive 'x' for nøyaktig én av variablene.")
        return

    try:
        vekstfaktor_obj = None # Bruker denne for å lagre vekstfaktoren uavhengig av om den er ukjent eller ikke

        # Prøver å parse input for kjente verdier
        n = float(n_input) if isinstance(n_input, (int, float)) else n_input
        g = float(g_input) if isinstance(g_input, (int, float)) else g_input
        t = float(tid_input) if isinstance(tid_input, (int, float)) else tid_input

        if isinstance(faktor_input, (int, float)): # Hvis faktor_input er et kjent tall
            if prosent_eller_vekst == 'p':
                prosent = float(faktor_input)
                vekstfaktor_obj = 1 + prosent / 100 if svar_okning_minking == 'a' else 1 - prosent / 100
            else: # vekstfaktor
                vekstfaktor_obj = float(faktor_input)
        elif isinstance(faktor_input, str) and faktor_input.lower() == 'x': # Riktig sammenligning
            vekstfaktor_obj = 'x' # Angi at vekstfaktor er ukjent

        # Løsning basert på hvilken variabel som er 'x'
        if isinstance(n, str) and n.lower() == 'x':
            if vekstfaktor_obj == 'x': # Korrigert: bruk ==
                print("Kan ikke løse for ny verdi når vekstfaktor også er ukjent.")
                return
            n = g * (vekstfaktor_obj ** t)
            print(f"Ny verdi = {n:.2f}")
        elif isinstance(g, str) and g.lower() == 'x':
            if vekstfaktor_obj == 'x': # Korrigert: bruk ==
                print("Kan ikke løse for opprinnelig verdi når vekstfaktor også er ukjent.")
                return
            g = n / (vekstfaktor_obj ** t)
            print(f"Opprinnelig verdi = {g:.2f}")
        elif isinstance(vekstfaktor_obj, str) and vekstfaktor_obj.lower() == 'x':
            if g == 0:
                print("Kan ikke beregne vekstfaktor hvis opprinnelig verdi er null.")
                return
            vekstfaktor_obj = (n / g) ** (1 / t)
            prosent = (vekstfaktor_obj - 1) * 100 if svar_okning_minking == "a" else (1 - vekstfaktor_obj) * 100
            print(f"Vekstfaktor = {vekstfaktor_obj:.4f} (tilsvarer {prosent:.2f}% {'økning' if svar_okning_minking == 'a' else 'reduksjon'})")
        elif isinstance(t, str) and t.lower() == 'x':
            if vekstfaktor_obj <= 0 or (n / g) <= 0:
                print("Ugyldige verdier for logaritmeberegning (vekstfaktor og/eller forhold må være > 0).")
                return
            if vekstfaktor_obj == 1:
                print("Vekstfaktor på 1 fører ikke til endring over tid, kan ikke beregne tid for endring.")
                return
            t = math.log(n / g) / math.log(vekstfaktor_obj)
            print(f"Tid = {t:.2f} år")
        else:
            print("En ukjent feil oppstod. Sørg for å følge instruksjonene.")

    except (ValueError, TypeError, ZeroDivisionError) as e:
        print(f"Det oppstod en feil med input eller beregning: {e}. Sørg for gyldige numeriske verdier.")
    except Exception as e:
        print(f"En uventet feil oppstod: {e}")

# Start programmet
main()

# $\color{green}{\text{Kapittel 2 - Personlig økonomi}}$

In [ ]:
# 2 Personlig økonomi: 2.1 Regneark
import pandas as pd
import matplotlib.pyplot as plt

# Data for den første måneden
data_måned1 = {
    'Tur': ['A', 'B', 'C'],
    'Pris per deltaker (kr)': [250, 300, 500],
    'Antall deltakere': [32, 42, 16]
}

# Data for den andre måneden
data_måned2 = {
    'Tur': ['A', 'B', 'C'],
    'Pris per deltaker (kr)': [250, 300, 500],
    'Antall deltakere': [35, 0, 12]
}

# Opprett DataFrames
df_måned1 = pd.DataFrame(data_måned1)
df_måned2 = pd.DataFrame(data_måned2)

# Beregn omsetning per tur for hver måned
df_måned1['Omsetning (kr)'] = df_måned1['Pris per deltaker (kr)'] * df_måned1['Antall deltakere']
df_måned2['Omsetning (kr)'] = df_måned2['Pris per deltaker (kr)'] * df_måned2['Antall deltakere']

# Beregn total omsetning for hver måned
total_omsetning_måned1 = df_måned1['Omsetning (kr)'].sum()
total_omsetning_måned2 = df_måned2['Omsetning (kr)'].sum()

# Legg til total rad
df_måned1.loc[len(df_måned1)] = ['Totalt', '', '', total_omsetning_måned1]
df_måned2.loc[len(df_måned2)] = ['Totalt', '', '', total_omsetning_måned2]

# Plot tabellene
fig, axs = plt.subplots(2, 1, figsize=(10, 8))

# Første måned
axs[0].axis('tight')
axs[0].axis('off')
table1 = axs[0].table(cellText=df_måned1.values, colLabels=df_måned1.columns, cellLoc='center', loc='center')
table1.auto_set_font_size(False)
table1.set_fontsize(12)
table1.scale(1.2, 1.2)
axs[0].set_title('Omsetning per tur for den første måneden', fontsize=14)

# Andre måned
axs[1].axis('tight')
axs[1].axis('off')
table2 = axs[1].table(cellText=df_måned2.values, colLabels=df_måned2.columns, cellLoc='center', loc='center')
table2.auto_set_font_size(False)
table2.set_fontsize(12)
table2.scale(1.2, 1.2)
axs[1].set_title('Omsetning per tur for den andre måneden', fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
# 2 Personlig økonomi: 2.2 Lønn og skatt
def eval_input(prompt, allow_exit=False):
    while True:
        user_input = input(prompt)
        if user_input.lower() in ['q', 'quit']:
            print("Avslutter programmet.")
            exit()
        try:
            return float(user_input)
        except ValueError:
            print("Ugyldig input. Prøv igjen." + (" Eller skriv 'q' for å avslutte." if allow_exit else ""))


def velg_lønnstype():
    print("\nVelg lønnstype:")
    print("1. Fast månedslønn")
    print("2. Timelønn")
    print("q. Avslutt")

    lønnstype = input("Ditt valg: ").strip().lower()
    if lønnstype == '1':
        fastlønn = eval_input("Skriv inn fast månedslønn (kr): ", allow_exit=True)
        timelønn = 0
        periode = "måned"
    elif lønnstype == '2':
        timelønn = eval_input("Skriv inn timelønn (kr): ", allow_exit=True)
        periodevalg = input("Hvilken periode gjelder timene for? (1. Uke, 2. Måned, 3. År): ").strip().lower()
        fastlønn = 0
        periode = {"1": "uke", "2": "måned", "3": "år"}.get(periodevalg, "måned")
    elif lønnstype == 'q':
        print("Avslutter programmet.")
        return None, None, None
    else:
        print("Ugyldig valg.")
        return velg_lønnstype()

    return fastlønn, timelønn, periode


def hent_overtid(timelønn):
    tillegg_sum = 0
    detaljer = []

    har_overtid = input("\nHar du jobbet overtid? (j/n): ").strip().lower()
    if har_overtid not in ['j', 'ja']:
        return 0, detaljer

    if timelønn == 0:
        timelønn = eval_input("Skriv inn timelønn (kr) for beregning av overtid: ", allow_exit=True)

    antall = int(eval_input("Hvor mange forskjellige overtidstillegg har du (f.eks. 20%, 50%, 100%)? ", allow_exit=True))
    for i in range(1, antall + 1):
        prosent = eval_input(f"Prosenttillegg for overtidstype {i} (f.eks. 50 for 50%): ", allow_exit=True)
        timer = eval_input(f"Antall timer med {prosent}% tillegg: ", allow_exit=True)
        lønn = timelønn * (1 + prosent / 100) * timer
        tillegg_sum += lønn
        detaljer.append((prosent, timer, lønn))

    return tillegg_sum, detaljer


def beregn_tabellkort(fastlønn):
    nærmeste_100 = int(fastlønn // 100 * 100)
    tabell = {
        25400: 5098, 25500: 5138, 25600: 5177, 25700: 5217, 25800: 5257, 25900: 5296, 
        26000: 5336, 26100: 5376, 26200: 5415, 26300: 5455, 26400: 5495, 26500: 5535, 
        26600: 5574, 26700: 5614, 26800: 5654, 26900: 5693, 27000: 5733, 27100: 5773, 
        27200: 5812, 27300: 5852, 27400: 5892, 27500: 5932, 27600: 5971, 27700: 6011, 
        27800: 6051, 27900: 6090, 28000: 6130, 28100: 6170, 28200: 6210, 28300: 6249, 
        28400: 6289, 28500: 6329, 28600: 6368, 28700: 6408, 28800: 6448, 28900: 6488, 
        29000: 6527, 29100: 6567, 29200: 6607, 29300: 6646, 29400: 6686, 29500: 6726, 
        29600: 6766, 29700: 6805, 29800: 6845, 29900: 6885, 30000: 6924, 30100: 6964, 
        30200: 7004, 30300: 7044, 30400: 7083, 30500: 7123, 30600: 7163, 30700: 7202, 
        30800: 7242, 30900: 7282, 31000: 7321, 31100: 7361, 31200: 7401, 31300: 7441, 
        31400: 7480, 31500: 7520, 31600: 7560, 31700: 7599, 31800: 7639, 31900: 7679, 
        32000: 7718, 32100: 7758, 32200: 7798, 32300: 7838, 32400: 7877, 32500: 7917, 
        32600: 7957, 32700: 7996, 32800: 8036, 32900: 8076, 33000: 8115, 33100: 8155, 
        33200: 8195, 33300: 8235, 33400: 8274, 33500: 8314, 33600: 8354, 33700: 8394, 
        33800: 8433, 33900: 8473, 34000: 8513, 34100: 8552, 34200: 8592, 34300: 8632, 
        34400: 8671, 34500: 8711, 34600: 8751, 34700: 8791, 34800: 8830, 34900: 8870, 
        35000: 8910, 35100: 8949, 35200: 8989, 35300: 9029, 35400: 9069, 35500: 9108, 
        35600: 9148, 35700: 9188, 35800: 9227, 35900: 9267, 36000: 9307, 36100: 9346, 
        36200: 9386, 36300: 9426, 36400: 9466, 36500: 9505, 36600: 9545, 36700: 9585, 
        36800: 9624, 36900: 9664, 37000: 9704, 37100: 9744, 37200: 9783, 37300: 9823, 
        37400: 9863, 37500: 9902, 37600: 9942, 37700: 9982, 37800: 10022, 37900: 10061, 
        38000: 10101, 38100: 10141, 38200: 10180, 38300: 10220, 38400: 10260, 38500: 10300, 
        38600: 10339, 38700: 10379, 38800: 10419, 38900: 10458, 39000: 10498, 39100: 10538, 
        39200: 10577, 39300: 10617, 39400: 10657, 39500: 10697, 39600: 10736, 39700: 10776, 
        39800: 10816, 39900: 10855, 40000: 10895, 40100: 10935, 40200: 10974, 40300: 11014, 
        40400: 11054, 40500: 11094, 40600: 11133, 40700: 11173, 40800: 11213, 40900: 11252, 
        41000: 11292, 41100: 11332, 41200: 11372, 41300: 11411, 41400: 11451, 41500: 11491, 
        41600: 11530, 41700: 11570, 41800: 11610, 41900: 11650, 42000: 11689, 42100: 11729,
    }
    return tabell.get(nærmeste_100, fastlønn * 0.25)


def beregn_prosentkort():
    return eval_input("Skriv inn prosentsats for forskuddstrekk (f.eks. 34): ", allow_exit=True) / 100


def generer_lønnsslipp(fastlønn, tillegg, tillegg_detaljer, skatt_fast, skatt_tillegg, periode, korttype):
    total = fastlønn + tillegg
    netto = total - skatt_fast - skatt_tillegg

    print("\n🧾 Lønnsslipp")
    print("--------------------------------------------------")
    print(f"Lønn beregnet ut fra periode: {periode}")
    
    if fastlønn > 0:
        print(f"Fastlønn: {fastlønn:.2f} kr")
    if tillegg > 0:
        print(f"Tillegg (overtid): {tillegg:.2f} kr")
        for prosent, timer, lønn in tillegg_detaljer:
            print(f"  - {timer} t × {prosent}% → {lønn:.2f} kr")

    print(f"Bruttolønn: {total:.2f} kr")
    print("--------------------------------------------------")

    if korttype == "tabell":
        print(f"Forskuddstrekk (tabellkort): {skatt_fast:.2f} kr")
        print(f"Forskuddstrekk (prosent av tillegg): {skatt_tillegg:.2f} kr")
    elif korttype == "prosent":
        print(f"Forskuddstrekk (prosentkort): {skatt_fast:.2f} kr")
    elif korttype == "frikort":
        print(f"Skatt: {skatt_fast:.2f} kr")

    print("--------------------------------------------------")
    print(f"Netto utbetalt: {netto:.2f} kr\n")


def frikort_behandling():
    fastlønn, timelønn, periode = velg_lønnstype()
    if fastlønn is None:
        return

    if fastlønn == 0:
        antall_timer = eval_input("Hvor mange timer har du jobbet i perioden?: ", allow_exit=True)
        fastlønn = timelønn * antall_timer

    tillegg, tillegg_detaljer = hent_overtid(timelønn)
    total = fastlønn + tillegg
    fribeløp = 55000
    skatt = 0

    if total > fribeløp:
        skatt = (total - fribeløp) * 0.25

    generer_lønnsslipp(fastlønn, tillegg, tillegg_detaljer, skatt, 0, periode, "frikort")


def tabellkort_behandling():
    fastlønn, timelønn, periode = velg_lønnstype()
    if fastlønn is None:
        return

    skatt_fast = beregn_tabellkort(fastlønn)
    tillegg, tillegg_detaljer = hent_overtid(timelønn)

    prosent = beregn_prosentkort()
    skatt_tillegg = tillegg * prosent

    generer_lønnsslipp(fastlønn, tillegg, tillegg_detaljer, skatt_fast, skatt_tillegg, periode, "tabell")


def prosentkort_behandling():
    fastlønn, timelønn, periode = velg_lønnstype()
    if fastlønn is None:
        return

    if fastlønn == 0:
        antall_timer = eval_input("Hvor mange timer har du jobbet i perioden?: ", allow_exit=True)
        fastlønn = timelønn * antall_timer

    tillegg, tillegg_detaljer = hent_overtid(timelønn)
    brutto_total = fastlønn + tillegg
    forskuddstrekk_prosent = eval_input("Skriv inn prosentsats for forskuddstrekk (f.eks. 34): ", allow_exit=True)
    skatt = brutto_total * (forskuddstrekk_prosent / 100)

    generer_lønnsslipp(fastlønn, tillegg, tillegg_detaljer, skatt, 0, periode, "prosent")


def main():
    while True:
        print("\n📌 Hva slags skattekort bruker du?")
        print("1. Frikort")
        print("2. Tabellkort")
        print("3. Prosentkort")
        print("q. Avslutt")
        valg = input("Ditt valg: ").strip().lower()

        if valg == '1':
            frikort_behandling()
        elif valg == '2':
            tabellkort_behandling()
        elif valg == '3':
            prosentkort_behandling()
        elif valg == 'q':
            print("Avslutter programmet.")
            break
        else:
            print("Ugyldig valg. Prøv igjen.")


if __name__ == "__main__":
    main()

In [ ]:
# 2 Personlig økonomi: 2.3 Sparing 
# Regel 4. Ny verdi = Opprinnelig verdi * Vekstfaktor^n hvor n er tiden + løsning av en ukjent i formelen = ett tall 
import math

def spør(prompt):
    svar = input(prompt)
    if svar.lower() == 'q':
        print("Du valgte å avslutte programmet.")
        return None
    return svar

def main():
    print("Dette programmet regner ut den nye verdien på et tall som skal øke eller minke med en viss prosent over tid.")
    print("Du kan skrive 'q' når som helst for å avslutte programmet.\n")

    svar = spør("Dersom tallet skal øke, skriv 'a'. Dersom tallet skal minke, skriv 'm': ")
    if svar is None:
        return
    svar = svar.lower()
    while svar not in ['a', 'm']:
        print("Du skrev inn verken 'a' eller 'm'.")
        svar = spør("Dersom tallet skal øke, skriv 'a'. Dersom tallet skal minke, skriv 'm': ")
        if svar is None:
            return
        svar = svar.lower()

    valg = spør("Vil du beregne ny verdi (n), gammel verdi (g), vekstfaktor (v), tid (t) eller løse for en ukjent verdi (x)? ")
    if valg is None:
        return
    valg = valg.lower()

    if valg == "n":
        beregn_ny_verdi(svar)
    elif valg == "g":
        beregn_gammel_verdi(svar)
    elif valg == "v":
        beregn_vekstfaktor(svar)
    elif valg == "t":
        beregn_tid(svar)
    elif valg == "x":
        løs_ukjent(svar)
    else:
        print("Ugyldig valg")

def beregn_ny_verdi(svar):
    tall = spør("Skriv inn den opprinnelige verdien: ")
    if tall is None: return
    prosent = spør("Skriv inn prosenten tallet skal endres med: ")
    if prosent is None: return
    tid = spør("Skriv inn tiden i antall år: ")
    if tid is None: return

    tall = float(tall)
    prosent = float(prosent)
    tid = float(tid)
    
    vekstfaktor = 1 + prosent / 100 if svar == "a" else 1 - prosent / 100
    ny_verdi = tall * (vekstfaktor ** tid)
    print(f"Den nye verdien etter {tid:.2f} år er {ny_verdi:.2f}")

def beregn_gammel_verdi(svar):
    ny_verdi = spør("Skriv inn den nye verdien: ")
    if ny_verdi is None: return
    prosent = spør("Skriv inn prosenten tallet skal endres med: ")
    if prosent is None: return
    tid = spør("Skriv inn tiden i antall år: ")
    if tid is None: return

    ny_verdi = float(ny_verdi)
    prosent = float(prosent)
    tid = float(tid)
    
    vekstfaktor = 1 + prosent / 100 if svar == "a" else 1 - prosent / 100
    gammel_verdi = ny_verdi / (vekstfaktor ** tid)
    print(f"Den opprinnelige verdien var {gammel_verdi:.2f}")

def beregn_vekstfaktor(svar):
    gammel_verdi = spør("Skriv inn den opprinnelige verdien: ")
    if gammel_verdi is None: return
    ny_verdi = spør("Skriv inn den nye verdien: ")
    if ny_verdi is None: return
    tid = spør("Skriv inn tiden i antall år: ")
    if tid is None: return

    gammel_verdi = float(gammel_verdi)
    ny_verdi = float(ny_verdi)
    tid = float(tid)
    
    vekstfaktor = (ny_verdi / gammel_verdi) ** (1 / tid)
    prosent = (vekstfaktor - 1) * 100 if svar == "a" else (1 - vekstfaktor) * 100
    print(f"Vekstfaktoren er {vekstfaktor:.4f}, som tilsvarer en prosentvis endring på {prosent:.2f}%")

def beregn_tid(svar):
    gammel_verdi = spør("Skriv inn den opprinnelige verdien: ")
    if gammel_verdi is None: return
    ny_verdi = spør("Skriv inn den nye verdien: ")
    if ny_verdi is None: return
    prosent = spør("Skriv inn prosenten tallet skal endres med: ")
    if prosent is None: return

    gammel_verdi = float(gammel_verdi)
    ny_verdi = float(ny_verdi)
    prosent = float(prosent)
    
    vekstfaktor = 1 + prosent / 100 if svar == "a" else 1 - prosent / 100
    tid = math.log(ny_verdi / gammel_verdi) / math.log(vekstfaktor)
    print(f"Tiden det tar for verdien å endres fra {gammel_verdi:.2f} til {ny_verdi:.2f} er {tid:.2f} år")

def løs_ukjent(svar):
    print("\nSkriv inn verdiene for tre av variablene. Skriv 'x' for den ukjente.")

    prosent_eller_vekst = spør("Vil du bruke prosent (p) eller vekstfaktor (v)? ")
    if prosent_eller_vekst is None:
        return
    prosent_eller_vekst = prosent_eller_vekst.lower()
    while prosent_eller_vekst not in ['p', 'v']:
        prosent_eller_vekst = spør("Ugyldig valg. Skriv 'p' for prosent eller 'v' for vekstfaktor: ")
        if prosent_eller_vekst is None:
            return
        prosent_eller_vekst = prosent_eller_vekst.lower()

    n_verdi = spør("Ny verdi: ")
    if n_verdi is None: return
    g_verdi = spør("Opprinnelig verdi: ")
    if g_verdi is None: return
    faktor_input = spør("Prosent/vekstfaktor: ")
    if faktor_input is None: return
    tid = spør("Tid (år): ")
    if tid is None: return

    try:
        if faktor_input.lower() == 'x':
            faktor_er_ukjent = True
        else:
            faktor_er_ukjent = False
            if prosent_eller_vekst == 'p':
                prosent = float(faktor_input)
                vekstfaktor = 1 + prosent / 100 if svar == 'a' else 1 - prosent / 100
            else:
                vekstfaktor = float(faktor_input)

        if n_verdi.lower() == 'x':
            g = float(g_verdi)
            t = float(tid)
            n = g * (vekstfaktor ** t)
            print(f"Ny verdi = {n:.2f}")
        elif g_verdi.lower() == 'x':
            n = float(n_verdi)
            t = float(tid)
            g = n / (vekstfaktor ** t)
            print(f"Opprinnelig verdi = {g:.2f}")
        elif faktor_er_ukjent:
            n = float(n_verdi)
            g = float(g_verdi)
            t = float(tid)
            vekstfaktor = (n / g) ** (1 / t)
            prosent = (vekstfaktor - 1) * 100 if svar == "a" else (1 - vekstfaktor) * 100
            print(f"Vekstfaktor = {vekstfaktor:.4f} (tilsvarer {prosent:.2f}% {'økning' if svar == 'a' else 'reduksjon'})")
        elif tid.lower() == 'x':
            n = float(n_verdi)
            g = float(g_verdi)
            t = math.log(n / g) / math.log(vekstfaktor)
            print(f"Tid = {t:.2f} år")
        else:
            print("Du må skrive 'x' for én av variablene.")
    except Exception as e:
        print(f"Det oppstod en feil: {e}")

# Start programmet
main()

In [ ]:
# 2 Personlig økonomi: 2.4 Serielån
import matplotlib.pyplot as plt
import pandas as pd

def beregn_serielån(lånebeløp, rente, antall_år, antall_perioder_per_år):
    antall_perioder = antall_år * antall_perioder_per_år
    terminbeløp_per_periode = lånebeløp / antall_perioder
    gjenværende_saldo = lånebeløp
    betalt_rente = []
    betalt_avdrag = []
    gjenværende_saldo_liste = []
    år_liste = []
    termin_liste = []
    for periode in range(antall_perioder):
        år = periode // antall_perioder_per_år + 1
        år_liste.append(år)
        termin_liste.append(periode + 1)
        betalt_rente_periode = rente / antall_perioder_per_år * gjenværende_saldo
        betalt_rente.append(betalt_rente_periode)
        betalt_avdrag_periode = terminbeløp_per_periode
        betalt_avdrag.append(betalt_avdrag_periode)
        gjenværende_saldo -= betalt_avdrag_periode
        gjenværende_saldo_liste.append(gjenværende_saldo)
    return år_liste, termin_liste, betalt_avdrag, betalt_rente, gjenværende_saldo_liste

def plott_lånebetalinger_serielån(år_liste, betalt_avdrag, betalt_rente, antall_år, antall_perioder_per_år):
    stolpebredde = 0.5 / antall_perioder_per_år
    x_pos = [i / antall_perioder_per_år for i in range(len(år_liste))]
    plt.bar(x_pos, betalt_avdrag, width=stolpebredde, align='center', label='Avdrag', edgecolor='black', linewidth=1, color='b')
    plt.bar(x_pos, betalt_rente, bottom=betalt_avdrag, width=stolpebredde, align='center', label='Renter', edgecolor='black', linewidth=1, color='r')
    plt.xticks(range(antall_år + 1))
    plt.xlabel('År')
    plt.ylabel('Beløp (NOK)')
    plt.title('Terminbeløp for serielån')
    plt.legend(loc='upper right')
    plt.grid(False)
    plt.show()

def lag_lånedataframe_serielån(år_liste, termin_liste, betalt_avdrag, betalt_rente, gjenværende_saldo_liste):
    data = {
        'År': år_liste,
        'Termin': termin_liste,
        'Avdrag': betalt_avdrag,
        'Rente': betalt_rente,
        'Terminbeløp': [a + r for a, r in zip(betalt_avdrag, betalt_rente)],
        'Kumulativ Rente': pd.Series(betalt_rente).cumsum(),
        'Kumulativ Avdrag': pd.Series(betalt_avdrag).cumsum(),
        'Restlån': gjenværende_saldo_liste
    }
    df = pd.DataFrame(data)
    df.index = [''] * len(df)  # Fjern radnumre
    return df

def hent_input(spørsmål, tillat_formler=True):
    while True:
        verdi = input(spørsmål)
        if verdi.lower() == 'q':
            return 'q'
        try:
            if tillat_formler:
                return float(eval(verdi))
            else:
                return int(verdi)
        except:
            print("Ugyldig input. Prøv igjen eller trykk 'q' for å avslutte.")

def format_beløp(beløp):
    return f"{beløp:,.0f} kr".replace(",", " ")  # Bruk mellomrom som tusenskille

def main():
    print("\n📊 Velkommen til serielån-kalkulatoren!")
    print("(Skriv inn tall, eller trykk 'q' for å avslutte.)\n")

    while True:
        har_kjøp = input("Har du noe du skal kjøpe? (ja/nei): ").strip().lower()
        if har_kjøp == 'q':
            break

        if har_kjøp == 'ja':
            kjøpesum = hent_input("1. Hva koster det du skal kjøpe? (f.eks. 250000): ")
            if kjøpesum == 'q': break

            sparebeløp = hent_input("2. Hvor mye penger har du i banken i dag? (f.eks. 100000): ")
            if sparebeløp == 'q': break

            lånebeløp = kjøpesum - sparebeløp
            if lånebeløp <= 0:
                print("🎉 Du har nok penger og trenger ikke lån!")
                continue

        elif har_kjøp == 'nei':
            sparebeløp = hent_input("1. Hvor mye penger har du i banken i dag? (f.eks. 100000): ")
            if sparebeløp == 'q': break

            ønsket_lån = hent_input("2. Hvor mye ønsker du å låne? (f.eks. 150000): ")
            if ønsket_lån == 'q': break

            lånebeløp = ønsket_lån
        else:
            print("Vennligst svar 'ja' eller 'nei', eller 'q' for å avslutte.\n")
            continue

        print(f"\n💡 Du trenger å låne: {format_beløp(lånebeløp)}")

        rente_prosent = hent_input("3. Årlig rente i prosent (f.eks. 4 for 4%): ")
        if rente_prosent == 'q': break
        rente = rente_prosent / 100

        antall_år = hent_input("4. Nedbetalingstid i år (f.eks. 5): ", tillat_formler=False)
        if antall_år == 'q': break

        antall_perioder = hent_input("5. Antall terminer per år (f.eks. 1 eller 12): ", tillat_formler=False)
        if antall_perioder == 'q': break

        print("\n🔄 Beregner serielån...\n")

        år_liste, termin_liste, betalt_avdrag, betalt_rente, saldo = beregn_serielån(
            lånebeløp, rente, antall_år, antall_perioder)

        df = lag_lånedataframe_serielån(år_liste, termin_liste, betalt_avdrag, betalt_rente, saldo)
        print(df.to_string(formatters={
            'Avdrag': lambda x: format_beløp(x),
            'Rente': lambda x: format_beløp(x),
            'Terminbeløp': lambda x: format_beløp(x),
            'Kumulativ Rente': lambda x: format_beløp(x),
            'Kumulativ Avdrag': lambda x: format_beløp(x),
            'Restlån': lambda x: format_beløp(x)
        }))

        plott_lånebetalinger_serielån(år_liste, betalt_avdrag, betalt_rente, antall_år, antall_perioder)

        total_rente = sum(betalt_rente)
        total_avdrag = sum(betalt_avdrag)
        total_betaling = total_rente + total_avdrag

        print("\n📌 Oppsummering:")
        print(f"- Du låner: {format_beløp(lånebeløp)}")
        print(f"- Nedbetalingstid: {antall_år} år, med {antall_perioder} termin(er) per år.")
        print(f"- Totalt betalt i avdrag: {format_beløp(total_avdrag)}")
        print(f"- Totalt betalt i renter: {format_beløp(total_rente)}")
        print(f"- Totalt betalt til sammen: {format_beløp(total_betaling)}\n")

        print(f"💬 Du betaler altså {format_beløp(total_betaling)} totalt over {antall_år} år.")

if __name__ == "__main__":
    main()

In [ ]:
# 2 Personlig økonomi: 2.4 Annuitetslån
import matplotlib.pyplot as plt
import pandas as pd

# Funksjon for beregning av annuitetslån
def beregn_annuitetslån(lånebeløp, rente, antall_år, antall_perioder_per_år, terminbeløp_per_periode=None):
    antall_perioder = antall_år * antall_perioder_per_år
    rente_per_periode = rente / antall_perioder_per_år
    
    if terminbeløp_per_periode is None:
        annuitetsfaktor = (rente_per_periode * (1 + rente_per_periode) ** antall_perioder) / ((1 + rente_per_periode) ** antall_perioder - 1)
        terminbeløp_per_periode = lånebeløp * annuitetsfaktor

    gjenværende_saldo = lånebeløp
    betalt_rente = []
    betalt_avdrag = []
    gjenværende_saldo_liste = []
    år_liste = []
    termin_liste = []
    for periode in range(antall_perioder):
        år = periode // antall_perioder_per_år + 1
        år_liste.append(år)
        termin_liste.append(periode + 1)
        betalt_rente_periode = rente_per_periode * gjenværende_saldo
        betalt_rente.append(betalt_rente_periode)
        betalt_avdrag_periode = terminbeløp_per_periode - betalt_rente_periode
        betalt_avdrag.append(betalt_avdrag_periode)
        gjenværende_saldo -= betalt_avdrag_periode
        gjenværende_saldo_liste.append(gjenværende_saldo)
    return år_liste, termin_liste, betalt_avdrag, betalt_rente, gjenværende_saldo_liste

# Funksjon for plotting av annuitetslån
def plott_lånebetalinger(år_liste, betalt_avdrag, betalt_rente, antall_år, antall_perioder_per_år):
    stolpebredde = 0.5 / antall_perioder_per_år
    x_pos = [i / antall_perioder_per_år for i in range(len(år_liste))]
    plt.bar(x_pos, betalt_avdrag, width=stolpebredde, align='center', label='Avdrag', edgecolor='black', linewidth=1, color='b')
    plt.bar(x_pos, betalt_rente, bottom=betalt_avdrag, width=stolpebredde, align='center', label='Renter', edgecolor='black', linewidth=1, color='r')
    plt.xticks(range(antall_år + 1))
    plt.xlabel('År')
    plt.ylabel('Beløp (NOK)')
    plt.title('Terminbeløp for annuitetslån')
    plt.legend(loc='upper right')
    plt.grid(False)
    plt.show()

# Funksjon for å lage lånedataframe med restlån
def lag_lånedataframe_annuitetslån(år_liste, termin_liste, betalt_avdrag, betalt_rente, gjenværende_saldo_liste):
    data = {
        'År': år_liste,
        'Termin': termin_liste,
        'Avdrag': betalt_avdrag,
        'Rente': betalt_rente,
        'Terminbeløp': [a + r for a, r in zip(betalt_avdrag, betalt_rente)],
        'Kumulativ Rente': pd.Series(betalt_rente).cumsum(),
        'Kumulativ Avdrag': pd.Series(betalt_avdrag).cumsum(),
        'Restlån': gjenværende_saldo_liste
    }
    df = pd.DataFrame(data)
    df.index = [''] * len(df)  # Fjern radnumre
    return df

# Funksjon for input med håndtering av feil
def hent_input(spørsmål, tillat_formler=True):
    while True:
        verdi = input(spørsmål)
        if verdi.lower() == 'q':
            return 'q'
        try:
            if tillat_formler:
                return float(eval(verdi))
            else:
                return int(verdi)
        except:
            print("Ugyldig input. Prøv igjen eller trykk 'q' for å avslutte.")

# Funksjon for å formatere beløp i kr
def format_beløp(beløp):
    return f"{beløp:,.0f} kr".replace(",", " ")  # Bruk mellomrom som tusenskille

# Hovedfunksjon for programmet
def main():
    print("\n📊 Velkommen til annuitetslån-kalkulatoren!")
    print("(Skriv inn tall, eller trykk 'q' for å avslutte.)\n")

    while True:
        har_kjøp = input("Har du noe du skal kjøpe? (ja/nei): ").strip().lower()
        if har_kjøp == 'q':
            break

        if har_kjøp == 'ja':
            kjøpesum = hent_input("1. Hva koster det du skal kjøpe? (f.eks. 250000): ")
            if kjøpesum == 'q': break

            sparebeløp = hent_input("2. Hvor mye penger har du i banken i dag? (f.eks. 100000): ")
            if sparebeløp == 'q': break

            lånebeløp = kjøpesum - sparebeløp
            if lånebeløp <= 0:
                print("🎉 Du har nok penger og trenger ikke lån!")
                continue

        elif har_kjøp == 'nei':
            sparebeløp = hent_input("1. Hvor mye penger har du i banken i dag? (f.eks. 100000): ")
            if sparebeløp == 'q': break

            ønsket_lån = hent_input("2. Hvor mye ønsker du å låne? (f.eks. 150000): ")
            if ønsket_lån == 'q': break

            lånebeløp = ønsket_lån
        else:
            print("Vennligst svar 'ja' eller 'nei', eller 'q' for å avslutte.\n")
            continue

        print(f"\n💡 Du trenger å låne: {format_beløp(lånebeløp)}")

        rente_prosent = hent_input("3. Årlig rente i prosent (f.eks. 4 for 4%): ")
        if rente_prosent == 'q': break
        rente = rente_prosent / 100

        antall_år = hent_input("4. Nedbetalingstid i år (f.eks. 5): ", tillat_formler=False)
        if antall_år == 'q': break

        antall_perioder = hent_input("5. Antall terminer per år (f.eks. 1 eller 12): ", tillat_formler=False)
        if antall_perioder == 'q': break

        print("\n🔄 Beregner annuitetslån...\n")

        år_liste, termin_liste, betalt_avdrag, betalt_rente, saldo = beregn_annuitetslån(
            lånebeløp, rente, antall_år, antall_perioder)

        df = lag_lånedataframe_annuitetslån(år_liste, termin_liste, betalt_avdrag, betalt_rente, saldo)
        print(df.to_string(formatters={
            'Avdrag': lambda x: format_beløp(x),
            'Rente': lambda x: format_beløp(x),
            'Terminbeløp': lambda x: format_beløp(x),
            'Kumulativ Rente': lambda x: format_beløp(x),
            'Kumulativ Avdrag': lambda x: format_beløp(x),
            'Restlån': lambda x: format_beløp(x)
        }))

        plott_lånebetalinger(år_liste, betalt_avdrag, betalt_rente, antall_år, antall_perioder)

        total_rente = sum(betalt_rente)
        total_avdrag = sum(betalt_avdrag)
        total_betaling = total_rente + total_avdrag

        print("\n📌 Oppsummering:")
        print(f"- Du låner: {format_beløp(lånebeløp)}")
        print(f"- Nedbetalingstid: {antall_år} år, med {antall_perioder} termin(er) per år.")
        print(f"- Totalt betalt i avdrag: {format_beløp(total_avdrag)}")
        print(f"- Totalt betalt i renter: {format_beløp(total_rente)}")
        print(f"- Totalt betalt til sammen: {format_beløp(total_betaling)}\n")

        print(f"💬 Du betaler altså {format_beløp(total_betaling)} totalt over {antall_år} år.")

if __name__ == "__main__":
    main()

In [ ]:
# 2 Personlig økonomi: 2.5 Kredittkort
import math
from sympy import sympify
from sympy.core.sympify import SympifyError
import matplotlib.pyplot as plt
from decimal import Decimal, getcontext

getcontext().prec = 12  # eller høyere ved behov

# Funksjon for å hente float med støtte for formler
def hent_float(prompt):
    from sympy import sympify, sqrt, sin, cos, pi, E
    tillatte_symboler = {"sqrt": sqrt, "pi": pi, "e": E, "sin": sin, "cos": cos}
    while True:
        svar = input(prompt).strip().lower()
        if svar == 'q':
            return None
        try:
            verdi = float(sympify(svar, locals=tillatte_symboler))
            return verdi
        except (SympifyError, ValueError, TypeError):
            print("❌ Ugyldig inntasting eller formel. Prøv igjen eller skriv 'q' for å avslutte.")

def finn_vekstfaktor(mnd_rente_prosent):
    return 1 + mnd_rente_prosent / 100

def belop_etter_tid(startbelop, vekstfaktor, antall_maaneder):
    return round(startbelop * (vekstfaktor ** antall_maaneder), 2)

def konverter_til_maaneder(antall, enhet):
    enhet = enhet.lower()
    if enhet == "uker":
        return round(antall * (52 / 12) / 4.3333)
    elif enhet == "år":
        return int(antall * 12)
    elif enhet == "måneder":
        return int(antall)
    else:
        print(f"⚠️ Ukjent tidsenhet '{enhet}'. Antar måneder.")
        return int(antall)

def effektiv_aarlig_rente(mnd_rente_prosent):
    vekstfaktor_mnd = 1 + mnd_rente_prosent / 100
    return round((vekstfaktor_mnd ** 12 - 1) * 100, 2)

def tid_for_dobling(mnd_rente_prosent):
    if mnd_rente_prosent <= 0:
        return float('inf')  # Dobling skjer aldri uten positiv rente
    vekstfaktor = 1 + mnd_rente_prosent / 100
    n = math.log(2) / math.log(vekstfaktor)
    return round(n, 2)

def print_beregningsresultat(tittel, beskrivelse, resultat, enhet="kr"):
    print(f"\n🔢 {tittel}")
    print(f"{beskrivelse} {resultat:.2f} {enhet}")

# 🔹 Kredittkortkalkulator
def kredittkort_beregn():
    print("\n💳 KREDITTKORT KALKULATOR (UTEN INNLEDENDE RENTEFRI PERIODE)")
    startbelop = hent_float("Hvor mye kostet varen / hva er kredittbeløpet? (kr): ")
    if startbelop is None or startbelop <= 0:
        if startbelop is not None: print("❌ Beløpet må være større enn 0.")
        return startbelop is not None

    rente_per_maaned_prosent = hent_float("Hva er den månedlige renten? (%): ")
    if rente_per_maaned_prosent is None or rente_per_maaned_prosent < 0:
        if rente_per_maaned_prosent is not None: print("❌ Månedsrenten kan ikke være negativ.")
        return rente_per_maaned_prosent is not None

    vekstfaktor = finn_vekstfaktor(rente_per_maaned_prosent)
    print(f"Beregnet månedlig vekstfaktor: {vekstfaktor:.4f}")

    betale_nu = input("Skal beløpet betales tilbake umiddelbart (ingen renter påløper)? (ja/nei): ").strip().lower()

    if betale_nu == 'ja':
        print("\nDu har valgt å betale med en gang.")
        print(f"Beløp å betale umiddelbart: {startbelop:.2f} kr")
    elif betale_nu == 'nei':
        print("\nBeløpet utsettes. Renter vil påløpe fra første måned.")
        while True:
            print("\nHva vil du beregne basert på utsatt betaling?")
            print("1: Hvor mye du skylder etter en gitt tid (med renter)")
            print("2: Samlet rentekostnad etter en gitt tid")
            print("3: Effektiv årlig rente")
            print("4: Hvor lang tid tar det før gjelden er doblet?")
            print("q: Gå tilbake til hovedmenyen")
            valg = input("Skriv tallet på valget ditt (1-4) eller 'q': ").strip().lower()

            if valg == 'q':
                print("🔙 Tilbake til hovedmenyen.")
                break
            elif valg == '1' or valg == '2':
                enhet_periode = input("Velg tidsenhet for perioden (uker/måneder/år): ").strip().lower()
                tid_periode_input = hent_float(f"Hvor mange {enhet_periode} har gått siden kjøpet? ")
                if tid_periode_input is None or tid_periode_input < 0:
                    if tid_periode_input is not None: print("❌ Tidsperioden kan ikke være negativ.")
                    continue

                tid_periode_maaneder = konverter_til_maaneder(tid_periode_input, enhet_periode)
                if tid_periode_maaneder < 0:
                    print("❌ Negativ tid i måneder etter konvertering er ikke gyldig.")
                    continue

                print(f"Du har valgt en periode på {tid_periode_input} {enhet_periode}, som tilsvarer ca. {tid_periode_maaneder} måneder.")
                sluttbelop_beregnet = belop_etter_tid(startbelop, vekstfaktor, tid_periode_maaneder)

                if valg == '1':
                    print_beregningsresultat(f"Skyldig Beløp etter {tid_periode_maaneder} mnd", "Totalt skyldig beløp:", sluttbelop_beregnet)
                elif valg == '2':
                    rente_kostnad_beregnet = round(sluttbelop_beregnet - startbelop, 2)
                    print_beregningsresultat(f"Rentekostnad etter {tid_periode_maaneder} mnd", "Total rentekostnad:", rente_kostnad_beregnet)
            elif valg == '3':
                effektiv_rente = effektiv_aarlig_rente(rente_per_maaned_prosent)
                print_beregningsresultat("Effektiv Årlig Rente", "Den effektive årlige renten er:", effektiv_rente, enhet="%")
            elif valg == '4':
                tid_dobling = tid_for_dobling(rente_per_maaned_prosent)
                if tid_dobling == float('inf'):
                    print("⚠️ Med 0 % rente vil gjelden aldri dobles.")
                else:
                    print_beregningsresultat("Tid for dobbling av gjeld", "Antall måneder før gjelden er doblet:", tid_dobling, enhet="måneder")
            else:
                print("❌ Ugyldig valg. Prøv igjen.")
    else:
        print("❌ Ugyldig svar for om du vil betale nå. Skriv 'ja' eller 'nei'.")
    return True

# 🔹 Generell renteberegning med rentefri periode
def generell_renteberegning_med_rentefri_periode():
    print("\n📘 GENERELL RENTEBEREGNING MED VALGFRI RENTEFRI PERIODE")
    startbelop = hent_float("Hva er startbeløpet/lånebeløpet? (kr): ")
    if startbelop is None or startbelop <= 0:
        if startbelop is not None: print("❌ Startbeløpet må være større enn 0.")
        return startbelop is not None

    mnd_rente_prosent = hent_float("Hva er den månedlige renten ETTER en eventuell rentefri periode? (%): ")
    if mnd_rente_prosent is None or mnd_rente_prosent < 0:
        if mnd_rente_prosent is not None: print("❌ Månedsrenten kan ikke være negativ.")
        return mnd_rente_prosent is not None

    enhet_rentefri = input("Velg tidsenhet for rentefri periode (uker/måneder/år, skriv '0' hvis ingen): ").strip().lower()
    rentefri_tid_input = 0
    if enhet_rentefri != '0':
        rentefri_tid_input = hent_float(f"Hvor lang er den rentefrie perioden i {enhet_rentefri}? ")
        if rentefri_tid_input is None or rentefri_tid_input < 0:
            if rentefri_tid_input is not None: print("❌ Rentefri periode kan ikke være negativ.")
            return rentefri_tid_input is not None

    rentefri_mnd = konverter_til_maaneder(rentefri_tid_input, enhet_rentefri if enhet_rentefri != '0' else "måneder")
    if rentefri_mnd < 0:
        print("❌ Negativ rentefri periode etter konvertering.")
        return True

    enhet_total = input("Velg total tidsenhet lånet/beløpet har stått (uker/måneder/år): ").strip().lower()
    total_tid_input = hent_float(f"Hvor lenge har lånet vært aktivt i {enhet_total}? ")
    if total_tid_input is None or total_tid_input < 0:
        if total_tid_input is not None: print("❌ Total tid kan ikke være negativ.")
        return total_tid_input is not None

    total_tid_mnd = konverter_til_maaneder(total_tid_input, enhet_total)

    if total_tid_mnd <= rentefri_mnd:
        print("ℹ️ Hele perioden er rentefri – ingen rente påløper.")
        print(f"Beløpet etter {total_tid_mnd} måneder er fortsatt: {startbelop:.2f} kr")
        return True

    rentebelagt_tid = total_tid_mnd - rentefri_mnd
    vekstfaktor = finn_vekstfaktor(mnd_rente_prosent)
    sluttbelop = belop_etter_tid(startbelop, vekstfaktor, rentebelagt_tid)

    print(f"\n⏳ Totalt måneder: {total_tid_mnd} | Rentefri måneder: {rentefri_mnd} | Renteperiode: {rentebelagt_tid}")
    print_beregningsresultat(f"Sluttbeløp etter {total_tid_mnd} mnd", "Beløpet du skylder totalt:", sluttbelop)
    print_beregningsresultat("Total rentekostnad", "Rentekostnaden etter renteperiode:", sluttbelop - startbelop)

    # Dobblingstid
    tid_dobling = tid_for_dobling(mnd_rente_prosent)
    if tid_dobling == float('inf'):
        print("⚠️ Med 0 % rente vil gjelden aldri dobles.")
    else:
        print_beregningsresultat("Tid for dobbling av gjeld (uten rentefri periode)", "Antall måneder før gjelden er doblet:", tid_dobling, enhet="måneder")

    return True

# 🔸 Hovedmeny
def hovedprogram():
    while True:
        print("\n📌 HOVEDMENY – Velg en kalkulator:")
        print("1: Kredittkortkalkulator (Renter fra første måned)")
        print("2: Generell renteberegning (Med valgfri rentefri periode)")
        print("q: Avslutt programmet")
        valg = input("Skriv tallet på valget ditt (1-2) eller 'q': ").strip().lower()

        if valg == 'q':
            print("✅ Avslutter programmet. Ha en fin dag!")
            break
        elif valg == '1':
            kredittkort_beregn()
        elif valg == '2':
            generell_renteberegning_med_rentefri_periode()
        else:
            print("❌ Ugyldig valg. Prøv igjen.")

# Kjør programmet
if __name__ == "__main__":
    hovedprogram()

# $\color{blue}{\text{Kapittel 3 - Formler og geometri}}$

In [3]:
# 3 Formler og geometri: Delkapittel 3.1 Formelregning
from sympy import symbols, Eq, parse_expr, Symbol, simplify, S, Number as SympyNumber
from sympy.core.relational import Relational
from sympy.solvers import solve
from sympy.solvers.inequalities import solve_univariate_inequality
from sympy.parsing.sympy_parser import standard_transformations, implicit_multiplication_application

# Kompatibilitet for RelationalOp
try:
    from sympy.core.relational import RelationalOp
except ImportError:
    RelationalOp = Relational # For eldre SymPy-versjoner

# Konfigurasjon for parser
transformations = standard_transformations + (implicit_multiplication_application,)

# Hjelpeordbok for symboler som kan kollidere med SymPy-konstanter
_RESERVED_NAMES_AS_SYMBOLS = {name: Symbol(name) for name in ["E", "I", "N", "O", "Q", "S"]}

# -------------------- Hjelpefunksjoner for Parsing --------------------
def custom_parse_expr(expr_str, local_dict_override=None, **kwargs):
    """
    Parser et uttrykk og sikrer at visse navn (E, I, N, O, Q, S)
    behandles som symboler, med mindre annet er spesifisert.
    Bruker standard 'transformations'.
    """
    effective_local_dict = _RESERVED_NAMES_AS_SYMBOLS.copy()
    if local_dict_override: # For tilfeller der vi IKKE vil overstyre (f.eks. verdiparsing)
        effective_local_dict.update(local_dict_override)
    
    if 'transformations' not in kwargs:
        kwargs['transformations'] = transformations
        
    return parse_expr(expr_str, local_dict=effective_local_dict, **kwargs)

# -------------------- Kjernefunksjoner --------------------

def parse_ligning(expr_str):
    """Parser et uttrykk pa formen 'venstre = hoyre' til en sympy-ligning."""
    venstre, hoyre = expr_str.split('=', maxsplit=1)
    return Eq(custom_parse_expr(venstre.strip()),
              custom_parse_expr(hoyre.strip()))

def løs_uttrykk(uttrykk_str):
    """Loser en ligning, ulikhet eller system av ligninger."""
    try:
        if ';' in uttrykk_str:
            ligninger = [parse_ligning(eq.strip()) for eq in uttrykk_str.split(';')]
            return solve(ligninger, dict=True)

        is_potential_inequality = any(op in uttrykk_str for op in ['<', '>', '<=', '>='])
        is_assignment_like = '=' in uttrykk_str and not any(op in uttrykk_str for op in ['<=', '>=', '!=', '=='])


        if is_potential_inequality and not is_assignment_like:
            if " & " in uttrykk_str or " | " in uttrykk_str:
                 return "X Sammensatte ulikheter med '&' eller '|' stottes ikke direkte. Prov en ulikhet."
            
            if '==' in uttrykk_str: 
                pass
            else:
                ulikhet = custom_parse_expr(uttrykk_str) 
                if not isinstance(ulikhet, (Relational, RelationalOp)):
                    return f"X Uttrykket '{uttrykk_str}' er ikke en gyldig ulikhetsstruktur."
                variabler = sorted(list(ulikhet.free_symbols), key=lambda s: s.name)
                if not variabler:
                    simplified_truth_value = simplify(ulikhet)
                    if simplified_truth_value == S.true: return S.Reals
                    if simplified_truth_value == S.false: return S.EmptySet
                    return f"Symbolsk konstant ulikhet: {ulikhet}"
                hoved_var = variabler[0]
                return solve_univariate_inequality(ulikhet, hoved_var, relational=False)

        if '==' in uttrykk_str:
             lhs, rhs = uttrykk_str.split('==', 1)
             ligning = Eq(custom_parse_expr(lhs.strip()), custom_parse_expr(rhs.strip()))
        elif '=' not in uttrykk_str: 
            parsed_lhs = custom_parse_expr(uttrykk_str)
            ligning = Eq(parsed_lhs, 0)
        else: 
            ligning = parse_ligning(uttrykk_str)
        
        return solve(ligning, dict=True)
    except Exception as e:
        return f"X Feil under losning: {e}"

def evaluer_uttrykk(uttrykk_str, kjente_verdier, symbolsk=True):
    """Evaluerer et uttrykk (eller hoyreside av en likning) med gitte verdier."""
    try:
        expr_to_parse = uttrykk_str
        if '=' in uttrykk_str and '==' not in uttrykk_str :
            parts = uttrykk_str.split('=', maxsplit=1)
            if not (parts[0].endswith('<') or parts[0].endswith('>') or parts[0].endswith('!')):
                 _, hoyre_side_str = parts
                 expr_to_parse = hoyre_side_str.strip()

        parsed_uttrykk = custom_parse_expr(expr_to_parse)
        
        subs_dict = {}
        for s in parsed_uttrykk.free_symbols:
            if s.name in kjente_verdier:
                subs_dict[s] = kjente_verdier[s.name]

        evaluert_uttrykk = parsed_uttrykk.subs(subs_dict)

        is_numeric_evaluable = hasattr(evaluert_uttrykk, 'is_Number') and evaluert_uttrykk.is_Number
        if not is_numeric_evaluable: 
             is_numeric_evaluable = hasattr(evaluert_uttrykk, 'is_number') and evaluert_uttrykk.is_number
        if not is_numeric_evaluable:
            is_numeric_evaluable = isinstance(evaluert_uttrykk, SympyNumber) or not evaluert_uttrykk.free_symbols

        if not symbolsk and is_numeric_evaluable:
            resultat = evaluert_uttrykk.evalf()
        else:
            resultat = evaluert_uttrykk
        
        return resultat
    except Exception as e:
        return f"X Feil under evaluering: {e}"

def løs_for_variabel(uttrykk_str, mål_variabel_navn, kjente_verdier):
    """Loser en ukjent gitt kjente verdier."""
    try:
        eq_str = uttrykk_str.split(';')[0].strip()
        
        if '==' in eq_str:
            lhs, rhs = eq_str.split('==', 1)
            ligning = Eq(custom_parse_expr(lhs.strip()), custom_parse_expr(rhs.strip()))
        elif '=' in eq_str:
            ligning = parse_ligning(eq_str) 
        else:
            ligning = Eq(custom_parse_expr(eq_str.strip()),0)

        mål_symbol = Symbol(mål_variabel_navn)
        
        subs_for_eq = {}
        for s in ligning.free_symbols:
            if s.name in kjente_verdier and s.name != mål_variabel_navn:
                 subs_for_eq[s] = kjente_verdier[s.name]

        substituert_ligning = ligning.subs(subs_for_eq)
        
        if mål_symbol not in substituert_ligning.free_symbols:
            if hasattr(substituert_ligning, 'lhs') and hasattr(substituert_ligning, 'rhs'):
                simplified_eq_check = simplify(substituert_ligning.lhs - substituert_ligning.rhs)
                if simplified_eq_check == 0: 
                    return f"Ligningen er alltid sann for de gitte verdiene. '{mål_variabel_navn}' kan vaere hva som helst (eller ikke relevant)."
                elif not substituert_ligning.free_symbols: 
                     return "Ligningen er usann/en selvmotsigelse for de gitte verdiene. Ingen losning."
            return f"Variabelen '{mål_variabel_navn}' finnes ikke i ligningen etter substitusjon, eller ligningen er ikke avhengig av den."

        return solve(substituert_ligning, mål_symbol)
    except Exception as e:
        return f"X Feil under isolering: {e}"

def hent_kjente_verdier():
    """Spor brukeren om variableverdier i formatet x=3, y=pi/2."""
    raw_input_str = input("Skriv inn kjente verdier (f.eks. x=3, y=pi/2, z=sqrt(2)):\n> ")

    try:
        verdier = {}
        if raw_input_str.strip() == "": return verdier
        for item in raw_input_str.split(','):
            key_val_pair = item.strip().split('=', maxsplit=1)
            if len(key_val_pair) != 2:
                print(f"Advarsel: Ugyldig format for '{item.strip()}'. Hopper over.")
                continue
            key, val_str = key_val_pair
            key = key.strip()
            val_str = val_str.strip()

            try:
                parsed_val = parse_expr(val_str, transformations=transformations, evaluate=True, local_dict={})
            except SyntaxError: 
                try:
                    parsed_val = float(val_str) 
                except ValueError:
                    try:
                        parsed_val = int(val_str) 
                    except ValueError:
                        print(f"Advarsel: Kunne ikke parse verdien '{val_str}' for '{key}'. Hopper over.")
                        continue
            
            is_num_type = isinstance(parsed_val, (int, float))
            is_sympy_num_obj = hasattr(parsed_val, 'is_Number') and parsed_val.is_Number
            if not is_sympy_num_obj: 
                is_sympy_num_obj = hasattr(parsed_val, 'is_number') and parsed_val.is_number
            
            if is_num_type and not is_sympy_num_obj : 
                 verdier[key] = SympyNumber(parsed_val)
            else: 
                verdier[key] = parsed_val
        return verdier
    except Exception as e:
        print(f"Advarsel: Ugyldig format for kjente verdier ({e}). Prov igjen med f.eks. x=3, y=pi/2.")
        return hent_kjente_verdier()

# -------------------- Hovedprogram --------------------
def main():
    print("Formel- og uttrykksloser med SymPy")
    print("Skriv 'q' nar som helst for a avslutte programmet.\n")

    if not hasattr(SympyNumber, 'is_Number') and hasattr(SympyNumber, 'is_number'):
        SympyNumber.is_Number = property(lambda self: self.is_number)
    elif not hasattr(SympyNumber, 'is_Number'): 
         SympyNumber.is_Number = property(lambda self: isinstance(self, SympyNumber))


    while True:
        uttrykk_str_input = input("Skriv inn et uttrykk, en ligning, ulikhet, eller system (separert med ';'):\n> ")
        if uttrykk_str_input.lower() == 'q':
            print("Avslutter programmet. Ha en fin dag!")
            break

        handling = input("Velg handling:\n1 = Evaluer uttrykk\n2 = Los ligning(er)/ulikhet\n3 = Isoler en variabel\n(q for a avslutte)\n> ")
        if handling.lower() == 'q':
            print("Avslutter programmet. Ha en fin dag!")
            break

        if handling == '1':
            verdier_input = hent_kjente_verdier()
            resultat = evaluer_uttrykk(uttrykk_str_input, verdier_input, symbolsk=False)
            
            if isinstance(resultat, str) and resultat.startswith("X"): print(resultat)
            elif hasattr(resultat, 'evalf'): 
                try:
                    num_val = resultat.evalf() 
                    if hasattr(num_val, 'is_Integer') and num_val.is_Integer:
                        print(f"Resultat: {int(num_val)}")
                    elif (hasattr(num_val, 'is_Float') and num_val.is_Float) or \
                         (hasattr(num_val, 'is_Rational') and num_val.is_Rational) or \
                         (hasattr(num_val, 'as_real_imag')): 
                        try:
                            py_float_val = float(num_val)
                            if py_float_val == int(py_float_val): 
                                print(f"Resultat: {int(py_float_val)}")
                            else:
                                print(f"Resultat: {py_float_val:.2f}") 
                        except (TypeError, ValueError, OverflowError): 
                             print(f"Resultat: {num_val}") 
                    else: 
                        print(f"Resultat: {num_val}")

                except (TypeError, AttributeError, ValueError): 
                    print(f"Resultat: {resultat}") 
            else: 
                print(f"Resultat: {resultat}")


        elif handling == '2':
            resultat = løs_uttrykk(uttrykk_str_input)
            print("Losning(er):", resultat)

        elif handling == '3':
            verdier_input = hent_kjente_verdier()
            mål_input = input("Hvilken variabel onsker du a isolere/lose for?\n> ")
            resultat = løs_for_variabel(uttrykk_str_input, mål_input, verdier_input)
            
            if isinstance(resultat, str) and resultat.startswith("X"): print(resultat)
            elif isinstance(resultat, str): print(f"Info: {resultat}") 
            elif isinstance(resultat, list):
                if not resultat: print(f"Ingen losning funnet for {mål_input}.")
                elif len(resultat) == 1: print(f"Isolert losning for {mål_input}: {resultat[0]}")
                else: print(f"Isolerte losninger for {mål_input}: {resultat}")
            else: print(f"Uventet resultat: {resultat}")
        else:
            print("Advarsel: Ugyldig valg. Prov igjen.")
        print("\n----------------------------\n")

if __name__ == "__main__":
    main()

Formel- og uttrykksloser med SymPy
Skriv 'q' nar som helst for a avslutte programmet.



Skriv inn et uttrykk, en ligning, ulikhet, eller system (separert med ';'):
>  2x-4=x+3
Velg handling:
1 = Evaluer uttrykk
2 = Los ligning(er)/ulikhet
3 = Isoler en variabel
(q for a avslutte)
>  2


Losning(er): [{x: 7}]

----------------------------



Skriv inn et uttrykk, en ligning, ulikhet, eller system (separert med ';'):
>  q


Avslutter programmet. Ha en fin dag!


In [4]:
# 3 Formler og geometri: Delkapittel 3.3 Enheter. Ordbok for prefikser og deres verdier. 
# Ordbok for prefikser med symbol, verdi og navn
import re

prefixes = {
    'T': {'value': 1_000_000_000_000, 'name': 'tera'},
    'G': {'value': 1_000_000_000, 'name': 'giga'},
    'M': {'value': 1_000_000, 'name': 'mega'},
    'k': {'value': 1_000, 'name': 'kilo'},
    'h': {'value': 100, 'name': 'hekto'},
    'd': {'value': 0.1, 'name': 'desi'},
    'c': {'value': 0.01, 'name': 'centi'},
    'm': {'value': 0.001, 'name': 'milli'},
    'μ': {'value': 0.000001, 'name': 'mikro'},
    'n': {'value': 0.000000001, 'name': 'nano'}
}

# Tid i sekunder
time_units = {
    's': 1,
    'min': 60,
    'h': 3600,
    'd': 86400,
    'y': 31536000
}

# Lengde i meter
length_units = {
    'm': 1,
    'km': 1000,
    'dm': 0.1,
    'cm': 0.01,
    'mm': 0.001
}

# Volum i m³
volume_units = {
    'm³': 1,
    'dm³': 0.001,
    'cm³': 0.000001,
    'liter': 0.001,
    'ml': 0.000001
}

# Masse i kg
mass_units = {
    'kg': 1,
    'g': 0.001,
    'mg': 0.000001
}

# Energi i joule (J)
energy_units = {
    'J': 1,
    'kJ': 1000,
    'MJ': 1_000_000,
    'Wh': 3600,
    'kWh': 3_600_000
}

# Hastighet (valgfri)
speed_units = {
    'm/s': 1,
    'km/h': 1000/3600,
    'knop': 1852/3600
}

def parse_value_unit(input_str):
    """
    Tolker en verdi med prefiks og enhet.
    Eksempel: '1.5 km', '200 mg', '3.2 L'
    Returnerer verdi i SI-enhet og selve enheten.
    """
    input_str = input_str.strip().replace(',', '.')  # tillat komma som desimal
    # Mønster for verdi + prefiks + enhet
    pattern = r"^([\d.]+)\s*([TGMkhdcμmn]?)([a-zA-Z³²/]+)$"
    match = re.match(pattern, input_str)
    if not match:
        raise ValueError(f"Ugyldig format: {input_str}")
    value = float(match.group(1))
    prefix = match.group(2)
    unit = match.group(3)

    prefix_factor = prefixes.get(prefix, {'value': 1})['value']

    # Her må vi finne hva slags type enhet det er for korrekt konvertering:
    if unit in length_units:
        base_value = value * prefix_factor * length_units[unit]
        base_unit = 'm'
    elif unit in volume_units:
        base_value = value * prefix_factor * volume_units[unit]
        base_unit = 'm³'
    elif unit in mass_units:
        base_value = value * prefix_factor * mass_units[unit]
        base_unit = 'kg'
    elif unit in time_units:
        base_value = value * prefix_factor * time_units[unit]
        base_unit = 's'
    elif unit in energy_units:
        base_value = value * prefix_factor * energy_units[unit]
        base_unit = 'J'
    elif unit in speed_units:
        base_value = value * prefix_factor * speed_units[unit]
        base_unit = 'm/s'
    else:
        # Ikke støttet enhet, men la den stå som er
        base_value = value * prefix_factor
        base_unit = unit

    return base_value, base_unit

def convert_to_unit(value_si, target_unit):
    """
    Konverterer en verdi i SI-enhet til ønsket enhet.
    Må vite type enhet for å finne riktig konverteringsfaktor.
    """
    # Finn hvilken kategori target_unit tilhører:
    if target_unit in length_units:
        return value_si / length_units[target_unit], target_unit
    elif target_unit in volume_units:
        return value_si / volume_units[target_unit], target_unit
    elif target_unit in mass_units:
        return value_si / mass_units[target_unit], target_unit
    elif target_unit in time_units:
        return value_si / time_units[target_unit], target_unit
    elif target_unit in energy_units:
        return value_si / energy_units[target_unit], target_unit
    elif target_unit in speed_units:
        return value_si / speed_units[target_unit], target_unit
    else:
        raise ValueError(f"Ukjent målenhet: {target_unit}")

def input_value_with_unit(prompt):
    while True:
        try:
            val, unit = parse_value_unit(input(prompt))
            return val, unit
        except Exception as e:
            print("Feil:", e, "- prøv igjen. (F.eks. 1.5 km, 200 mg)")

# Del 4 funksjoner for fart, utslipp og konvertering

def diesel_forbruk_km_per_liter(l_per_mil):
    # L/mil → km/l
    return 10 / l_per_mil

def co2_utslipp_per_liter(km_per_liter, gram_per_km):
    total_gram = km_per_liter * gram_per_km
    return total_gram / 1000  # kg

def knop_til_kmh(knop):
    return knop * 1.852

def kmh_til_knop(kmh):
    return kmh / 1.852

# Del 5 - Medisinsk doseutregning

def medisinsk_dose(masse_kg, dose_mg_per_kg):
    """
    Regner total dose i mg gitt kroppsmasse og dose per kg
    """
    return masse_kg * dose_mg_per_kg

# Formler for fart, tid og strekning

def regn_ut_fart(s=None, t=None, v=None):
    # Alle i SI: s (m), t (s), v (m/s)
    if v is None and s is not None and t is not None:
        if t == 0:
            raise ValueError("Tid kan ikke være null.")
        return s / t
    elif s is None and v is not None and t is not None:
        return v * t
    elif t is None and s is not None and v is not None:
        if v == 0:
            raise ValueError("Fart kan ikke være null.")
        return s / v
    else:
        raise ValueError("Nøyaktig én variabel må være None for å regne ut.")

# Formler for masse, volum og tetthet

def regn_ut_tetthet(m=None, v=None, d=None):
    # Tetthet = masse / volum (kg/m³)
    if d is None and m is not None and v is not None:
        if v == 0:
            raise ValueError("Volum kan ikke være null.")
        return m / v
    elif m is None and d is not None and v is not None:
        return d * v
    elif v is None and m is not None and d is not None:
        if d == 0:
            raise ValueError("Tetthet kan ikke være null.")
        return m / d
    else:
        raise ValueError("Nøyaktig én variabel må være None for å regne ut.")

# Hovedmeny Del 6 - Fysiske formler med enheter

def del_6_meny():
    print("\nDel 6: Fysiske formler og enhetsberegninger")
    print("1. Regn ut fart (s, t, v)")
    print("2. Regn ut tetthet (m, v, d)")
    valg = input("Velg et alternativ (1-2): ")

    if valg == '1':
        print("Oppgi to av tre variabler (strekning, tid, fart). Bruk enheter. (F.eks. '10 km', '30 min')")
        s_val, s_unit = input_value_with_unit("Strekning: ")
        t_val, t_unit = input_value_with_unit("Tid: ")
        v_val, v_unit = input_value_with_unit("Fart: ")
        # Finn hvilken som er None (bruk verdi 0 eller -1 som tomt)
        # Vi krever at bruker skriver "0 enhet" eller "0" hvis ukjent
        inputs = {'s': s_val if s_val > 0 else None,
                  't': t_val if t_val > 0 else None,
                  'v': v_val if v_val > 0 else None}

        try:
            if inputs['v'] is None:
                v = regn_ut_fart(s=inputs['s'], t=inputs['t'], v=None)
                # Konverter fart til ønsket enhet (la bruker velge)
                v_out, v_unit_out = convert_to_unit(v, 'm/s')
                print(f"Fart: {v:.3f} m/s")
            elif inputs['s'] is None:
                s = regn_ut_fart(s=None, t=inputs['t'], v=inputs['v'])
                print(f"Strekning: {s:.3f} m")
            elif inputs['t'] is None:
                t = regn_ut_fart(s=inputs['s'], t=None, v=inputs['v'])
                print(f"Tid: {t:.3f} sekunder")
            else:
                print("Skriv 0 for ukjent variabel.")
        except Exception as e:
            print("Feil:", e)

    elif valg == '2':
        print("Oppgi to av tre variabler (masse, volum, tetthet). Bruk enheter.")
        m_val, m_unit = input_value_with_unit("Masse: ")
        v_val, v_unit = input_value_with_unit("Volum: ")
        d_val, d_unit = input_value_with_unit("Tetthet: ")
        inputs = {'m': m_val if m_val > 0 else None,
                  'v': v_val if v_val > 0 else None,
                  'd': d_val if d_val > 0 else None}

        try:
            if inputs['d'] is None:
                d = regn_ut_tetthet(m=inputs['m'], v=inputs['v'], d=None)
                print(f"Tetthet: {d:.3f} kg/m³")
            elif inputs['m'] is None:
                m = regn_ut_tetthet(m=None, v=inputs['v'], d=inputs['d'])
                print(f"Masse: {m:.3f} kg")
            elif inputs['v'] is None:
                v = regn_ut_tetthet(m=inputs['m'], v=None, d=inputs['d'])
                print(f"Volum: {v:.6f} m³")
            else:
                print("Skriv 0 for ukjent variabel.")
        except Exception as e:
            print("Feil:", e)
    else:
        print("Ugyldig valg.")

# Menyer del 1-5

def del_1_meny():
    print("\nDel 1: Konverter en verdi med prefiks")
    val, unit = input_value_with_unit("Skriv verdi med enhet (f.eks. '1.5 km'): ")
    target_unit = input("Til enhet (f.eks. m, cm, liter): ").strip()
    try:
        result, res_unit = convert_to_unit(val, target_unit)
        print(f"{val} {unit} = {result:.6f} {res_unit}")
    except Exception as e:
        print("Feil:", e)

def del_2_meny():
    print("\nDel 2: Konverter tid")
    val, unit = input_value_with_unit("Skriv tid med enhet (f.eks. '2 h'): ")
    target_unit = input("Til tidsenhet (s, min, h, d, y): ").strip()
    try:
        result, res_unit = convert_to_unit(val, target_unit)
        print(f"{val} {unit} = {result:.2f} {res_unit}")
    except Exception as e:
        print("Feil:", e)

def del_4_meny():
    print("\nDel 4: Fart, forbruk og utslipp")
    print("1. Regn ut km per liter (fra L/mil)")
    print("2. Regn ut CO₂-utslipp per liter (fra g/km)")
    print("3. Konverter knop til km/h")
    print("4. Konverter km/h til knop")

    valg = input("Velg et alternativ (1-4): ")

    if valg == '1':
        lpm = float(input("Oppgi forbruk i L/mil: "))
        kmpl = diesel_forbruk_km_per_liter(lpm)
        print(f"Bilen kjører {kmpl:.2f} km per liter diesel.")
    elif valg == '2':
        gram_per_km = float(input("Oppgi CO₂-utslipp i gram per km: "))
        kmpl = float(input("Oppgi bilens rekkevidde i km per liter: "))
        utslipp = co2_utslipp_per_liter(kmpl, gram_per_km)
        print(f"Bilen slipper ut {utslipp:.2f} kg CO₂ per liter diesel.")
    elif valg == '3':
        knop = float(input("Oppgi farten i knop: "))
        print(f"{knop} knop = {knop_til_kmh(knop):.2f} km/h")
    elif valg == '4':
        kmh = float(input("Oppgi farten i km/h: "))
        print(f"{kmh} km/h = {kmh_til_knop(kmh):.2f} knop")
    else:
        print("Ugyldig valg.")

def del_5_meny():
    print("\nDel 5: Medisinsk doseutregning")
    masse, masse_enhet = input_value_with_unit("Oppgi kroppsmasse (f.eks. '70 kg'): ")
    dose_per_kg = float(input("Oppgi dose i mg per kg: "))
    total_dose = medisinsk_dose(masse, dose_per_kg)
    print(f"Total dose: {total_dose:.2f} mg")

# Hovedmeny

def main():
    while True:
        print("\nEnhetskonvertering og beregninger")
        print("1. Konverter en verdi med prefiks")
        print("2. Konverter tid")
        print("3. Avslutt")
        print("4. Fart og sammensatte enheter")
        print("5. Tetthet og medisinens styrke")
        print("6. Fysiske formler og enhetsberegninger")
        print("q: Avslutt programmet")

        valg = input("Velg et alternativ: ")
        if valg == "q":
            print("Program avsluttes.")
            break
        choice = input("Velg et alternativ (1-6): ")
        if choice == '1':
            del_1_meny()
        elif choice == '2':
            del_2_meny()
        elif choice == '3':
            print("Avslutter programmet.")
            break
        elif choice == '4':
            del_4_meny()
        elif choice == '5':
            del_5_meny()
        elif choice == '6':
            del_6_meny()
        else:
            print("Ugyldig valg. Prøv igjen.")

if __name__ == "__main__":
    main()


Enhetskonvertering og beregninger
1. Konverter en verdi med prefiks
2. Konverter tid
3. Avslutt
4. Fart og sammensatte enheter
5. Tetthet og medisinens styrke
6. Fysiske formler og enhetsberegninger
q: Avslutt programmet


Velg et alternativ:  4
Velg et alternativ (1-6):  1



Del 1: Konverter en verdi med prefiks


Skriv verdi med enhet (f.eks. '1.5 km'):  1.4 km
Til enhet (f.eks. m, cm, liter):  m


1400.0 m = 1400.000000 m

Enhetskonvertering og beregninger
1. Konverter en verdi med prefiks
2. Konverter tid
3. Avslutt
4. Fart og sammensatte enheter
5. Tetthet og medisinens styrke
6. Fysiske formler og enhetsberegninger
q: Avslutt programmet


Velg et alternativ:  q


Program avsluttes.


In [ ]:
# 3 Formler og geometri: Delkapittel 3.3 Enheter. Omregning mellom tommer og cm 2: Omregning av sammensatte enheter (km/t og m/s) 3: Energibruk per mil (kWh/mil)
from fractions import Fraction

def parse_input(value):
    try:
        return float(Fraction(value))
    except ValueError:
        print("Ugyldig inndata. Bruk tall eller brøk (f.eks. 1/2).")
        return None

def konverter_lengde():
    while True:
        print("\n--- Omregning mellom tommer og cm ---")
        print("1: Tommer til cm")
        print("2: Cm til tommer")
        print("q: Tilbake til hovedmenyen")
        valg = input("Velg (1/2), eller q: ")
        if valg == "q":
            return
        elif valg == "1":
            tommer = parse_input(input("Antall tommer: "))
            if tommer is not None:
                print(f"{tommer} tommer = {tommer * 2.54:.2f} cm")
        elif valg == "2":
            cm = parse_input(input("Antall cm: "))
            if cm is not None:
                print(f"{cm} cm = {cm / 2.54:.2f} tommer")
        else:
            print("Ugyldig valg. Prøv igjen.")

def konverter_fart():
    while True:
        print("\n--- Omregning mellom km/t og m/s ---")
        print("1: km/t til m/s")
        print("2: m/s til km/t")
        print("q: Tilbake til hovedmenyen")
        valg = input("Velg (1/2), eller q: ")
        if valg == "q":
            return
        elif valg == "1":
            kmh = parse_input(input("Fart i km/t: "))
            if kmh is not None:
                print(f"{kmh} km/t = {kmh / 3.6:.2f} m/s")
        elif valg == "2":
            ms = parse_input(input("Fart i m/s: "))
            if ms is not None:
                print(f"{ms} m/s = {ms * 3.6:.2f} km/t")
        else:
            print("Ugyldig valg. Prøv igjen.")

def energibruk_per_mil():
    while True:
        print("\n--- Energibruk per mil (kWh/mil) ---")
        print("q: Tilbake til hovedmenyen")
        forbruk_input = input("Total energiforbruk i kWh (eller q): ")
        if forbruk_input == "q":
            return
        distanse_input = input("Kjørt distanse i km (eller q): ")
        if distanse_input == "q":
            return
        forbruk = parse_input(forbruk_input)
        distanse = parse_input(distanse_input)
        if forbruk is not None and distanse is not None and distanse != 0:
            per_mil = forbruk / (distanse / 10)
            print(f"Energibruk per mil: {per_mil:.2f} kWh/mil")
        else:
            print("Ugyldige verdier. Prøv igjen.")

def hovedmeny():
    while True:
        print("\n=== HOVEDMENY ===")
        print("1: Omregning mellom tommer og cm")
        print("2: Omregning av sammensatte enheter (km/t og m/s)")
        print("3: Energibruk per mil (kWh/mil)")
        print("q: Avslutt programmet")

        valg = input("Velg et alternativ: ")
        if valg == "q":
            print("Program avsluttes.")
            break
        elif valg == "1":
            konverter_lengde()
        elif valg == "2":
            konverter_fart()
        elif valg == "3":
            energibruk_per_mil()
        else:
            print("Ugyldig valg. Prøv igjen.")

# Start programmet
hovedmeny()

In [ ]:
# 3 Formler og geometri: Forholdskalkulator med saft og vann som eksempel
from fractions import Fraction

# 🔧 Konverteringsfunksjon for mengde-input
def les_mengde(prompt):
    while True:
        tekst = input(f"{prompt} (f.eks. 1.5l, 15dl, 100cl, 250ml, 1/2l) eller q for å avslutte: ").strip().lower()
        if tekst == "q":
            print("\n📤 Programmet avsluttes. Takk for at du brukte kalkulatoren! 🧃")
            return None
        try:
            if tekst.endswith("l") and not tekst.endswith("ml"):
                mengde = Fraction(tekst[:-1]) * 10         # liter til dl
            elif tekst.endswith("dl"):
                mengde = Fraction(tekst[:-2])              # dl
            elif tekst.endswith("cl"):
                mengde = Fraction(tekst[:-2]) / 10         # cl til dl
            elif tekst.endswith("ml"):
                mengde = Fraction(tekst[:-2]) / 100        # ml til dl
            else:
                mengde = Fraction(tekst)                   # tolkes som dl
            return float(mengde)
        except:
            print("❌ Ugyldig mengde. Prøv igjen.")

# 📦 Les forhold saft:vann
def les_forhold():
    while True:
        forhold = input("Oppgi forhold mellom saft og vann (f.eks. 1:5 eller 1/6:5) eller q for å avslutte: ").strip()
        if forhold.lower() == "q":
            print("\n📤 Programmet avsluttes.")
            return None
        try:
            saft_del, vann_del = forhold.split(":")
            saft = Fraction(saft_del)
            vann = Fraction(vann_del)
            return saft, vann
        except:
            print("❌ Ugyldig format. Bruk f.eks. 1:5 eller 1/3:2")

# 🧾 Vis resultat i flere enheter
def vis_resultat(saft_dl, vann_dl):
    total_dl = saft_dl + vann_dl
    print(f"\n🧮 Resultat:")
    print(f"- Saft: {saft_dl:.2f} dl ({saft_dl/10:.2f} l, {saft_dl*10:.0f} cl, {saft_dl*100:.0f} ml)")
    print(f"- Vann: {vann_dl:.2f} dl ({vann_dl/10:.2f} l, {vann_dl*10:.0f} cl, {vann_dl*100:.0f} ml)")
    print(f"- Totalt: {total_dl:.2f} dl ({total_dl/10:.2f} l)")

# 🔢 Kalkulasjoner
def beregn_fra_totalmengde():
    forhold = les_forhold()
    if forhold is None:
        return
    saft, vann = forhold
    total = les_mengde("Hvor mye ferdig drikke ønsker du")
    if total is None:
        return
    total_deler = saft + vann
    saft_dl = (saft / total_deler) * total
    vann_dl = (vann / total_deler) * total
    vis_resultat(saft_dl, vann_dl)

def beregn_fra_saftmengde():
    forhold = les_forhold()
    if forhold is None:
        return
    saft, vann = forhold
    saft_dl = les_mengde("Hvor mye saft har du")
    if saft_dl is None:
        return
    faktor = saft_dl / float(saft)
    vann_dl = float(vann) * faktor
    vis_resultat(saft_dl, vann_dl)

def beregn_fra_vannmengde():
    forhold = les_forhold()
    if forhold is None:
        return
    saft, vann = forhold
    vann_dl = les_mengde("Hvor mye vann har du")
    if vann_dl is None:
        return
    faktor = vann_dl / float(vann)
    saft_dl = float(saft) * faktor
    vis_resultat(saft_dl, vann_dl)

def beregn_fra_prosent():
    prosent_saft = input("Hvor mange prosent av drikken skal være saft? (f.eks. 20) eller q for å avslutte: ")
    if prosent_saft.lower() == "q":
        print("\n📤 Programmet avsluttes.")
        return
    try:
        prosent_saft = float(prosent_saft)
        total = les_mengde("Hvor mye ferdig drikke ønsker du")
        if total is None:
            return
        saft_dl = (prosent_saft / 100) * total
        vann_dl = total - saft_dl
        vis_resultat(saft_dl, vann_dl)
    except:
        print("❌ Ugyldig prosent. Prøv igjen.")

# 💰 Fordeling av beløp etter brøk
def fordel_beløp():
    print("\n💰 Fordeling av beløp mellom tre personer der to får oppgitt brøk og siste får resten.")
    try:
        total = float(input("Hvor mye penger skal fordeles totalt?: "))
        andel1 = Fraction(input("Hvor stor andel skal første person ha?: "))
        andel2 = Fraction(input("Hvor stor andel skal andre person ha?: "))
        if andel1 + andel2 > 1:
            print("❌ Summen av andelene er mer enn 1. Prøv igjen.")
            return
        andel3 = 1 - andel1 - andel2
        beløp1 = total * float(andel1)
        beløp2 = total * float(andel2)
        beløp3 = total * float(andel3)
        print(f"\n📊 Fordeling:")
        print(f"- Person 1 ({andel1}): {beløp1:.2f} kr")
        print(f"- Person 2 ({andel2}): {beløp2:.2f} kr")
        print(f"- Person 3 ({andel3}): {beløp3:.2f} kr")
    except:
        print("❌ Ugyldig input. Prøv igjen.")

# 🧃 Hovedmeny
def hovedprogram():
    while True:
        print("\n🧃 Forholdskalkulator for saft og vann 🧃")
        print("1. Beregn saft og vann fra forhold og total mengde")
        print("2. Beregn vann og total mengde fra forhold og saftmengde")
        print("3. Beregn saft og total mengde fra forhold og vannmengde")
        print("4. Beregn mengder fra ønsket prosent saft og total mengde")
        print("5. Fordel et pengebeløp etter brøker")
        print("6. Avslutt")
        valg = input("Velg et alternativ (1-6) eller q for å avslutte: ").strip().lower()

        if valg == "1":
            beregn_fra_totalmengde()
        elif valg == "2":
            beregn_fra_saftmengde()
        elif valg == "3":
            beregn_fra_vannmengde()
        elif valg == "4":
            beregn_fra_prosent()
        elif valg == "5":
            fordel_beløp()
        elif valg == "6" or valg == "q":
            print("\n📤 Programmet avsluttes. Takk for at du brukte kalkulatoren! 🧃")
            return
        else:
            print("❌ Ugyldig valg. Prøv igjen.")

        nytt = input("\n🔁 Vil du gjøre en ny beregning? (j/n): ").strip().lower()
        if nytt != "j":
            print("\n📤 Programmet avsluttes. Ha en fin dag!")
            return

# ▶️ Start programmet i Jupyter
hovedprogram()

In [5]:
# 3.6 Pytagoras setning
import ipywidgets as widgets
from IPython.display import display
import math
import matplotlib.pyplot as plt

# Funksjon for å konvertere input med komma til flyttall
def convert_to_float(value):
    try:
        return float(value.replace(',', '.'))
    except ValueError:
        return None

# Funksjon for å beregne manglende side
def calculate_pythagoras(a, b, c):
    if a is None and b is not None and c is not None:
        a = math.sqrt(c**2 - b**2)
        return f"Den manglende siden a er {a:.2f}", a, b, c
    elif b is None and a is not None and c is not None:
        b = math.sqrt(c**2 - a**2)
        return f"Den manglende siden b er {b:.2f}", a, b, c
    elif c is None and a is not None and b is not None:
        c = math.sqrt(a**2 + b**2)
        return f"Den manglende siden c (hypotenus) er {c:.2f}", a, b, c
    else:
        return "Vennligst fyll inn to av tre felt for å beregne den manglende siden.", a, b, c

# Funksjon for å tegne trekanten
def draw_triangle(a, b, c):
    fig, ax = plt.subplots()
    ax.plot([0, a], [0, 0], 'b-', label=f'a = {a:.2f}')
    ax.plot([0, 0], [0, b], 'r-', label=f'b = {b:.2f}')
    ax.plot([0, a], [b, 0], 'g-', label=f'c = {c:.2f}')
    ax.set_xlim(0, max(a, b, c))
    ax.set_ylim(0, max(a, b, c))
    ax.set_aspect('equal', 'box')
    ax.legend()
    plt.show()

# Inndatafelt med bred layout
a_input = widgets.Text(description="a (katet 1):", layout=widgets.Layout(width='600px'))
b_input = widgets.Text(description="b (katet 2):", layout=widgets.Layout(width='600px'))
c_input = widgets.Text(description="c (hypotenus):", layout=widgets.Layout(width='600px'))

# Utdatafelt
output = widgets.Output()

# Kalkuleringsknapp
calc_button = widgets.Button(description="Beregn")

# Hva som skjer når knappen trykkes
def on_calc_button_clicked(b):
    with output:
        output.clear_output()
        a = convert_to_float(a_input.value)
        b = convert_to_float(b_input.value)
        c = convert_to_float(c_input.value)
        if a is None and a_input.value != '':
            print("Ugyldig verdi for a. Vennligst bruk tall med punktum eller komma som desimaltegn.")
        elif b is None and b_input.value != '':
            print("Ugyldig verdi for b. Vennligst bruk tall med punktum eller komma som desimaltegn.")
        elif c is None and c_input.value != '':
            print("Ugyldig verdi for c. Vennligst bruk tall med punktum eller komma som desimaltegn.")
        else:
            result, a, b, c = calculate_pythagoras(a, b, c)
            print(result)
            if a and b and c:
                draw_triangle(a, b, c)

calc_button.on_click(on_calc_button_clicked)

# Vis widgetene
display(a_input, b_input, c_input, calc_button, output)

Text(value='', description='a (katet 1):', layout=Layout(width='600px'))

Text(value='', description='b (katet 2):', layout=Layout(width='600px'))

Text(value='', description='c (hypotenus):', layout=Layout(width='600px'))

Button(description='Beregn', style=ButtonStyle())

Output()

In [2]:
# Delkapittel 3.6 og Delkapittel 3.7 Volum, areal av ulike figurer og objekter
import ipywidgets as widgets
from IPython.display import display
import math

def convert_to_cm(value, from_unit):
    """Konverterer en numerisk verdi fra en gitt enhet til centimeter."""
    if from_unit == "dm":
        return value * 10
    elif from_unit == "m":
        return value * 100
    elif from_unit == "mm":
        return value / 10
    # "cm" eller ukjent enhet (som "liter" for lengde) endrer ikke verdien
    return value

def convert_from_cm(value, to_unit, dimension):
    """Konverterer fra cm, cm^2 eller cm^3 til en valgt målenhet."""
    if to_unit == "liter" and dimension == 3:
        return value / 1000  # 1 liter = 1000 cm^3

    factor = 1.0
    if to_unit == "dm":
        factor = 10.0
    elif to_unit == "m":
        factor = 100.0
    elif to_unit == "mm":
        factor = 0.1

    return value / (factor ** dimension)

def calculate_area(shape, values_cm):
    """Beregner arealet av 2D-figurer, forventer verdier i cm."""
    if shape == "Trekant":
        return 0.5 * values_cm[0] * values_cm[1]
    elif shape == "Firkant":
        return values_cm[0] * values_cm[1]
    elif shape == "Kvadrat":
        return values_cm[0] ** 2
    elif shape == "Sirkel":
        return math.pi * values_cm[0] ** 2
    elif shape == "Parallellogram":
        return values_cm[0] * values_cm[1]
    elif shape == "Trapes":
        return 0.5 * (values_cm[0] + values_cm[1]) * values_cm[2]
    return 0

def calculate_volume(shape, values_cm):
    """Beregner volumet av 3D-objekter, forventer verdier i cm."""
    if shape == "Firkantet prisme":
        return values_cm[0] * values_cm[1] * values_cm[2]
    elif shape == "Trekantet prisme":
        return 0.5 * values_cm[0] * values_cm[1] * values_cm[2]
    elif shape == "Kule":
        return (4/3) * math.pi * values_cm[0] ** 3
    elif shape == "Sylinder":
        return math.pi * values_cm[0] ** 2 * values_cm[1]
    elif shape == "Kjegle":
        return (1/3) * math.pi * values_cm[0] ** 2 * values_cm[1]
    elif shape == "Pyramide":
        return (1/3) * values_cm[0] ** 2 * values_cm[1]
    return 0

def calculate_surface_area(shape, values_cm):
    """Beregner overflatearealet av 3D-objekter, forventer verdier i cm."""
    if shape == "Firkantet prisme":
        l, b, h = values_cm
        return 2 * (l * b + b * h + h * l)
    elif shape == "Trekantet prisme":
        base, height, length = values_cm
        hypotenuse = math.sqrt(base**2 + height**2)
        return (base * height) + (base + height + hypotenuse) * length
    elif shape == "Kule":
        r = values_cm[0]
        return 4 * math.pi * r ** 2
    elif shape == "Sylinder":
        r, h = values_cm
        return 2 * math.pi * r * (r + h)
    elif shape == "Kjegle":
        r, h = values_cm
        s = math.sqrt(r**2 + h**2)
        return math.pi * r * (r + s)
    elif shape == "Pyramide":
        base_side, height = values_cm
        slant_height = math.sqrt((base_side / 2)**2 + height**2)
        return base_side**2 + 2 * base_side * slant_height
    return 0

def on_calculate_button_clicked(b):
    shape = shape_dropdown.value
    unit = unit_dropdown.value
    
    visible_fields = [field for field in input_fields if field.layout.display != 'none']
    
    if not all(field.value for field in visible_fields):
        result_label.value = "Vennligst fyll inn alle nødvendige felter."
        return

    try:
        # Les streng-verdier fra synlige felter
        values_str = [field.value.replace(",", ".") for field in visible_fields]
        # Konverter til flyttall
        values_float = [float(v) for v in values_str]
        # Konverter inndata fra valgt enhet til cm
        values_cm = [convert_to_cm(v, unit) for v in values_float]

    except ValueError:
        result_label.value = "Ugyldig tall oppgitt. Bruk kun tall."
        return

    if shape in areal_figures:
        area_cm2 = calculate_area(shape, values_cm)
        result_area = convert_from_cm(area_cm2, unit, 2)
        result_label.value = f"Areal: {result_area:.2f} {unit}²"
    elif shape in volum_figures:
        volume_cm3 = calculate_volume(shape, values_cm)
        surface_area_cm2 = calculate_surface_area(shape, values_cm)
        
        # Konverter volum til valgt enhet
        vol_unit_str = "liter" if unit == "liter" else f"{unit}³"
        result_volume = convert_from_cm(volume_cm3, unit, 3)
        
        # Konverter overflateareal. Hvis 'liter' er valgt, vis overflate i dm² som en fornuftig standard.
        surf_unit = unit if unit != "liter" else "dm"
        result_surface_area = convert_from_cm(surface_area_cm2, surf_unit, 2)
        
        result_label.value = (f"Volum: {result_volume:.2f} {vol_unit_str}, "
                              f"Overflateareal: {result_surface_area:.2f} {surf_unit}²")

def on_shape_dropdown_change(change):
    shape = change['new']
    for field in input_fields:
        field.layout.display = 'none'
        field.value = ""

    labels_map = {
        "Trekant": ["Grunnlinje:", "Høyde:"], "Firkant": ["Lengde:", "Bredde:"],
        "Kvadrat": ["Side:"], "Sirkel": ["Radius:"],
        "Parallellogram": ["Grunnlinje:", "Høyde:"],
        "Trapes": ["Grunnlinje 1:", "Grunnlinje 2:", "Høyde:"],
        "Firkantet prisme": ["Lengde:", "Bredde:", "Høyde:"],
        "Trekantet prisme": ["Base (trekant):", "Høyde (trekant):", "Lengde (prisme):"],
        "Kule": ["Radius:"], "Sylinder": ["Radius:", "Høyde:"],
        "Kjegle": ["Radius:", "Høyde:"],
        "Pyramide": ["Side (kvadr. base):", "Høyde:"]
    }
    labels = labels_map.get(shape, [])
    for i, label in enumerate(labels):
        input_fields[i].description = label
        input_fields[i].layout.display = 'block'
    
    if shape in volum_figures:
        unit_dropdown.options = ["cm", "dm", "m", "mm", "liter"]
    else:
        unit_dropdown.options = ["cm", "dm", "m", "mm"]
        if unit_dropdown.value == "liter": unit_dropdown.value = "cm"

# GUI-komponenter
areal_figures = ["Trekant", "Firkant", "Kvadrat", "Sirkel", "Parallellogram", "Trapes"]
volum_figures = ["Firkantet prisme", "Trekantet prisme", "Kule", "Sylinder", "Kjegle", "Pyramide"]

shape_dropdown = widgets.Dropdown(options=areal_figures + volum_figures, description="Figur:")
unit_dropdown = widgets.Dropdown(options=["cm", "dm", "m", "mm"], description="Enhet:")
input_fields = [widgets.Text() for _ in range(3)]
calculate_button = widgets.Button(description="Beregn")
result_label = widgets.Label(value="")

calculate_button.on_click(on_calculate_button_clicked)
shape_dropdown.observe(on_shape_dropdown_change, names='value')

on_shape_dropdown_change({'new': shape_dropdown.value})

display(shape_dropdown, unit_dropdown, *input_fields, calculate_button, result_label)

Dropdown(description='Figur:', options=('Trekant', 'Firkant', 'Kvadrat', 'Sirkel', 'Parallellogram', 'Trapes',…

Dropdown(description='Enhet:', options=('cm', 'dm', 'm', 'mm'), value='cm')

Text(value='', description='Grunnlinje:', layout=Layout(display='block'))

Text(value='', description='Høyde:', layout=Layout(display='block'))

Text(value='', layout=Layout(display='none'))

Button(description='Beregn', style=ButtonStyle())

Label(value='')

In [3]:
# Delkapittel 3.8 Formlikhetskalkulator - KORRIGERT OG FORBEDRET
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
from itertools import permutations
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning, module='matplotlib')

def get_triangle_coordinates(s, v):
    """
    Calculates vertex coordinates for a triangle given its sides and angles.
    Uses a robust Side-Angle-Side (SAS) approach for plotting.
    s = [s0, s1, s2], v = [v0, v1, v2]
    """
    # Check triangle inequality if all sides are known
    if all(side > 1e-6 for side in s):
        sides_sorted = sorted(s)
        if sides_sorted[0] + sides_sorted[1] <= sides_sorted[2] + 1e-6: # Add tolerance
            return None  # Invalid triangle

    s0, s1, s2 = s
    v0, v1, v2 = (np.radians(angle) for angle in v)

    # Prioritize SAS construction for stability.
    # Case 1: Angle v0 (between s1 and s2) is known.
    if s1 > 1e-6 and s2 > 1e-6 and v[0] > 1e-6:
        A = (0, 0)
        B = (s2, 0)
        C = (s1 * np.cos(v0), s1 * np.sin(v0))
        return [A, B, C]
    # Case 2: Angle v1 (between s0 and s2) is known.
    elif s0 > 1e-6 and s2 > 1e-6 and v[1] > 1e-6:
        B = (0, 0)
        A = (s2, 0)
        C = (s0 * np.cos(v1), s0 * np.sin(v1))
        # Return in standard A, B, C order
        return [A, B, C]
    # Case 3: Angle v2 (between s0 and s1) is known.
    elif s0 > 1e-6 and s1 > 1e-6 and v[2] > 1e-6:
        C = (0, 0)
        B = (s0, 0)
        A = (s1 * np.cos(v2), s1 * np.sin(v2))
        return [A, B, C]

    return None # Not enough information for a stable plot

def beregn_geometri_synthesis(sider_in, vinkler_in, num_sides, num_figures, correspondence_maps_raw):
    sider = list(sider_in)
    vinkler = list(vinkler_in)
    logg = []

    def log_change(message):
        if message not in logg:
            logg.append(message)
        return True

    correspondence = {}
    if correspondence_maps_raw:
        for fig_target_idx, perm_tuple in correspondence_maps_raw.items():
            if perm_tuple is None: continue
            correspondence[fig_target_idx] = {target_local_idx: ref_local_idx for target_local_idx, ref_local_idx in enumerate(perm_tuple)}

    for iteration in range(10):
        made_change_in_iteration = False

        # 1. Intra-figure calculations
        for f_idx in range(num_figures):
            start_idx = f_idx * num_sides
            current_s = sider[start_idx : start_idx + num_sides]
            current_v = vinkler[start_idx : start_idx + num_sides]

            # Angle Sum
            known_angles = [v for v in current_v if v > 1e-6]
            unknown_angle_indices = [i for i, v in enumerate(current_v) if v < 1e-6]
            if len(unknown_angle_indices) == 1 and len(known_angles) == num_sides - 1:
                total_angle_sum = 180 * (num_sides - 2)
                missing_angle_val = total_angle_sum - sum(known_angles)
                if missing_angle_val > 1e-6:
                    v_idx_global = start_idx + unknown_angle_indices[0]
                    vinkler[v_idx_global] = missing_angle_val
                    log_change(f"Figur {f_idx+1}: Vinkelsum brukt for vinkel {unknown_angle_indices[0]}.")
                    made_change_in_iteration = True
                    current_v = vinkler[start_idx : start_idx + num_sides]

            if num_sides == 3:
                # --- PYTHAGORAS (CORRECTED LOGIC) ---
                for i in range(3):
                    # Convention: side s_i is opposite angle v_i.
                    # If angle v_i is 90 degrees, then side s_i is the hypotenuse.
                    if abs(current_v[i] - 90) < 1e-6:
                        hyp_idx = start_idx + i
                        kat1_idx = start_idx + (i + 1) % 3
                        kat2_idx = start_idx + (i + 2) % 3

                        hyp_val = sider[hyp_idx]
                        kat1_val = sider[kat1_idx]
                        kat2_val = sider[kat2_idx]

                        # Case 1: Find hypotenuse from the two legs (kateter)
                        if kat1_val > 1e-6 and kat2_val > 1e-6 and hyp_val < 1e-6:
                            sider[hyp_idx] = np.sqrt(kat1_val**2 + kat2_val**2)
                            log_change(f"Figur {f_idx+1}: Pythagoras fant hypotenus s{i}.")
                            made_change_in_iteration = True
                        # Case 2: Find one leg from hypotenuse and the other leg
                        elif hyp_val > 1e-6 and kat1_val > 1e-6 and kat2_val < 1e-6 and hyp_val**2 > kat1_val**2:
                            sider[kat2_idx] = np.sqrt(hyp_val**2 - kat1_val**2)
                            log_change(f"Figur {f_idx+1}: Pythagoras fant katet s{(i + 2) % 3}.")
                            made_change_in_iteration = True
                        elif hyp_val > 1e-6 and kat2_val > 1e-6 and kat1_val < 1e-6 and hyp_val**2 > kat2_val**2:
                            sider[kat1_idx] = np.sqrt(hyp_val**2 - kat2_val**2)
                            log_change(f"Figur {f_idx+1}: Pythagoras fant katet s{(i + 1) % 3}.")
                            made_change_in_iteration = True
                        
                        if made_change_in_iteration:
                           current_s = sider[start_idx : start_idx + num_sides]
                
                # Sine Rule (s_i / sin(v_i) = k)
                known_pairs = [(current_s[i], current_v[i]) for i in range(3) if current_s[i] > 1e-6 and current_v[i] > 1e-6]
                if known_pairs:
                    ratio = known_pairs[0][0] / np.sin(np.radians(known_pairs[0][1]))
                    for i in range(3):
                        # Find side from angle
                        if sider[start_idx+i] < 1e-6 and vinkler[start_idx+i] > 1e-6:
                            sider[start_idx + i] = ratio * np.sin(np.radians(vinkler[start_idx+i]))
                            log_change(f"Figur {f_idx+1}: Sinussetning fant side s{i}.")
                            made_change_in_iteration = True
                        # Find angle from side
                        elif vinkler[start_idx+i] < 1e-6 and sider[start_idx+i] > 1e-6:
                            sin_val = sider[start_idx+i] / ratio
                            if -1 <= sin_val <= 1:
                                # Note: Sine rule can be ambiguous (two solutions). Assume acute angle.
                                vinkler[start_idx + i] = np.degrees(np.arcsin(sin_val))
                                log_change(f"Figur {f_idx+1}: Sinussetning fant vinkel {i}.")
                                made_change_in_iteration = True

                # Cosine Rule (find side)
                for i in range(3):
                    angle_A = current_v[i]
                    side_b = current_s[(i + 1) % 3]
                    side_c = current_s[(i + 2) % 3]
                    if side_b > 1e-6 and side_c > 1e-6 and angle_A > 1e-6 and current_s[i] < 1e-6:
                        val_sq = side_b**2 + side_c**2 - 2 * side_b * side_c * np.cos(np.radians(angle_A))
                        if val_sq > 0:
                            sider[start_idx + i] = np.sqrt(val_sq)
                            log_change(f"Figur {f_idx+1}: Cosinussetning fant side s{i}.")
                            made_change_in_iteration = True

                # Cosine Rule (find angle)
                if sum(1 for s in current_s if s > 1e-6) == 3 and any(v < 1e-6 for v in current_v):
                    for i in range(3):
                        if current_v[i] < 1e-6:
                            side_a = current_s[i]
                            side_b = current_s[(i + 1) % 3]
                            side_c = current_s[(i + 2) % 3]
                            cos_val_num = side_b**2 + side_c**2 - side_a**2
                            cos_val_den = 2 * side_b * side_c
                            if abs(cos_val_den) > 1e-9:
                                cos_val = np.clip(cos_val_num / cos_val_den, -1.0, 1.0)
                                vinkler[start_idx + i] = np.degrees(np.arccos(cos_val))
                                log_change(f"Figur {f_idx+1}: Cosinussetning fant vinkel {i}.")
                                made_change_in_iteration = True

        # 2. Inter-figure calculations (Similarity)
        if num_figures > 1 and correspondence:
            # Sync angles
            for f_target_idx in range(1, num_figures):
                if f_target_idx in correspondence:
                    for target_local_idx, ref_local_idx in correspondence[f_target_idx].items():
                        ref_g_idx = ref_local_idx
                        target_g_idx = f_target_idx * num_sides + target_local_idx
                        if vinkler[ref_g_idx] > 1e-6 and vinkler[target_g_idx] < 1e-6:
                            vinkler[target_g_idx] = vinkler[ref_g_idx]
                            log_change(f"Formlikhet: Vinkel {target_local_idx} (Fig {f_target_idx+1}) satt lik vinkel {ref_local_idx} (Fig 1).")
                            made_change_in_iteration = True
                        elif vinkler[target_g_idx] > 1e-6 and vinkler[ref_g_idx] < 1e-6:
                            vinkler[ref_g_idx] = vinkler[target_g_idx]
                            log_change(f"Formlikhet: Vinkel {ref_local_idx} (Fig 1) satt lik vinkel {target_local_idx} (Fig {f_target_idx+1}).")
                            made_change_in_iteration = True
            
            # Calculate and apply scale factor
            for f_target_idx in range(1, num_figures):
                if f_target_idx not in correspondence: continue
                scale_factor = None
                # Find scale factor from a known pair of corresponding sides
                for tl_idx, rl_idx in correspondence[f_target_idx].items():
                    ref_g_idx = rl_idx
                    target_g_idx = f_target_idx * num_sides + tl_idx
                    if sider[ref_g_idx] > 1e-6 and sider[target_g_idx] > 1e-6:
                        scale_factor = sider[target_g_idx] / sider[ref_g_idx]
                        log_change(f"Formlikhet: Målestokk Fig {f_target_idx+1}/Fig 1 = {scale_factor:.3f} (fra s{tl_idx}/s{rl_idx}).")
                        break
                
                # Apply scale factor to find unknown sides
                if scale_factor is not None:
                    for tl_idx, rl_idx in correspondence[f_target_idx].items():
                        ref_g_idx = rl_idx
                        target_g_idx = f_target_idx * num_sides + tl_idx
                        if sider[ref_g_idx] > 1e-6 and sider[target_g_idx] < 1e-6:
                            sider[target_g_idx] = sider[ref_g_idx] * scale_factor
                            log_change(f"Formlikhet: Side s{tl_idx} (Fig {f_target_idx+1}) beregnet.")
                            made_change_in_iteration = True
                        elif sider[target_g_idx] > 1e-6 and sider[ref_g_idx] < 1e-6:
                            sider[ref_g_idx] = sider[target_g_idx] / scale_factor
                            log_change(f"Formlikhet: Side s{rl_idx} (Fig 1) beregnet.")
                            made_change_in_iteration = True

        if not made_change_in_iteration:
            break
    
    return sider, vinkler, logg

class FormlikhetApp(widgets.VBox):
    def __init__(self):
        super().__init__()
        # Simplified state management
        self.side_inputs, self.angle_inputs, self.corr_widgets = [], [], []
        self._ui_update_active = False # Flag to prevent observer loops
        self._create_ui()

    def _create_ui(self):
        style = {'description_width': 'initial'}
        self.num_sides_slider = widgets.IntSlider(value=3, min=3, max=4, description='Antall kanter:', style=style)
        self.num_figures_slider = widgets.IntSlider(value=2, min=1, max=2, description='Antall figurer:', style=style)
        self.solve_button = widgets.Button(description="Beregn", icon="calculator", button_style='info')
        reset_button = widgets.Button(description="Nullstill Alt", icon="refresh", button_style='danger')
        
        self.form_container = widgets.VBox()
        self.output_area = widgets.Output(layout={'border': '1px solid black', 'padding': '5px'})
        self.log_output = widgets.Output(layout={'border': '1px solid #ccc', 'padding': '5px', 'margin_top': '10px'})

        self.children = [widgets.HTML(value="<h2>Formlikhetskalkulator</h2>"),
                         widgets.HBox([self.num_sides_slider, self.num_figures_slider]),
                         widgets.HBox([self.solve_button, reset_button]),
                         self.form_container, self.output_area, self.log_output]
        
        self.num_sides_slider.observe(self._rebuild_form, 'value')
        self.num_figures_slider.observe(self._rebuild_form, 'value')
        self.solve_button.on_click(self._solve_geometry)
        reset_button.on_click(self._reset_all)
        self._rebuild_form()

    def _rebuild_form(self, change=None):
        self._ui_update_active = True
        self.side_inputs.clear(); self.angle_inputs.clear(); self.corr_widgets.clear()
        num_sides, num_figures = self.num_sides_slider.value, self.num_figures_slider.value
        
        v_names_global = [chr(65 + j) for j in range(26)]
        figur_bokser = []
        for i in range(num_figures):
            v_names = v_names_global[i * num_sides : (i + 1) * num_sides]
            s_widgets = [widgets.FloatText(description=f"Side s{k} (motsatt {v_names[k]}):", layout={'width': '250px'}) for k in range(num_sides)]
            a_widgets = [widgets.FloatText(description=f"Vinkel {v_names[k]}:", layout={'width': '200px'}) for k in range(num_sides)]
            self.side_inputs.extend(s_widgets)
            self.angle_inputs.extend(a_widgets)

            corr_box_content = []
            if i > 0:
                ref_v_names = v_names_global[0 : num_sides]
                perms = list(permutations(range(num_sides)))
                options = {"Velg korrespondanse...": None}
                for p in perms:
                    opt_text = ", ".join([f"{v_names[target_idx]}↔{ref_v_names[ref_idx]}" for target_idx, ref_idx in enumerate(p)])
                    options[opt_text] = p
                dropdown = widgets.Dropdown(options=options, description=f"Fig {i+1} ↔ Fig 1:", layout={'width': 'auto'})
                self.corr_widgets.append(dropdown)
                corr_box_content.append(dropdown)
            
            box = widgets.VBox([widgets.HTML(f"<b>Figur {i + 1} ({', '.join(v_names)})</b>"),
                                widgets.HBox([widgets.VBox(s_widgets), widgets.VBox(a_widgets)]),
                                widgets.VBox(corr_box_content), widgets.HTML("<hr>")])
            figur_bokser.append(box)

        self.form_container.children = figur_bokser
        self._ui_update_active = False
        self._reset_all()

    def _reset_all(self, btn=None):
        self._ui_update_active = True
        for widget in self.side_inputs + self.angle_inputs:
            widget.value = 0.0
        for widget in self.corr_widgets:
            widget.value = None
        self._ui_update_active = False
        self._clear_outputs_and_plot()

    def _clear_outputs_and_plot(self):
        with self.output_area:
            clear_output()
            plt.figure(figsize=(5 * self.num_figures_slider.value, 4))
            plt.text(0.5, 0.5, "Input verdier og trykk 'Beregn'", ha='center', va='center')
            plt.axis('off'); plt.show()
        with self.log_output:
            clear_output(); print("Logg vil vises her...")
    
    def _get_correspondence_map(self):
        return {i+1: self.corr_widgets[i].value for i, w in enumerate(self.corr_widgets) if self.corr_widgets[i].value}

    def _solve_geometry(self, btn=None):
        if self._ui_update_active: return
        num_sides, num_figures = self.num_sides_slider.value, self.num_figures_slider.value
        
        s_init = [w.value if w.value > 0 else 0.0 for w in self.side_inputs]
        v_init = [w.value if w.value > 0 else 0.0 for w in self.angle_inputs]

        s_calc, v_calc, logg = beregn_geometri_synthesis(s_init, v_init, num_sides, num_figures, self._get_correspondence_map())
        
        self._ui_update_active = True
        for i, val in enumerate(s_calc): self.side_inputs[i].value = round(val, 3) if val > 1e-6 else 0.0
        for i, val in enumerate(v_calc): self.angle_inputs[i].value = round(val, 2) if val > 1e-6 else 0.0
        self._ui_update_active = False

        self._plot_figures(s_calc, v_calc)
        with self.log_output:
            clear_output(wait=True)
            print("Beregningsteg:\n" + "\n".join(f"- {item}" for item in logg) if logg else "Ingen nye beregninger gjort.")

    def _plot_figures(self, sider_data, vinkler_data):
        with self.output_area:
            clear_output(wait=True)
            num_sides, num_figures = self.num_sides_slider.value, self.num_figures_slider.value
            fig, axs = plt.subplots(1, num_figures, figsize=(5 * num_figures, 4.5), squeeze=False)
            v_names_global = [chr(65 + j) for j in range(26)]

            for f_idx in range(num_figures):
                ax = axs[0, f_idx]
                ax.set_aspect('equal', 'box'); ax.axis('off')
                v_names = v_names_global[f_idx*num_sides : (f_idx+1)*num_sides]
                ax.set_title(f"Figur {f_idx + 1} ({', '.join(v_names)})")

                s_fig = sider_data[f_idx*num_sides : (f_idx+1)*num_sides]
                v_fig = vinkler_data[f_idx*num_sides : (f_idx+1)*num_sides]

                coords = None
                if num_sides == 3:
                    coords = get_triangle_coordinates(s_fig, v_fig)
                elif num_sides == 4:
                    ax.text(0.5, 0.5, "Plotting av firkanter\ner ikke støttet.", ha='center', va='center', transform=ax.transAxes)

                if coords:
                    polygon = plt.Polygon(coords, closed=True, fill=True, edgecolor='blue', facecolor='lightblue', alpha=0.7)
                    ax.add_patch(polygon)
                    
                    # Labels for vertices (A, B, C) and angles
                    for i, (x, y) in enumerate(coords):
                        angle_label = f"{v_fig[i]:.1f}°" if v_fig[i] > 1e-3 else ""
                        ax.text(x, y, f"  {v_names[i]}\n  ({angle_label})", va='center', ha='left', fontsize=9)

                    # Labels for sides
                    for i in range(num_sides):
                        p1, p2 = np.array(coords[i]), np.array(coords[(i + 1) % num_sides])
                        mid_point = (p1 + p2) / 2
                        side_len = s_fig[(i + 2) % num_sides] # Side opposite vertex (i+2) is between vertex i and i+1
                        if side_len > 1e-3:
                            ax.text(mid_point[0], mid_point[1], f"{side_len:.2f}", color='darkred', ha='center', va='center',
                                    bbox=dict(facecolor='white', alpha=0.5, pad=0.1, edgecolor='none'))
                    
                    all_x, all_y = [c[0] for c in coords], [c[1] for c in coords]
                    padding = (max(np.ptp(all_x), np.ptp(all_y))) * 0.15 + 0.5
                    ax.set_xlim(min(all_x) - padding, max(all_x) + padding)
                    ax.set_ylim(min(all_y) - padding, max(all_y) + padding)
                elif num_sides != 4:
                    ax.text(0.5, 0.5, "Ikke nok data til å tegne\neller ugyldig trekant.", ha='center', va='center', transform=ax.transAxes)
            
            plt.tight_layout(pad=1.0); plt.show()

app = FormlikhetApp()
display(app)

FormlikhetApp(children=(HTML(value='<h2>Formlikhetskalkulator</h2>'), HBox(children=(IntSlider(value=3, descri…

In [2]:
# Del 1: Forbedret og utvidet kode
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
from itertools import permutations
import warnings

# --- Globale innstillinger ---
# Ignorerer vanlige advarsler fra numpy og matplotlib under kjøring for en renere opplevelse.
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning, module='matplotlib')


# ##############################################################################
# Del 1: Objektorientert Beregningsmotor
# Denne delen definerer de logiske klassene for geometriske figurer.
# ##############################################################################

class Shape:
    """En baseklasse for alle geometriske figurer."""
    def __init__(self, name, sides, angles):
        self.name = name
        self.num_sides = len(sides)
        # Sørger for at input er lister av flyttall
        self.s = [float(s) if s is not None else 0.0 for s in sides]
        self.v = [float(v) if v is not None else 0.0 for v in angles]
        self.log = []
        self.area = 0.0
        self.v_names = [chr(65 + i) for i in range(self.num_sides)] # A, B, C...

    def log_change(self, message):
        """Legger til en melding i loggen hvis den ikke allerede eksisterer."""
        if message not in self.log:
            self.log.append(message)

    def solve(self):
        """Kjører alle løsningsmetoder. Skal overskrives av subklasser."""
        raise NotImplementedError("Subklasser må implementere solve-metoden.")


class Triangle(Shape):
    """Representerer en trekant med tilhørende beregningslogikk."""
    def __init__(self, name, sides, angles):
        if len(sides) != 3 or len(angles) != 3:
            raise ValueError("Trekanten må ha 3 sider og 3 vinkler.")
        super().__init__(name, sides, angles)

    def solve(self):
        """Iterativ løser for trekanten for å fange opp nye funn."""
        for _ in range(5):  # Itererer flere ganger for å sikre at alle mulige løsninger er funnet
            changed = False
            if self._solve_angle_sum(): changed = True
            if self._solve_pythagoras(): changed = True
            if self._solve_cosine_rule_side(): changed = True
            if self._solve_cosine_rule_angle(): changed = True
            if self._solve_sine_rule(): changed = True
            if not changed:
                break  # Avslutt hvis ingen endringer ble gjort i en iterasjon
        self._calculate_area()

    def _solve_angle_sum(self):
        """Finner en ukjent vinkel hvis to er kjent (sum = 180°)."""
        known_angles = [a for a in self.v if a > 1e-6]
        if len(known_angles) == 2:
            unknown_indices = [i for i, a in enumerate(self.v) if a < 1e-6]
            if unknown_indices:
                missing_angle = 180 - sum(known_angles)
                self.v[unknown_indices[0]] = missing_angle
                self.log_change(f"Vinkelsum brukt for vinkel {self.v_names[unknown_indices[0]]}.")
                return True
        return False

    def _solve_pythagoras(self):
        """Bruker Pythagoras' læresetning i rettvinklede trekanter."""
        # s[i] er motstående til v[i]. For rett vinkel i C (v[2]), er s[2] hypotenus.
        for i in range(3):
            if abs(self.v[i] - 90) < 1e-6:  # Rett vinkel ved v[i]
                hyp_idx, kat1_idx, kat2_idx = i, (i + 1) % 3, (i + 2) % 3
                hyp, k1, k2 = self.s[hyp_idx], self.s[kat1_idx], self.s[kat2_idx]

                # Finner hypotenus
                if k1 > 1e-6 and k2 > 1e-6 and hyp < 1e-6:
                    self.s[hyp_idx] = np.sqrt(k1**2 + k2**2)
                    self.log_change(f"Pythagoras fant hypotenus s{hyp_idx}.")
                    return True
                # Finner et katet
                if hyp > 1e-6 and k1 > 1e-6 and k2 < 1e-6 and hyp**2 > k1**2:
                    self.s[kat2_idx] = np.sqrt(hyp**2 - k1**2)
                    self.log_change(f"Pythagoras fant katet s{kat2_idx}.")
                    return True
                if hyp > 1e-6 and k2 > 1e-6 and k1 < 1e-6 and hyp**2 > k2**2:
                    self.s[kat1_idx] = np.sqrt(hyp**2 - k2**2)
                    self.log_change(f"Pythagoras fant katet s{kat1_idx}.")
                    return True
        return False

    def _solve_cosine_rule_side(self):
        """Bruker cosinussetningen for å finne en ukjent side."""
        for i in range(3):
            # Finn side s[i] med vinkel v[i] og de to andre sidene
            v_A, s_b, s_c = self.v[i], self.s[(i+1)%3], self.s[(i+2)%3]
            if v_A > 1e-6 and s_b > 1e-6 and s_c > 1e-6 and self.s[i] < 1e-6:
                val_sq = s_b**2 + s_c**2 - 2 * s_b * s_c * np.cos(np.radians(v_A))
                if val_sq > 0:
                    self.s[i] = np.sqrt(val_sq)
                    self.log_change(f"Cosinussetningen fant side s{i}.")
                    return True
        return False

    def _solve_cosine_rule_angle(self):
        """Bruker cosinussetningen for å finne en ukjent vinkel."""
        if all(s > 1e-6 for s in self.s):
            for i in range(3):
                if self.v[i] < 1e-6:
                    s_a, s_b, s_c = self.s[i], self.s[(i+1)%3], self.s[(i+2)%3]
                    cos_val = (s_b**2 + s_c**2 - s_a**2) / (2 * s_b * s_c)
                    if -1 <= cos_val <= 1:
                        self.v[i] = np.degrees(np.arccos(cos_val))
                        self.log_change(f"Cosinussetningen fant vinkel {self.v_names[i]}.")
                        return True
        return False

    def _solve_sine_rule(self):
        """Bruker sinussetningen for å finne ukjente sider eller vinkler."""
        known_pairs = [(self.s[i] / np.sin(np.radians(self.v[i])), i) for i in range(3) if self.s[i] > 1e-6 and self.v[i] > 1e-6]
        if not known_pairs: return False

        ratio, _ = known_pairs[0]
        made_change = False
        for i in range(3):
            # Finner side
            if self.s[i] < 1e-6 and self.v[i] > 1e-6:
                self.s[i] = ratio * np.sin(np.radians(self.v[i]))
                self.log_change(f"Sinussetningen fant side s{i}.")
                made_change = True
            # Finner vinkel
            elif self.v[i] < 1e-6 and self.s[i] > 1e-6:
                sin_val = self.s[i] / ratio
                # NB: Håndterer ikke det tvetydige tilfellet (hvor vinkelen kan være > 90) for enkelhets skyld.
                if -1 <= sin_val <= 1:
                    self.v[i] = np.degrees(np.arcsin(sin_val))
                    self.log_change(f"Sinussetningen fant vinkel {self.v_names[i]}.")
                    made_change = True
        return made_change

    def _calculate_area(self):
        """Beregner arealet av trekanten med Herons formel eller sinus-formelen."""
        if all(s > 1e-6 for s in self.s):
            s_p = sum(self.s) / 2
            val_inside_sqrt = s_p * (s_p - self.s[0]) * (s_p - self.s[1]) * (s_p - self.s[2])
            if val_inside_sqrt > 1e-9:
                self.area = np.sqrt(val_inside_sqrt)
                self.log_change(f"Areal beregnet med Herons formel: {self.area:.2f}")
                return

        for i in range(3):
            v_C, s_a, s_b = self.v[i], self.s[(i+1)%3], self.s[(i+2)%3]
            if v_C > 1e-6 and s_a > 1e-6 and s_b > 1e-6:
                self.area = 0.5 * s_a * s_b * np.sin(np.radians(v_C))
                self.log_change(f"Areal beregnet med sin(vinkel): {self.area:.2f}")
                return

class Quadrilateral(Shape):
    """Representerer en firkant."""
    def __init__(self, name, sides, angles):
        if len(sides) != 4 or len(angles) != 4:
            raise ValueError("Firkanten må ha 4 sider og 4 vinkler.")
        super().__init__(name, sides, angles)

    def solve(self):
        """Løser for enkle egenskaper ved firkanten."""
        self._solve_angle_sum()
        # En generell firkant er ikke "stiv". Areal/sider kan ikke finnes uten mer info (f.eks. en diagonal).
        # Legger til en loggmelding for å informere brukeren.
        if sum(1 for s in self.s if s > 1e-6) < 4 or sum(1 for v in self.v if v > 1e-6) < 4:
             self.log_change("Info: En firkant er ikke rigid. Alle sider og vinkler kan ikke bestemmes uten mer informasjon, som en diagonal.")

    def _solve_angle_sum(self):
        """Finner en ukjent vinkel hvis tre er kjent (sum = 360°)."""
        known_angles = [a for a in self.v if a > 1e-6]
        if len(known_angles) == 3:
            unknown_idx = -1
            for i, a in enumerate(self.v):
                if a < 1e-6:
                    unknown_idx = i
                    break
            if unknown_idx != -1:
                missing_angle = 360 - sum(known_angles)
                self.v[unknown_idx] = missing_angle
                self.log_change(f"Vinkelsum (360°) brukt for vinkel {self.v_names[unknown_idx]}.")
                return True
        return False

def solve_geometry_system(figures, correspondence_map):
    """
    Hovedfunksjon som håndterer beregninger mellom formlike figurer.
    `figures`: en liste med Shape-objekter (Triangle/Quadrilateral).
    `correspondence_map`: definerer koblingen mellom figurene.
    """
    if not figures:
        return [], "Ingen figurer å løse."

    # 1. Løs hver figur internt først
    for fig in figures:
        fig.solve()

    # 2. Bruk formlikhet mellom figurene (iterativt)
    for _ in range(5):
        changed_this_iteration = False
        ref_fig = figures[0]

        for target_idx, corr_tuple in correspondence_map.items():
            if target_idx >= len(figures) or corr_tuple is None:
                continue
            target_fig = figures[target_idx]

            # a. Synkroniser vinkler
            for target_local, ref_local in enumerate(corr_tuple):
                if ref_fig.v[ref_local] > 1e-6 and target_fig.v[target_local] < 1e-6:
                    target_fig.v[target_local] = ref_fig.v[ref_local]
                    target_fig.log_change(f"Formlikhet: Vinkel {target_fig.v_names[target_local]} satt fra {ref_fig.name}.")
                    changed_this_iteration = True
                elif target_fig.v[target_local] > 1e-6 and ref_fig.v[ref_local] < 1e-6:
                    ref_fig.v[ref_local] = target_fig.v[target_local]
                    ref_fig.log_change(f"Formlikhet: Vinkel {ref_fig.v_names[ref_local]} satt fra {target_fig.name}.")
                    changed_this_iteration = True

            # b. Finn og anvend målestokk
            scale_factor = None
            for target_local, ref_local in enumerate(corr_tuple):
                if ref_fig.s[ref_local] > 1e-6 and target_fig.s[target_local] > 1e-6:
                    scale_factor = target_fig.s[target_local] / ref_fig.s[ref_local]
                    break
            
            if scale_factor:
                # Logg målestokken kun én gang
                log_msg = f"Målestokk {target_fig.name}/{ref_fig.name} er {scale_factor:.3f}."
                if log_msg not in target_fig.log and log_msg not in ref_fig.log:
                    target_fig.log_change(log_msg)

                for target_local, ref_local in enumerate(corr_tuple):
                    # Fra referanse til mål
                    if ref_fig.s[ref_local] > 1e-6 and target_fig.s[target_local] < 1e-6:
                        target_fig.s[target_local] = ref_fig.s[ref_local] * scale_factor
                        target_fig.log_change(f"Formlikhet: Side s{target_local} beregnet.")
                        changed_this_iteration = True
                    # Fra mål til referanse
                    elif target_fig.s[target_local] > 1e-6 and ref_fig.s[ref_local] < 1e-6:
                        ref_fig.s[ref_local] = target_fig.s[target_local] / scale_factor
                        ref_fig.log_change(f"Formlikhet: Side s{ref_local} beregnet.")
                        changed_this_iteration = True
        
        # 3. Kjør intern løser på nytt for å bruke ny info fra formlikhet
        if changed_this_iteration:
            for fig in figures:
                fig.solve()
        else:
            break # Avslutt hvis ingen endring skjedde i hele systemet

    # Samle all logg til slutt
    full_log = []
    for i, fig in enumerate(figures):
        full_log.append(f"--- Logg for {fig.name} ---")
        full_log.extend(fig.log)
    
    # Legg til en oppsummerende melding hvis ingen sider er kjent
    sider_kjent = any(s > 1e-6 for fig in figures for s in fig.s)
    vinkler_synkronisert = any("Formlikhet: Vinkel" in msg for msg in full_log)
    if not sider_kjent and vinkler_synkronisert:
        full_log.append("--- Oppsummering ---")
        full_log.append("Figurene er formlike basert på vinkler, men sidelengder kan ikke beregnes uten minst én kjent sidelengde for å etablere en målestokk.")
        
    return figures, full_log


# ##############################################################################
# Del 2: Plottehjelper og Interaktiv Applikasjon
# Denne delen bygger det interaktive grensesnittet og visualiserer resultatene.
# ##############################################################################

def get_triangle_coordinates(s_list, v_list):
    """Beregner hjørnekoordinater for en trekant for plotting."""
    s0, s1, s2 = s_list
    v0, v1, v2 = v_list
    
    # Sjekk om en trekant kan dannes (trekantulikheten)
    if all(s > 1e-6 for s in [s0, s1, s2]):
        sides = sorted([s0, s1, s2])
        if sides[0] + sides[1] <= sides[2] + 1e-6: # Legger til toleranse
            return None

    # Plasserer hjørne A (v0) i origo (0,0)
    A = (0, 0)
    
    # Prøver å plassere B langs x-aksen basert på side c (s2)
    if s2 > 1e-6:
        B = (s2, 0)
        # Finn C basert på side b (s1) og vinkel A (v0)
        if s1 > 1e-6 and v0 > 1e-6:
            C = (s1 * np.cos(np.radians(v0)), s1 * np.sin(np.radians(v0)))
            return [A, B, C]
        # Finn C basert på side a (s0) og vinkel B (v1)
        elif s0 > 1e-6 and v1 > 1e-6:
            # Finner C relativt til B
            Cx_rel_B = s0 * np.cos(np.radians(180 - v1))
            Cy_rel_B = s0 * np.sin(np.radians(180 - v1))
            C = (B[0] + Cx_rel_B, B[1] + Cy_rel_B)
            return [A, B, C]

    # Fallback: Bruk to sider og vinkelen mellom dem
    # Vinkel v0 er mellom s1 og s2
    if s1 > 1e-6 and s2 > 1e-6 and v0 > 1e-6:
        B = (s2, 0)
        C = (s1 * np.cos(np.radians(v0)), s1 * np.sin(np.radians(v0)))
        return [A, B, C]
    # Vinkel v1 er mellom s0 og s2
    elif s0 > 1e-6 and s2 > 1e-6 and v1 > 1e-6:
        A_temp = (s2, 0)
        C_temp = (s0 * np.cos(np.radians(v1)), s0 * np.sin(np.radians(v1)))
        return [A_temp, (0, 0), C_temp] # Returner i rekkefølge A, B, C
    # Vinkel v2 er mellom s0 og s1
    elif s0 > 1e-6 and s1 > 1e-6 and v2 > 1e-6:
        B_temp = (s0, 0)
        A_temp = (s1 * np.cos(np.radians(v2)), s1 * np.sin(np.radians(v2)))
        return [A_temp, B_temp, (0,0)] # Returner i rekkefølge A, B, C

    return None  # Ikke nok informasjon til å tegne


class FormlikhetApp(widgets.VBox):
    """Hovedklassen for den interaktive applikasjonen."""
    def __init__(self):
        super().__init__()
        self._ui_update_active = False # Flagg for å unngå uendelige løkker ved UI-oppdatering
        self._create_ui()
        self._rebuild_form()

    def _create_ui(self):
        """Bygger alle UI-komponentene."""
        header = widgets.HTML(value="<h2>Formlikhetskalkulator</h2>")
        self.num_sides_slider = widgets.IntSlider(value=3, min=3, max=4, description='Antall kanter:')
        self.num_figures_slider = widgets.IntSlider(value=2, min=1, max=2, description='Antall figurer:')
        self.solve_button = widgets.Button(description="Beregn", icon="calculator", button_style='info')
        reset_button = widgets.Button(description="Nullstill", icon="refresh", button_style='danger')
        
        self.form_container = widgets.VBox()
        self.output_area = widgets.Output(layout={'border': '1px solid black', 'padding': '5px', 'margin_top':'10px'})
        self.log_output = widgets.HTML(value="", layout={'border': '1px solid #ccc', 'padding': '10px', 'margin_top': '10px'})

        self.children = [
            header,
            widgets.HBox([self.num_sides_slider, self.num_figures_slider]),
            widgets.HBox([self.solve_button, reset_button]),
            self.form_container,
            self.output_area,
            self.log_output
        ]
        
        self.num_sides_slider.observe(self._rebuild_form, 'value')
        self.num_figures_slider.observe(self._rebuild_form, 'value')
        self.solve_button.on_click(self._solve_geometry)
        reset_button.on_click(self._reset_all)

    def _rebuild_form(self, change=None):
        """Gjenoppbygger input-feltene når antall sider/figurer endres."""
        self._ui_update_active = True
        num_sides, num_figures = self.num_sides_slider.value, self.num_figures_slider.value

        self.side_inputs, self.angle_inputs, self.corr_widgets = [], [], []
        self.correspondence_maps = {}
        
        figur_bokser = []
        v_names_global = [chr(65 + j) for j in range(26)]
        
        for i in range(num_figures):
            start_v_name_idx = i * num_sides
            current_fig_v_names = v_names_global[start_v_name_idx : start_v_name_idx + num_sides]
            
            s_widgets = [widgets.FloatText(description=f"Side s{k} (motsatt {current_fig_v_names[k]}):", value=0.0, layout={'width': '250px'}) for k in range(num_sides)]
            a_widgets = [widgets.FloatText(description=f"Vinkel {current_fig_v_names[k]}:", value=0.0, layout={'width': '200px'}) for k in range(num_sides)]
            
            self.side_inputs.extend(s_widgets)
            self.angle_inputs.extend(a_widgets)

            corr_box = widgets.Box()
            if i > 0:
                ref_fig_v_names = v_names_global[0:num_sides]
                perms = list(permutations(range(num_sides)))
                options = {"Velg korrespondanse...": None}
                for p_indices in perms:
                    option_text = ", ".join([f"{current_fig_v_names[t]}↔{ref_fig_v_names[r]}" for t, r in enumerate(p_indices)])
                    options[option_text] = p_indices

                dropdown = widgets.Dropdown(options=options, description=f"Fig {i+1} ↔ Fig 1:", layout={'width': 'auto', 'min_width':'350px'})
                dropdown.observe(self._handle_corr_change(i), names='value')
                self.corr_widgets.append(dropdown)
                corr_box = widgets.VBox([dropdown])

            box = widgets.VBox([
                widgets.HTML(value=f"<b>Figur {i + 1} ({', '.join(current_fig_v_names)})</b>"),
                widgets.HBox([widgets.VBox(s_widgets), widgets.VBox(a_widgets)]),
                corr_box,
                widgets.HTML("<hr>")
            ])
            figur_bokser.append(box)

        self.form_container.children = figur_bokser
        self._ui_update_active = False
        self._clear_outputs_and_plot()

    def _handle_corr_change(self, fig_idx_target):
        """Håndterer endring i korrespondanse-dropdown."""
        def _f(change):
            if self._ui_update_active: return
            self.correspondence_maps[fig_idx_target] = change.new
            self._solve_geometry() # Løs på nytt når korrespondansen endres
        return _f

    def _reset_all(self, btn=None):
        """Nullstiller alle verdier og bygger UI på nytt."""
        self.num_sides_slider.value = 3
        self.num_figures_slider.value = 2
        self._rebuild_form()

    def _clear_outputs_and_plot(self):
        """Tømmer output-området og logger."""
        with self.output_area:
            clear_output()
            plt.figure(figsize=(10, 4.5))
            plt.text(0.5, 0.5, "Input verdier og trykk 'Beregn'", ha='center', va='center', fontsize=12)
            plt.axis('off')
            plt.show()
        self.log_output.value = "<i>Logg vil vises her...</i>"
        
    def _solve_geometry(self, btn=None):
        """Orkestrerer hele beregningsprosessen."""
        num_sides, num_figures = self.num_sides_slider.value, self.num_figures_slider.value
        
        figures_to_solve = []
        for i in range(num_figures):
            start = i * num_sides
            end = start + num_sides
            s_vals = [s.value for s in self.side_inputs[start:end]]
            v_vals = [v.value for v in self.angle_inputs[start:end]]
            name = f"Figur {i+1}"
            
            if num_sides == 3:
                figures_to_solve.append(Triangle(name, s_vals, v_vals))
            elif num_sides == 4:
                figures_to_solve.append(Quadrilateral(name, s_vals, v_vals))
        
        # Kjør den objektorienterte løseren
        solved_figures, log_messages = solve_geometry_system(figures_to_solve, self.correspondence_maps)
        
        # Oppdater UI med resultatene
        self._update_ui_from_calculations(solved_figures)
        self._plot_figures(solved_figures)
        
        # Vis logg
        self.log_output.value = "<br>".join(log_messages).replace("---", "<hr><b>").replace("---", "</b>")

    def _update_ui_from_calculations(self, solved_figures):
        """Oppdaterer input-feltene med beregnede verdier."""
        self._ui_update_active = True
        s_flat = [s for fig in solved_figures for s in fig.s]
        v_flat = [v for fig in solved_figures for v in fig.v]
        
        for i, val in enumerate(s_flat):
            self.side_inputs[i].value = round(val, 3) if val > 1e-6 else 0.0
        for i, val in enumerate(v_flat):
            self.angle_inputs[i].value = round(val, 2) if val > 1e-6 else 0.0
        self._ui_update_active = False

    def _plot_figures(self, solved_figures):
        """Tegner figurene basert på de løste verdiene."""
        with self.output_area:
            clear_output(wait=True)
            num_figures = len(solved_figures)
            fig, axs = plt.subplots(1, num_figures, figsize=(5 * num_figures, 4.5), squeeze=False)
            
            for f_idx, current_fig in enumerate(solved_figures):
                ax = axs[0, f_idx]
                ax.clear()
                ax.set_title(current_fig.name)
                ax.set_aspect('equal', adjustable='box')
                ax.axis('off')

                coords = None
                if isinstance(current_fig, Triangle):
                    coords = get_triangle_coordinates(current_fig.s, current_fig.v)
                
                # Plotting for firkanter er ikke implementert, da de ikke er rigide.
                if coords:
                    polygon = plt.Polygon(coords, closed=True, fill=True, edgecolor='blue', facecolor='lightblue', alpha=0.7)
                    ax.add_patch(polygon)
                    
                    for i in range(current_fig.num_sides):
                        # Tekst for hjørne og vinkel
                        angle_label = f"{current_fig.v[i]:.1f}°" if current_fig.v[i] > 1e-3 else "?"
                        ax.text(coords[i][0], coords[i][1], f"  {current_fig.v_names[i]} ({angle_label})",
                                va='bottom', ha='left', fontsize=9)

                        # Tekst for sidelengde (plassert midt på og utenfor kanten)
                        p1, p2 = np.array(coords[i]), np.array(coords[(i + 1) % current_fig.num_sides])
                        side_len_idx = (i + 2) % 3 # Siden motsatt vinkel i+2
                        side_len_val = current_fig.s[side_len_idx]
                        if side_len_val > 1e-3:
                           mid_point = (p1+p2)/2
                           offset_vector = (p2-p1)/np.linalg.norm(p2-p1)
                           offset_vector = np.array([-offset_vector[1], offset_vector[0]]) # Roter 90 grader
                           text_pos = mid_point + offset_vector * (ax.get_ylim()[1] - ax.get_ylim()[0]) * 0.05
                           ax.text(text_pos[0], text_pos[1], f"{side_len_val:.2f}", color='darkred', ha='center', va='center', fontsize=9)


                    all_x = [c[0] for c in coords]
                    all_y = [c[1] for c in coords]
                    padding_x = (max(all_x) - min(all_x)) * 0.2 + 1
                    padding_y = (max(all_y) - min(all_y)) * 0.2 + 1
                    ax.set_xlim(min(all_x) - padding_x, max(all_x) + padding_x)
                    ax.set_ylim(min(all_y) - padding_y, max(all_y) + padding_y)
                else:
                    msg = "Ikke nok data til å tegne."
                    if isinstance(current_fig, Quadrilateral):
                        msg = "Plotting er ikke støttet for firkanter."
                    ax.text(0.5, 0.5, msg, transform=ax.transAxes, ha='center', va='center')
            
            plt.tight_layout(pad=2.0)
            plt.show()


# --- Kjører applikasjonen ---
# Denne linjen instansierer appen og viser den i output-cellen.
app = FormlikhetApp()
display(app)

FormlikhetApp(children=(HTML(value='<h2>Formlikhetskalkulator</h2>'), HBox(children=(IntSlider(value=3, descri…

In [8]:
# Delkapittel 3.9 Design og produktutvikling
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import re

# --- Globale Widgets ---
output = widgets.Output()
beregn_knapp = widgets.Button(description="Beregn", button_style='success', icon='check')

# --- Funksjoner for beregning og visualisering ---

def floor_div(a, b):
    return math.floor(a / b) if b > 0 else 0

def simulate_image_crop(orig_w, orig_h, new_w, new_h):
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_title("Visuell bildebeskjæring")
    ax.set_xlim(0, orig_w)
    ax.set_ylim(0, orig_h)
    ax.set_aspect('equal', adjustable='box')
    ax.add_patch(patches.Rectangle((0, 0), orig_w, orig_h, fill=False, edgecolor='blue', linewidth=2, label=f"Original: {int(orig_w)}x{int(orig_h)}"))
    ax.add_patch(patches.Rectangle((0, 0), new_w, new_h, fill=True, color='orange', alpha=0.6, label=f"Beskjært: {int(new_w)}x{int(new_h)}"))
    ax.legend()
    plt.gca().invert_yaxis()
    plt.show()

# --- Widget-oppsett ---

# Felles
lengde_widget = widgets.FloatText(description='Lengde:')
bredde_widget = widgets.FloatText(description='Bredde:')
hoyde_widget = widgets.FloatText(description='Høyde:')
radius_widget = widgets.FloatText(description='Radius:')
del_widget = widgets.FloatText(description='Del:')
hele_widget = widgets.FloatText(description='Hele:')
a_widget = widgets.FloatText(description='Verdi A (langside):')
b_widget = widgets.FloatText(description='Verdi B (kortside):')

# Spesifikke
form_widget = widgets.Dropdown(options=['Prisme', 'Sylinder'], description='Form:')
areal_form_widget = widgets.Dropdown(options=['Rektangel', 'Sirkel'], description='Form:')
container_l = widgets.FloatText(description='Container Lengde:')
container_b = widgets.FloatText(description='Container Bredde:')
container_h = widgets.FloatText(description='Container Høyde:')
item_l = widgets.FloatText(description='Eske Lengde:')
item_b = widgets.FloatText(description='Eske Bredde:')
item_h = widgets.FloatText(description='Eske Høyde:')
item_d = widgets.FloatText(description='Eske Diameter:')
item_form_widget = widgets.Dropdown(options=['Firkantet (prisme)', 'Rund (sylinder)'], description='Eskens form:')
img_w = widgets.IntText(description='Bildebredde (px):', value=3360)
img_h = widgets.IntText(description='Bildehøyde (px):', value=2520)
target_w = widgets.IntText(description='Mål-bredde:', value=21)
target_h = widgets.IntText(description='Mål-høyde:', value=30)
preserve_dim = widgets.Dropdown(options=['Behold bredde', 'Behold høyde'], description='Handling:', value='Behold høyde')

ramme_liste_widget = widgets.Textarea(
    value='10x15, 20x20, 24x30, 30x50, 21x30',
    placeholder='Skriv rammestørrelser, f.eks. 10x15, 20x20...',
    description='Rammestørrelser:',
    style={'description_width': 'initial'}
)

task_type = widgets.Dropdown(
    options=[
        ('Velg oppgavetype...', 'None'),
        ('Beste Bilderamme (Optimal Beskj.)', 'Beste Bilderamme'),
        ('Pakking av esker', 'Pakking av esker'),
        ('Bildeformat og beskjæring', 'Bildeformat og beskjæring'),
        ('Forhold / Standardformater', 'Forhold'),
        ('Areal', 'Areal'),
        ('Volum', 'Volum'),
        ('Prosent', 'Prosent')
    ],
    value='None',
    description='Oppgave:',
    style={'description_width': 'initial'}
)

input_container = widgets.VBox()

def update_inputs(change):
    task = change['new']
    widgets_to_show = []
    if task == 'Beste Bilderamme':
        widgets_to_show = [widgets.Label("Original bildestørrelse:"), img_w, img_h, widgets.HTML("<hr>"), ramme_liste_widget]
    elif task == 'Pakking av esker':
        widgets_to_show = [widgets.Label("Dimensjoner på stor container:"), container_l, container_b, container_h, widgets.HTML("<hr>"), widgets.Label("Dimensjoner på liten eske:"), item_form_widget]
        if item_form_widget.value == 'Firkantet (prisme)': widgets_to_show.extend([item_l, item_b, item_h])
        else: widgets_to_show.extend([item_d, item_h])
    elif task == 'Bildeformat og beskjæring':
        widgets_to_show = [widgets.Label("Originalt bilde:"), img_w, img_h, widgets.HTML("<hr>"), widgets.Label("Målformat:"), target_w, target_h, preserve_dim]
    elif task == 'Forhold':
        widgets_to_show = [a_widget, b_widget]
    elif task == 'Areal':
        widgets_to_show = [areal_form_widget]
        if areal_form_widget.value == 'Rektangel': widgets_to_show.extend([lengde_widget, bredde_widget])
        else: widgets_to_show.append(radius_widget)
    elif task == 'Volum':
        widgets_to_show = [form_widget]
        if form_widget.value == 'Prisme': widgets_to_show.extend([lengde_widget, bredde_widget, hoyde_widget])
        else: widgets_to_show.extend([radius_widget, hoyde_widget])
    elif task == 'Prosent':
        widgets_to_show = [del_widget, hele_widget]
    if widgets_to_show:
        widgets_to_show.append(beregn_knapp)
    input_container.children = tuple(widgets_to_show)
    with output: clear_output()

def on_button_clicked(b):
    with output:
        clear_output()
        try:
            task = task_type.value
            if task == 'Beste Bilderamme':
                # (Kode for beste bilderamme er uendret)
                orig_w, orig_h, original_area = float(img_w.value), float(img_h.value), float(img_w.value) * float(img_h.value)
                frame_strings, results = re.split(r'[,\n]', ramme_liste_widget.value), []
                for frame_str in frame_strings:
                    frame_str = frame_str.strip()
                    if 'x' in frame_str:
                        parts = frame_str.split('x')
                        try:
                            ratio_w, ratio_h = float(parts[0]), float(parts[1])
                            if ratio_w == 0 or ratio_h == 0: continue
                            area_lost1 = abs(original_area - (orig_w * (orig_w * (ratio_h / ratio_w))))
                            area_lost2 = abs(original_area - ((orig_h * (ratio_w / ratio_h)) * orig_h))
                            min_area_lost = min(area_lost1, area_lost2)
                            results.append((frame_str, min_area_lost, (min_area_lost / original_area) * 100))
                        except (ValueError, IndexError): continue
                results.sort(key=lambda x: x[1])
                print(f"Beste ramme for bilde på {int(orig_w)}x{int(orig_h)} piksler:\n")
                print(f"{'Rang':<5} {'Ramme':<10} {'Piksler tapt (areal)':<25} {'% av bildet tapt'}")
                print("-" * 60)
                for i, (frame, area, percent) in enumerate(results): print(f"{i+1:<5} {frame:<10} {int(area):<25} {percent:.1f}%")

            elif task == 'Pakking av esker':
                item_size_l = item_l.value if item_form_widget.value == 'Firkantet (prisme)' else item_d.value
                item_size_b = item_b.value if item_form_widget.value == 'Firkantet (prisme)' else item_d.value
                ant_l, ant_b, ant_h = floor_div(container_l.value, item_size_l), floor_div(container_b.value, item_size_b), floor_div(container_h.value, item_h.value)
                total_ant = ant_l * ant_b * ant_h
                print("Beregning for pakking:\n")
                print(f"Antall i lengden: floor({container_l.value} / {item_size_l}) = {ant_l}")
                print(f"Antall i bredden: floor({container_b.value} / {item_size_b}) = {ant_b}")
                print(f"Antall i høyden:  floor({container_h.value} / {item_h.value}) = {ant_h}")
                print("---------------------------------")
                print(f"Totalt antall esker som får plass: {ant_l} × {ant_b} × {ant_h} = {total_ant}\n")
                vol_container = container_l.value * container_b.value * container_h.value
                if item_form_widget.value == 'Firkantet (prisme)': vol_item = item_l.value * item_b.value * item_h.value
                else: vol_item = math.pi * (item_d.value / 2)**2 * item_h.value
                total_item_vol = total_ant * vol_item
                fyllingsgrad = (total_item_vol / vol_container) * 100 if vol_container > 0 else 0
                print(f"Volum av container: {vol_container:.2f} enheter³")
                print(f"Totalt volum av {total_ant} esker: {total_item_vol:.2f} enheter³")
                print(f"Fyllingsgrad (utnyttelse): {fyllingsgrad:.1f}%")
                # NYTT:
                print(f"Andel luft/tomrom: {100 - fyllingsgrad:.1f}%")

            elif task == 'Bildeformat og beskjæring':
                orig_w, orig_h, ratio_w, ratio_h = img_w.value, img_h.value, target_w.value, target_h.value
                if ratio_w == 0 or ratio_h == 0: print("Feil: Målformat kan ikke være null."); return
                if preserve_dim.value == 'Behold bredde':
                    new_w, new_h, pixels_cropped, dim_cropped = orig_w, orig_w * (ratio_h / ratio_w), orig_h - (orig_w * (ratio_h / ratio_w)), "høyden"
                else:
                    new_w, new_h, pixels_cropped, dim_cropped = orig_h * (ratio_w / ratio_h), orig_h, orig_w - (orig_h * (ratio_w / ratio_h)), "bredden"
                print(f"Originalt format: {orig_w} x {orig_h} piksler")
                print(f"Målformat: {ratio_w}:{ratio_h}")
                print(f"Handling: {preserve_dim.value}\n---------------------------------")
                print(f"Ny størrelse blir: {int(new_w)} x {int(new_h)} piksler.")
                print(f"Du må fjerne {int(abs(pixels_cropped))} piksler fra {dim_cropped}.")
                simulate_image_crop(orig_w, orig_h, new_w, new_h)

            elif task == 'Forhold':
                if b_widget.value == 0: print("Feil: Kan ikke dele på null."); return
                ratio = a_widget.value / b_widget.value
                print(f"Forholdet {a_widget.value} / {b_widget.value} = {ratio:.3f}\n")
                # NYTT:
                if 1.61 < ratio < 1.62:
                    print("Dette er veldig nært det gyldne snitt (φ ≈ 1.618).")
                elif 1.41 < ratio < 1.42:
                    print("Dette er veldig nært forholdet for A-serie papir (√2 ≈ 1.414).")

            elif task == 'Areal':
                if areal_form_widget.value == 'Rektangel': print(f"Areal av rektangel: {lengde_widget.value} × {bredde_widget.value} = {lengde_widget.value * bredde_widget.value:.2f} enheter²")
                else: print(f"Areal av sirkel: π × {radius_widget.value}² = {math.pi * radius_widget.value**2:.2f} enheter²")
            
            elif task == 'Volum':
                if form_widget.value == 'Prisme': print(f"Volum av prisme: {lengde_widget.value} × {bredde_widget.value} × {hoyde_widget.value} = {lengde_widget.value * bredde_widget.value * hoyde_widget.value:.2f} enheter³")
                else: print(f"Volum av sylinder: π × {radius_widget.value}² × {hoyde_widget.value} = {math.pi * radius_widget.value**2 * hoyde_widget.value:.2f} enheter³")

            elif task == 'Prosent':
                if hele_widget.value == 0: print("Feil: 'Hele' kan ikke være null."); return
                print(f"'{del_widget.value}' utgjør {(del_widget.value / hele_widget.value) * 100:.2f}% av '{hele_widget.value}'.")

        except Exception as e:
            print(f"En feil oppstod: {e}")

# --- Koble hendelser ---
task_type.observe(update_inputs, names='value')
for w in [areal_form_widget, form_widget, item_form_widget]:
    w.observe(lambda change: update_inputs({'new': task_type.value}), names='value')
beregn_knapp.on_click(on_button_clicked)

# --- Start GUI ---
display(widgets.VBox([task_type, input_container, output]))
update_inputs({'new': task_type.value})

# $\color{green}{\text{Kapittel 4 - Statistikk}}$

In [ ]:
# Finne median, gjennomsnitt, modus (altså typetall), variasjonsbredden, antallet tall i listen (frekvensen) 
# og standardavviket til en liste med tall
from statistics import median, mean, mode, stdev, StatisticsError

# Liste med tallverdier
liste = [9, 14, 5, 9, 7, 5, 27, 12, 3, 8, 14, 4, 10, 2, 6]               # Endre tallene i frekvenstabellen

# Beregner medianen og gjennomsnittet
medianen = round(median(liste), 2)
gjennomsnittet = round(mean(liste), 2)

# Forsøker å beregne typetallet og håndterer tilfeller der det ikke finnes et unikt typetall
try:
    typetallet = round(mode(liste), 2)
except StatisticsError:
    typetallet = "Ingen unik modus"

# Beregner variasjonsbredden
variasjonsbredden = max(liste) - min(liste)

# Beregner standardavviket
standardavviket = round(stdev(liste), 0)

# Beregner antallet tall i listen
antall_tall = len(liste)

# Skriver ut resultatene
print("Medianen er", medianen)
print("Gjennomsnittet er", gjennomsnittet)
print("Typetallet er", typetallet)
print("Variasjonsbredden er", variasjonsbredden)
print("Standardavviket er", standardavviket)
print("Antallet tall i listen er", antall_tall)

In [ ]:
# Søyle, sektor og linjediagram med 1 sett med frekvenser, Farger : https://matplotlib.org/stable/gallery/color/named_colors.html
import matplotlib.pyplot as plt

# Liste med navn og frekvenser
navn = ["Eple", "Pære", "Granateple", "Kiwi", "Banan"]        # Endre tallene så de passer til din oppgave
frekvenser = [20, 30, 10, 25, 15]                             # Endre tallene så de passer til din oppgave

# Fargevalg
farger = ['hotpink','coral','yellowgreen','b','g']

# Beregn totalfrekvensen
total = sum(frekvenser)

# Beregn andelene (i prosent) for hver sektor
andelene = [100 * frek / total for frek in frekvenser]

# Kakediagram
plt.pie(andelene, labels=navn, autopct='%1.1f%%', colors=farger)
plt.title("Oversikt over frukt")                                  # Endre overskriften 
plt.axis('equal')
plt.show()

# Søylediagram
plt.bar(navn, frekvenser, color=farger)
plt.title("Frekvenser av navn")                                  # Endre overskriften 
plt.xlabel("Frukt")                                              # Endre x-akse navnet
plt.ylabel("Frekvens")                                           # Endre y-akse navnet
plt.ylim(ymin=0)
plt.show()

# Linjediagram
plt.plot(navn, frekvenser, marker='o', color='#ff7f0e')
plt.title("Frekvenser av navn")                                  # Endre overskriften 
plt.xlabel("Navn")                                               # Endre x-akse navnet
plt.ylabel("Frekvens")                                           # Endre y-akse navnet
plt.ylim(ymin=0)
plt.show()

In [ ]:
# 4 Statistikk: 4.6 Statistiske beregninger
import pandas as pd
import matplotlib.pyplot as plt

# 1. Datatabell – rediger disse verdiene etter behov
data = {
    'Minutt': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Slag per minutt': [120, 150, 100, 130, 140, 100, 80, 120, 150, 110, 80]
}

df = pd.DataFrame(data)

# 2. Beregning av gjennomsnittspuls
average_pulse = df['Slag per minutt'].mean()
print(f"Gjennomsnittspuls: {average_pulse}")

# 3. Identifisering av hvileperioder (puls < 100)
rest_periods = df[df['Slag per minutt'] < 100]
print("Hvileperioder (puls < 100):")
print(rest_periods)

# 4. Linjediagram med markering av hvileperioder
plt.figure(figsize=(10, 6))
plt.plot(df['Minutt'], df['Slag per minutt'], marker='o', label='Pulse')
plt.scatter(rest_periods['Minutt'], rest_periods['Slag per minutt'], color='red', label='Hvileperioder (puls < 100)')
plt.xlabel('Minutt')
plt.ylabel('Slag per minutt')
plt.title('Utvikling av Edvards pulstakt over tid')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 4 Statistikk: 4.6 Statistiske beregninger
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np

# --- Funksjoner ---

def analyser_testresultater(filbane):
    df = pd.read_csv(filbane)
    fagdata = df.iloc[:, 1:]

    print("\n🎓 Statistisk analyse av kartleggingsresultater:")
    print("------------------------------------------------")
    print("Gjennomsnitt:\n", fagdata.mean())
    print("\nMedian:\n", fagdata.median())
    print("\nTypetall (modus):\n", fagdata.mode().iloc[0])
    print("\nStandardavvik:\n", fagdata.std())
    print("\nVarians:\n", fagdata.var())
    print("\nVariasjonsbredde:\n", fagdata.max() - fagdata.min())

    # Frekvenstabeller
    rounded = fagdata.round(-1)
    for kol in rounded.columns:
        print(f"\nFrekvenstabell for {kol}:\n{rounded[kol].value_counts().sort_index()}")

    # Diagrammer
    fagdata.mean().plot(kind='bar', title='Gjennomsnitt per fag')
    plt.ylabel('Poeng')
    plt.show()

    fagdata.iloc[0].plot(kind='pie', autopct='%1.1f%%', title='Fagfordeling for første elev')
    plt.ylabel('')
    plt.show()

    fagdata.T.plot(kind='line', title='Utvikling per fag (per elev)', marker='o')
    plt.xlabel('Fag')
    plt.ylabel('Resultat')
    plt.show()

    # Regresjonsanalyse for én elev (for enkelhets skyld: elev 1)
    fagnavn = fagdata.columns
    x = np.arange(len(fagnavn))
    y = fagdata.iloc[0].values
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

    print("\n📈 Regresjonsanalyse (elev 1 over fag):")
    print(f"Stigningstall: {slope:.2f}")
    print(f"Konstantledd: {intercept:.2f}")
    print(f"R² (forklaringsgrad): {r_value**2:.2f}")
    print(f"Standardfeil: {std_err:.2f}")
    print(f"95 % konfidensintervall for stigningstall:")
    ci_lower = slope - 1.96 * std_err
    ci_upper = slope + 1.96 * std_err
    print(f"  [{ci_lower:.2f}, {ci_upper:.2f}]")

    # Tegn regresjonslinje
    y_pred = slope * x + intercept
    plt.plot(fagnavn, y, 'o', label='Data (elev 1)')
    plt.plot(fagnavn, y_pred, '-', label='Regresjonslinje')
    plt.title('Regresjon av resultater (elev 1)')
    plt.ylabel('Poeng')
    plt.legend()
    plt.show()


def analyser_bysykkeldata(filbane):
    df = pd.read_csv(filbane)

    if 'FreeBikes' not in df.columns or 'DockingStation' not in df.columns:
        print("⚠️ Filen mangler nødvendige kolonner (FreeBikes, DockingStation).")
        return

    ledige = df[df['FreeBikes'] > 0]
    print("\n🚲 Analyse av bysykkelstasjoner:")
    print(f"Antall stasjoner med ledige sykler: {len(ledige)}")
    print("\nStasjoner med ledige sykler:")
    print(ledige[['DockingStation', 'FreeBikes']])

    ledige.set_index('DockingStation')['FreeBikes'].plot(kind='bar', title='Ledige sykler per stasjon')
    plt.ylabel('Antall sykler')
    plt.xlabel('Stasjon')
    plt.show()

# --- Meny ---

print("📊 Statistikkverktøy")
print("1: Kartleggingstest")
print("2: Bysykkeldata")

valg = input("Velg (1/2): ")

if valg == '1':
    fil1 = input("Filbane til testresultater (CSV):\n> ")
    analyser_testresultater(fil1)

elif valg == '2':
    fil2 = input("Filbane til bysykkeldata (CSV):\n> ")
    analyser_bysykkeldata(fil2)

else:
    print("Ugyldig valg.")

In [ ]:
# 4 Statistikk: 4.6 Statistiske beregninger
import pandas as pd
import matplotlib.pyplot as plt

# Filbane til din CSV-fil
filbane = r'C:\Users\hanska_a\OneDrive - Akershus fylkeskommune\0Drømtorp 2024-2025 Algebra og funksjoner\Matte\Test_kartlegging (1).csv'

# Les inn data
try:
    df = pd.read_csv(filbane)
except:
    df = pd.read_csv(filbane, sep=';')

# Fjern mellomrom i kolonnenavn hvis nødvendig
df.columns = df.columns.str.strip()

# Fagkolonner (alle unntatt Navn)
fagkolonner = df.columns[1:]

# Beregninger
df['Gjennomsnitt'] = df[fagkolonner].mean(axis=1)
df['Median'] = df[fagkolonner].median(axis=1)
df['Typetall'] = df[fagkolonner].mode(axis=1)[0]

# 📋 Skriv ut hele tabellen med alt inkludert
print("\n🧾 Resultater (inkludert statistiske mål):\n")
print(df.to_string(index=False))

# 📊 Lag grafer for hver elev
for i, rad in df.iterrows():
    navn = rad['Navn']
    fagverdier = rad[fagkolonner].astype(float)  # Sikrer riktig datatype

    print(f"\n📊 Diagrammer for {navn}")

    # Søyle
    fagverdier.plot(kind='bar', title=f'Søylediagram: {navn}', ylabel='Poeng', color='cornflowerblue')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Sektor
    plt.figure()
    fagverdier.plot.pie(title=f'Sektordiagram: {navn}', autopct='%1.1f%%')
    plt.ylabel('')
    plt.tight_layout()
    plt.show()

    # Linje
    fagverdier.plot(kind='line', title=f'Linjediagram: {navn}', marker='o', linestyle='-', color='green')
    plt.ylabel('Poeng')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# $\color{red}{\text{Kapittel 5 - Yrkesøkonomi}}$

In [ ]:
# 5 Yrkesøkonomi: 5.2 Merverdiavgift
# 5.2 Merverdiavgift beregning: Pris uten MVA = Pris med MVA/Vekstfaktoren 
def beregn_pris_uten_mva(pris_med_mva, vekstfaktor):
    return pris_med_mva / vekstfaktor

def beregn_pris_med_mva(pris_uten_mva, vekstfaktor):
    return pris_uten_mva * vekstfaktor

def beregn_vekstfaktor(pris_med_mva, pris_uten_mva):
    return pris_med_mva / pris_uten_mva

def velg_vekstfaktor():
    print("Velg MVA-sats:")
    print("1. 25% (For de fleste varer eller tjenester)")
    print("2. 15% (For mat og drikke)")
    print("3. 12% (For persontransport, kinobilletter og utleie av rom)")
    print("4. 0% (Helsetjenester, undervisningstjenester og kulturelle tjenester)")
    
    valg = input("Velg et alternativ (1/2/3/4): ").strip()
    if valg == '1':
        return 1.25
    elif valg == '2':
        return 1.15
    elif valg == '3':
        return 1.12
    elif valg == '4':
        return 1.00
    else:
        print("Ugyldig valg. Standard vekstfaktor 1.25 (25%) brukes.")
        return 1.25

def hovedprogram():
    while True:
        print("\nVelkommen til MVA kalkulator!")
        print("Trykk 'q' for å avslutte programmet.\n")
       
        print("Velg en beregning:")
        print("1. Pris uten MVA")
        print("2. Pris med MVA")
        print("3. Vekstfaktoren")

        choice = input("Velg et alternativ (1/2/3/q): ").strip().lower()
        if choice == 'q':
            print("Programmet avsluttes.")
            break

        try:
            if choice == '1':
                pris_med_mva = float(input("\nOppgi prisen med MVA: "))
                vekstfaktor = velg_vekstfaktor()
                pris_uten_mva = beregn_pris_uten_mva(pris_med_mva, vekstfaktor)
                resultat = f"\nPrisen uten MVA er {round(pris_uten_mva, 2)}"
            elif choice == '2':
                pris_uten_mva = float(input("\nOppgi prisen uten MVA: "))
                vekstfaktor = velg_vekstfaktor()
                pris_med_mva = beregn_pris_med_mva(pris_uten_mva, vekstfaktor)
                mva_belop = pris_med_mva - pris_uten_mva
                resultat = (
                    f"\nPrisen med MVA er {round(pris_med_mva, 2)}"
                    f"\nMerverdiavgiften utgjør {round(mva_belop, 2)}"
                )
            elif choice == '3':
                pris_med_mva = float(input("\nOppgi prisen med MVA: "))
                pris_uten_mva = float(input("Oppgi prisen uten MVA: "))
                vekstfaktor = beregn_vekstfaktor(pris_med_mva, pris_uten_mva)
                resultat = f"\nVekstfaktoren er {round(vekstfaktor, 2)}"
            else:
                resultat = "\nUgyldig valg. Vennligst oppgi 1, 2 eller 3."
        except ValueError:
            resultat = "\nUgyldig inndata. Vennligst oppgi et tall."

        print(resultat)

if __name__ == "__main__":
    hovedprogram()


Velkommen til MVA kalkulator!
Trykk 'q' for å avslutte programmet.

Velg en beregning:
1. Pris uten MVA
2. Pris med MVA
3. Vekstfaktoren


Velg et alternativ (1/2/3/q):  2

Oppgi prisen uten MVA:  200


Velg MVA-sats:
1. 25% (For de fleste varer eller tjenester)
2. 15% (For mat og drikke)
3. 12% (For persontransport, kinobilletter og utleie av rom)
4. 0% (Helsetjenester, undervisningstjenester og kulturelle tjenester)


Velg et alternativ (1/2/3/4):  1



Prisen med MVA er 250.0
Merverdiavgiften utgjør 50.0

Velkommen til MVA kalkulator!
Trykk 'q' for å avslutte programmet.

Velg en beregning:
1. Pris uten MVA
2. Pris med MVA
3. Vekstfaktoren


In [ ]:
# 5 Yrkesøkonomi: 5.3 Priskalkyler
# Priskalkyle Oppgave 7.133a:

materialkostnad_per_kort = 23         # Pris per takkekort
antall_kort = 20                       # Antall takkekort kunden ønsker
indirekte_kostnader = 1100            # Faste kostnader som strøm, husleie osv.
fortjeneste_prosent = 0.41            # Ønsket fortjeneste (41 %)
mva = 0.25                             # Merverdiavgift (25 %)

# Lønnskostnader
timelønn = 350                         # Grunnlønn per time
overtidssats = timelønn * 1.25         # Overtidstillegg (25 % ekstra)
timer_fotografering = 3               # Antall timer brukt på fotografering
timer_redigering = 5                  # Antall timer brukt på redigering og etterarbeid

# Beregner lønnskostnader
lønn_fotografering = timer_fotografering * overtidssats
lønn_redigering = timer_redigering * timelønn
total_lønn = lønn_fotografering + lønn_redigering

# Beregner materialkostnader
total_materialkostnad = materialkostnad_per_kort * antall_kort

# Beregner selvkost (alle kostnader før fortjeneste og mva)
selvkost = total_materialkostnad + total_lønn + indirekte_kostnader

# Legger til fortjeneste
pris_med_fortjeneste = selvkost * (1 + fortjeneste_prosent)

# Legger til merverdiavgift
sluttpris = pris_med_fortjeneste * (1 + mva)

# Skriver ut en oversikt over alle kostnader og sluttpris
print("Kostnadsoversikt:")
print(f"Materialkostnader: {total_materialkostnad} kr")
print(f"Lønnskostnader: {total_lønn} kr")
print(f"Indirekte kostnader: {indirekte_kostnader} kr")
print(f"Selvkost: {selvkost} kr")
print(f"Pris med fortjeneste: {pris_med_fortjeneste:.2f} kr")
print(f"Sluttpris inkl. mva: {round(sluttpris)} kr")

In [ ]:
# 5 Yrkesøkonomi: 5.3 Priskalkyler
import re

# Evaluerer trygge matematiske uttrykk
def evaluer_uttrykk(uttrykk):
    uttrykk = uttrykk.replace(' ', '')
    if re.match(r'^[0-9\.\*\+\-/()]+$', uttrykk):
        return eval(uttrykk)
    else:
        raise ValueError("Ugyldig uttrykk")

# Henter og validerer kostnader og lønn
def hent_kostnader():
    print("\nOppgi kostnader – (skriv 'q' for å avslutte):")
    while True:
        try:
            direkte_input = input("Direkte kostnader/Materialkostnader (uten mva.) (feks råvarer): ").strip().lower()
            if direkte_input == 'q':
                return None
            direkte = evaluer_uttrykk(direkte_input)
            if direkte < 0:
                raise ValueError

            indirekte_input = input("Indirekte kostnader (feks husleie, strøm): ").strip().lower()
            if indirekte_input == 'q':
                return None
            indirekte = evaluer_uttrykk(indirekte_input)
            if indirekte < 0:
                raise ValueError

            # NYTT: Velg metode for lønnskostnader
            print("\nVil du oppgi lønnskostnader direkte, eller beregne dem?")
            lønn_valg = input("Skriv 'j' for å oppgi direkte lønnskostnader, eller 'n' for å beregne ut fra timelønn og minutter brukt: ").strip().lower()
            if lønn_valg == 'q':
                return None

            if lønn_valg == 'j':
                lønn_input = input("Oppgi lønnskostnader direkte (kr): ").strip().lower()
                if lønn_input == 'q':
                    return None
                lønn = evaluer_uttrykk(lønn_input)
                if lønn < 0:
                    raise ValueError

            elif lønn_valg == 'n':
                timelønn_input = input("Oppgi timelønn (kr/t): ").strip().lower()
                if timelønn_input == 'q':
                    return None
                timelønn = evaluer_uttrykk(timelønn_input)
                if timelønn < 0:
                    raise ValueError

                minutter_input = input("Hvor mange minutter er brukt av timen til arbeidet? ").strip().lower()
                if minutter_input == 'q':
                    return None
                minutter = evaluer_uttrykk(minutter_input)
                if minutter < 0:
                    raise ValueError

                lønn = timelønn * (minutter / 60)

            else:
                print("Ugyldig valg. Skriv 'j' for ja, 'n' for nei, eller 'q' for å avslutte.")
                continue

            return direkte + indirekte + lønn

        except ValueError:
            print("Ugyldig inndata. Vennligst oppgi et gyldig POSITIVT tall eller uttrykk – eller 'q' for å avslutte.")


def velg_vekstfaktor():
    print("\nVelg MVA-sats (skriv 'q' for å avslutte):")
    print("1. 25% (For de fleste varer eller tjenester)")
    print("2. 15% (Næringsmidler, altså mat og drikke)")
    print("3. 12% (For persontransport, kinobilletter, Inngangsbilletter til museer, gallerier, o.l., overnatting og utleie av rom)")
    print("4. 0% (Helsetjenester, undervisningstjenester og kulturelle tjenester)")
    
    while True:
        valg = input("Velg et alternativ (1/2/3/4): ").strip().lower()
        if valg == 'q':
            return None
        vekstfaktorer = {
            '1': 1.25,
            '2': 1.15,
            '3': 1.12,
            '4': 1.00
        }
        if valg in vekstfaktorer:
            return vekstfaktorer[valg]
        else:
            print("Ugyldig valg. Velg 1, 2, 3 eller 4 – eller 'q' for å avslutte.")

def beregn_pris():
    selvkost = hent_kostnader()
    if selvkost is None:
        print("\nAvslutter programmet. Takk for at du brukte priskalkulatoren!")
        return True  # signaliser at programmet skal avsluttes

    # Hvis selvkost er 0, gi mulighet til å skrive det inn manuelt
    if selvkost == 0:
        svar = input("\nSelvkost er 0 kr. Vil du skrive inn en selvkost manuelt? (j/n): ").strip().lower()
        if svar == 'j':
            while True:
                manuelt_input = input("Oppgi ønsket selvkost (kr): ").strip().lower()
                if manuelt_input == 'q':
                    print("Avslutter beregningen. Takk for at du brukte priskalkulatoren!")
                    return True
                try:
                    manuelt_beløp = evaluer_uttrykk(manuelt_input)
                    if manuelt_beløp > 0:
                        selvkost = manuelt_beløp
                        break
                    else:
                        print("Selvkost må være et positivt tall.")
                except ValueError:
                    print("Ugyldig verdi. Prøv igjen.")
        else:
            print("Avslutter beregningen. Takk for at du brukte priskalkulatoren!")
            return True

    # Spør om bruker vil gå videre
    svar = input(f"\nSelvkost er beregnet til {selvkost:.2f} kr. Vil du gå videre og legge til fortjeneste? (j/n): ").strip().lower()
    if svar != 'j':
        print("Avslutter beregningen. Takk for at du brukte priskalkulatoren!")
        return True

    while True:
        try:
            fortjeneste_input = input("\nØnsket fortjeneste i prosent (%): ").strip().lower()
            if fortjeneste_input == 'q':
                print("\nAvslutter programmet. Takk for at du brukte priskalkulatoren!")
                return True
            fortjeneste_prosent = evaluer_uttrykk(fortjeneste_input)
            if fortjeneste_prosent < 0:
                print("Fortjenesteprosent kan ikke være negativ.")
                continue
            break
        except ValueError:
            print("Ugyldig inndata. Vennligst oppgi et positivt tall eller 'q' for å avslutte.")

    fortjeneste = selvkost * (fortjeneste_prosent / 100)
    pris_uten_mva = selvkost + fortjeneste

    vekstfaktor = velg_vekstfaktor()
    if vekstfaktor is None:
        print("\nAvslutter programmet. Takk for at du brukte priskalkulatoren!")
        return True

    pris_med_mva = pris_uten_mva * vekstfaktor
    mva_beløp = pris_med_mva - pris_uten_mva

    print("\n--- Priskalkyle ---")
    print(f"Summen av kostnader/selvkosten er: {selvkost:.2f} kr")
    print(f"Fortjenesten i ({fortjeneste_prosent}%) er: {fortjeneste:.2f} kr")
    print(f"Prisen uten merverdiavgift er: {pris_uten_mva:.2f} kr")
    print(f"Merverdiavgiften er: {mva_beløp:.2f} kr")
    print(f"Prisen med merverdiavgift er: {pris_med_mva:.2f} kr")

    return False  # fortsett programmet

def hovedmeny():
    avslutt = False
    while not avslutt:
        avslutt = beregn_pris()
        if not avslutt:
            while True:
                igjen = input("\nVil du gjøre en ny beregning? (j = ja / q = avslutt): ").strip().lower()
                if igjen == 'j':
                    break
                elif igjen == 'q':
                    print("Takk for at du brukte priskalkulatoren!")
                    avslutt = True
                    break
                else:
                    print("Ugyldig valg. Skriv 'j' for ja eller 'q' for avslutt.")

# Start programmet
hovedmeny()

In [ ]:
# 5 Yrkesøkonomi: 5.5 Anbud
def calculate_bid():
    while True:
        print("\n--- Anbudsberegning ---")
        avslutt = input("Trykk 'q' for å avslutte eller Enter for å fortsette: ")
        if avslutt.lower() == 'q':
            print("Programmet er avsluttet.")
            break

        try:
            innkjopssum_uten_mva = eval(input("Innkjøpssum uten mva (f.eks. 5*100): "))
            fortjeneste_prosent = eval(input("Fortjeneste i prosent (f.eks. 80): "))
            lonnskostnader = eval(input("Lønnskostnader (f.eks. 10*500): "))
            mva_sats = eval(input("Merverdiavgift i prosent (f.eks. 25): "))

            fortjeneste = innkjopssum_uten_mva * (fortjeneste_prosent / 100)
            utsalgssum = innkjopssum_uten_mva + fortjeneste
            pris_uten_mva = utsalgssum + lonnskostnader
            mva_belop = pris_uten_mva * (mva_sats / 100)
            pris_med_mva = pris_uten_mva + mva_belop
            inntekt_til_bedriften = pris_uten_mva - innkjopssum_uten_mva

            print("\n--- Resultat ---")
            print(f"Utsalgssum (materialer, råvarer): {utsalgssum:.2f} kr")
            print(f"Pris uten merverdiavgift: {pris_uten_mva:.2f} kr")
            print(f"Merverdiavgift: {mva_belop:.2f} kr")
            print(f"Pris med merverdiavgift: {pris_med_mva:.2f} kr")
            print(f"Inntekten til bedriften: {inntekt_til_bedriften:.2f} kr")

            print("\nUlemper med for lavt anbud:")
            print("- Risiko for underskudd")
            print("- Manglende dekning av uforutsette kostnader")
            print("- Redusert kvalitet på arbeidet")

            print("\nUlemper med for høyt anbud:")
            print("- Risiko for å miste kunden")
            print("- Redusert konkurranseevne")
            print("- Oppfattes som overpriset")

        except Exception as e:
            print(f"Feil i input: {e}. Prøv igjen.")

# Kjør funksjonen i en celle i Jupyter Notebook
calculate_bid()

In [ ]:
# 5 Yrkesøkonomi: 5.6 Velferdsteknologi
STANDARD_TIMELØNN = 340

def spør_om_timelønn():
    svar = input("Vil du oppgi timelønn? (ja/nei): ").strip().lower()
    if svar == 'ja':
        return float(input("Skriv inn timelønn (kr): "))
    else:
        print(f"Standard timelønn brukes: {STANDARD_TIMELØNN} kr")
        return STANDARD_TIMELØNN

def beregn_sykefravær_besparelse(tidligere_prosent, ny_prosent, årsverk, timer_per_årsverk, timelønn):
    totale_timer = årsverk * timer_per_årsverk
    forskjell_i_prosent = (tidligere_prosent - ny_prosent) / 100
    sparte_timer = forskjell_i_prosent * totale_timer
    sparte_kroner = sparte_timer * timelønn
    return round(sparte_timer), round(sparte_kroner)

def beregn_investering_lønnsomhet(antall_pasienter, kostnad_per_enhet, spart_tid_per_dag, årslønn, arbeidstimer_per_år):
    gjennomsnittlig_timelønn = årslønn / arbeidstimer_per_år
    total_spart_tid_per_år = (spart_tid_per_dag / 60) * 365 * antall_pasienter
    totale_besparelser_per_år = total_spart_tid_per_år * gjennomsnittlig_timelønn
    total_investering_kostnad = antall_pasienter * kostnad_per_enhet
    netto_besparelser = totale_besparelser_per_år - total_investering_kostnad
    return netto_besparelser

def beregn_tid_og_kostnad(besøk_per_uke, minutter_spart_per_besøk, timelønn, låskostnad):
    besøk_per_år = besøk_per_uke * 52
    total_spart_tid_minutter = besøk_per_år * minutter_spart_per_besøk
    total_spart_tid_timer = total_spart_tid_minutter / 60
    spart_kroner = total_spart_tid_timer * timelønn
    netto_besparelse = spart_kroner - låskostnad
    return round(total_spart_tid_timer, 2), round(spart_kroner, 2), round(netto_besparelse, 2)

def beregn_tilbakebetalingstid(besøk_per_uke, minutter_spart_per_besøk, timelønn, låskostnad):
    spart_per_uke = (minutter_spart_per_besøk / 60) * besøk_per_uke * timelønn
    if spart_per_uke == 0:
        return float('inf')
    return round(låskostnad / spart_per_uke)

def beregn_dusjtoalett_besparelse(antall_beboere, andel_med_behov, minutter_spart_per_dag, dager_per_år):
    brukere_med_behov = antall_beboere * andel_med_behov
    timer_spart_per_dag = (minutter_spart_per_dag / 60) * brukere_med_behov
    timer_spart_per_år = timer_spart_per_dag * dager_per_år
    return round(timer_spart_per_år, 2)

def beregn_tilbakebetalingstid_dusjtoalett(totalkostnad, timer_spart_per_år, timelønn):
    spart_kroner_per_år = timer_spart_per_år * timelønn
    if spart_kroner_per_år == 0:
        return float('inf'), 0
    år = totalkostnad / spart_kroner_per_år
    måneder = år * 12
    return round(måneder, 1), round(spart_kroner_per_år, 2)

def maks_toaletter_for_nullresultat(spart_kroner_per_år, kostnad_per_toalett):
    if kostnad_per_toalett == 0:
        return float('inf')
    maks_toaletter = spart_kroner_per_år // kostnad_per_toalett
    return int(maks_toaletter)

def beregn_generell_investering(antall_enheter, kostnad_per_enhet, spart_timer_per_år, timelønn):
    investering = antall_enheter * kostnad_per_enhet
    besparelse = spart_timer_per_år * timelønn
    netto = besparelse - investering
    tilbakebetalingstid = investering / besparelse if besparelse > 0 else float('inf')
    return investering, besparelse, netto, tilbakebetalingstid

def hovedprogram():
    while True:
        print("\nVelg beregningstype:")
        print("1. Beregn besparelse ved redusert sykefravær")
        print("2. Beregn lønnsomhet ved investering i velferdsteknologi")
        print("3. Beregn besparelse og tilbakebetalingstid for elektronisk dørlås")
        print("4. Beregn besparelse og lønnsomhet for dusjtoaletter")
        print("5. Generell investering og besparelsesanalyse (f.eks. løfteheis)")
        print("Trykk 'q' for å avslutte.")
        valg = input("Ditt valg: ")

        if valg == 'q':
            print("Avslutter programmet.")
            break

        elif valg == '1':
            tidligere_prosent = float(input("Tidligere sykefraværsprosent: "))
            ny_prosent = float(input("Ny sykefraværsprosent: "))
            årsverk = int(input("Antall årsverk: "))
            timer_per_årsverk = int(input("Timer per årsverk: "))
            timelønn = spør_om_timelønn()

            sparte_timer, sparte_kroner = beregn_sykefravær_besparelse(
                tidligere_prosent, ny_prosent, årsverk, timer_per_årsverk, timelønn
            )
            print(f"\nSparte timer per år: {sparte_timer}")
            print(f"Sparte kroner per år: {sparte_kroner} kr")

        elif valg == '2':
            antall_pasienter = int(input("Antall pasienter: "))
            kostnad_per_enhet = float(input("Kostnad per enhet (kr): "))
            spart_tid_per_dag = float(input("Spart tid per dag per pasient (minutter): "))
            årslønn = float(input("Årslønn per ansatt (kr): "))
            arbeidstimer_per_år = float(input("Antall arbeidstimer per år: "))

            netto_besparelser = beregn_investering_lønnsomhet(
                antall_pasienter, kostnad_per_enhet, spart_tid_per_dag, årslønn, arbeidstimer_per_år
            )

            if netto_besparelser > 0:
                print(f"\nInvesteringen er lønnsom. Netto besparelser i løpet av ett år: {netto_besparelser:.2f} kr")
            else:
                print(f"\nInvesteringen er ikke lønnsom. Netto tap i løpet av ett år: {abs(netto_besparelser):.2f} kr")

        elif valg == '3':
            besøk_per_uke = int(input("Antall ukentlige besøk: "))
            minutter_spart_per_besøk = float(input("Tid spart per besøk (minutter): "))
            timelønn = spør_om_timelønn()
            låskostnad = float(input("Kostnad for elektronisk dørlås (kr): "))

            timer_spart, kroner_spart, netto = beregn_tid_og_kostnad(
                besøk_per_uke, minutter_spart_per_besøk, timelønn, låskostnad
            )
            uker_tilbakebetaling = beregn_tilbakebetalingstid(
                besøk_per_uke, minutter_spart_per_besøk, timelønn, låskostnad
            )

            print(f"\nÅrlig spart tid: {timer_spart} timer")
            print(f"Årlig spart beløp: {kroner_spart:.2f} kr")
            print(f"Netto besparelse første år (etter låskostnad): {netto:.2f} kr")
            print(f"Antall uker før investeringen er spart inn: {uker_tilbakebetaling} uker")

        elif valg == '4':
            antall_beboere = int(input("Antall beboere: "))
            andel_med_behov = float(input("Andel med behov (f.eks. 0.6 for 60%): "))
            minutter_spart_per_dag = float(input("Minutter spart per dag per bruker: "))
            dager_per_år = 365
            antall_toaletter = int(input("Antall toaletter som skal byttes: "))
            kostnad_per_toalett = float(input("Kostnad per dusjtoalett (kr): "))
            timelønn = spør_om_timelønn()

            timer_spart_per_år = beregn_dusjtoalett_besparelse(
                antall_beboere, andel_med_behov, minutter_spart_per_dag, dager_per_år
            )
            totalkostnad = antall_toaletter * kostnad_per_toalett
            måneder, spart_kroner_per_år = beregn_tilbakebetalingstid_dusjtoalett(
                totalkostnad, timer_spart_per_år, timelønn
            )
            maks_toaletter = maks_toaletter_for_nullresultat(spart_kroner_per_år, kostnad_per_toalett)

            print(f"\nÅrlig spart tid: {timer_spart_per_år} timer")
            print(f"Årlig spart beløp: {spart_kroner_per_år:.2f} kr")
            print(f"Tid før investeringen er spart inn: {måneder} måneder")
            print(f"Maksimalt antall toaletter som kan kjøpes for å gå i null på ett år: {maks_toaletter}")

        elif valg == '5':
            print("\nGenerell investering – for eksempel løfteheis")

            antall_enheter = int(input("Antall enheter som skal kjøpes (f.eks. 2 løfteheiser): "))
            kostnad_per_enhet = float(input("Kostnad per enhet (kr): "))
            spart_timer_per_år = float(input("Forventet spart tid totalt per år (timer): "))
            timelønn = spør_om_timelønn()

            investering, besparelse, netto, tilbakebetalingstid = beregn_generell_investering(
                antall_enheter, kostnad_per_enhet, spart_timer_per_år, timelønn
            )

            print(f"\nTotal investering: {investering:.2f} kr")
            print(f"Forventet årlig besparelse: {besparelse:.2f} kr")
            print(f"Netto resultat etter ett år: {'+' if netto >= 0 else ''}{netto:.2f} kr")
            print(f"Tilbakebetalingstid: {tilbakebetalingstid:.2f} år")

        else:
            print("Ugyldig valg. Prøv igjen.")

if __name__ == "__main__":
    hovedprogram()